In [ ]:
# --- CSS STYLE ---
from IPython.core.display import HTML
def css_styling():
    styles = open("../input/kaggle-data-science-survey-20172021/style.css", "r").read()
    return HTML("<style>"+styles+"</style>")
css_styling()

<center><img src="https://i.imgur.com/iOHRYxA.png"></center>
<center><h1 style="font-family: parklane">How are the 💃Ladies and the 🎩Gents doing?</h1></center>
<center><h3 style="font-family: times-new-roman; font-size: 1.728em">- An analysis of parties in comparison and in time -</h3></center>

# Foreword

<p style="font-family: times-new-roman">This notebook is a celebration of both Ladies and Gentlemen, a deep dive into what is their evolution, some differences, similarities, and the overall story of their journey.</p>
<p style="font-family: times-new-roman">This notebook will not be a showcase of why or why not the numbers for women don't match with the ones for men. Rather, this will be a celebration of differences and an homage for both genders and their own pace and beauty unfolding into Data Science. In the end, we are all people.</p>
<p style="font-family: times-new-roman">So, if you are curious and you dare ...</p>
<p style="font-family: times-new-roman"><b>🎉 It's a party! 🎉</b></p>

# Methodology

<p style="font-family: times-new-roman">This year I have decided to take a 2-dimensional look into things: A first look regarding how 💃Ladies and 🎩Gents are conducting their Data Sciencing, as well as a second look through 🕒time and how it has impacted the preferences and overall way we do Data Science.</p>

<p style="font-family: times-new-roman">Hence, I have taken all Kaggle Data Science Surveys starting 2021 and went back in time up until 2017, gathering as much information I could for each year. Hence, I ended up with a table of 106,000 rows and 293 columns, containing aggregated information for 5 years: from 2021 to 2017 inclusively.</p>

<p style="font-family: times-new-roman">The aggregation was done <i>manually</i>, as the order of the columns, the naming of the questions as well as methodology changed from one year to another.</p>

> <p style="font-family: times-new-roman">📌 <b>Note</b>: There have been years where some questions or choices have not been provided. Hence, I matched them as accurately as possible and, to keep everything clean and compare in a precise manner, I am going to <b>compute the proportions on year and gender</b> instead of overall numbers.</p>

## 📖 Libraries

In [ ]:
import os
import pandas as pd
import numpy as np
import imageio
import wandb
from PIL import Image
from IPython.core.display import display, HTML, Javascript
import IPython.display as py_display
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

os.environ["WANDB_SILENT"] = "true"
CONFIG = {'competition': 'kaggle-survey-2021', '_wandb_kernel': 'aot'}

# Secrets
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("wandb")

# Log in W&B personal API
! wandb login $secret_value_0

## 📖 Functions & Helpers

In [ ]:
# =============== FUNCTIONS ===============
def save_dataset_artifact(run_name, artifact_name, path):
    '''Saves dataset to W&B Artifactory.
    run_name: name of the experiment
    artifact_name: under what name should the dataset be stored
    path: path to the dataset'''
    
    run = wandb.init(project='kaggle-survey-2021', 
                     name=run_name, 
                     config=CONFIG, anonymous="allow")
    artifact = wandb.Artifact(name=artifact_name, 
                              type='dataset')
    artifact.add_file(path)

    wandb.log_artifact(artifact)
    wandb.finish()
    
    
def create_wandb_plot(x_data=None, y_data=None, x_name=None, y_name=None, 
                      title=None, log=None, plot="line"):
    '''Create and save lineplot/barplot in W&B Environment.
    x_data & y_data: Pandas Series containing x & y data
    x_name & y_name: strings containing axis names
    title: title of the graph
    log: string containing name of log'''
    
    data = [[label, val] for (label, val) in zip(x_data, y_data)]
    table = wandb.Table(data=data, columns = [x_name, y_name])
    
    if plot == "line":
        wandb.log({log : wandb.plot.line(table, x_name, y_name, title=title)})
    elif plot == "bar":
        wandb.log({log : wandb.plot.bar(table, x_name, y_name, title=title)})
    elif plot == "scatter":
        wandb.log({log : wandb.plot.scatter(table, x_name, y_name, title=title)})
        
        
def create_wandb_hist(x_data=None, x_name=None, title=None, log=None):
    '''Create and save histogram in W&B Environment.
    x_data: Pandas Series containing x values
    x_name: strings containing axis name
    title: title of the graph
    log: string containing name of log'''
    
    data = [[x] for x in x_data]
    table = wandb.Table(data=data, columns=[x_name])
    wandb.log({log : wandb.plot.histogram(table, x_name, title=title)})
    
    
def wb_prep_data(df):
    # First 2 columns: Year and Gender
    # The rest: Categories

    df = df[df["Year"]==2021]

    df = df.melt(id_vars=["Year", "Gender"],
                 value_vars=list(df.columns[2:]),
                 value_name="Count")
    df.columns = ["Year", "Gender", "Category", "Count"]
    man = df[df["Gender"] == "Man"]
    woman = df[df["Gender"] == "Woman"]
    return man, woman


def get_data_multiple_answers(cols, new_names):
    '''Select and prep data for the columns mentiones.
    It contains year & gender data as well.
    
    cols: must be ar array of valid names from 1 question
    new_names: the new names these coluns should have
    return: new prepped data (in percentages per year & gender)'''
    
    cols.extend(["Year", "What is your gender? - Selected Choice"])
    new_names.extend(["Year", "Gender"])

    data = df[cols]
    data.columns = new_names
    data = data[data["Gender"].isin(["Man", "Woman"])]

    data = data.groupby(["Year", "Gender"]).count().reset_index()
    data = pd.merge(data, total, on=["Year", "Gender"])

    cols_perc = list(data.columns[2:-1])
    for c in cols_perc:
        data[c] = data[c]/data["Count"] * 100
        data[c] = data[c].apply(lambda x: round(x, 1))
    data.drop(columns="Count", inplace=True)
    
    return data
    
    
def adjust_amount(data, amount_column):
    '''Takes in dataset and calculates the adjusted Amount (in dollars) per country.
    data: a dataframe containing Year, Gender, Country and Amount column
    amount_column: name of the column to be adjusted (e.g.: 1,000-9,000)'''
    
    data = data[data["Country"]!='Other'].reset_index(drop=True)

    # Compute Middle Pay
    data['lower'] = data[amount_column].apply(lambda x: int(x.split('-')[0].replace(',', '')))
    data['upper'] = data[amount_column].apply(lambda x: int(x.split('-')[1].replace(',', '')) 
                                        if x.split('-')[0] != '500,000' 
                                        else 1000000)
    data['middle'] = -(-(data['upper'] + data['lower']) // 2)

    # Get Cost of Living data
    cost_living = pd.read_csv("../input/2020-cost-of-living/cost of living 2020.csv")
    cost_living = cost_living[cost_living["McMeal($)"]>0]

    # Merge the info
    data = data.replace({'United States of America':'United States',
                         'Viet Nam':'Vietnam',
                         'United Kingdom of Great Britain and Northern Ireland' : 'United Kingdom',
                         'Iran, Islamic Republic of...' : 'Iran',
                         'Republic of Korea' : 'South Korea'})

    data = pd.merge(left=data, right=cost_living[['Country', 'McMeal($)']], how='inner', on='Country')

    # Compute Adjusted Amount
    data["Amount"] = -(-data['middle'] // data['McMeal($)']).astype(int)

    data = data.drop(columns=["lower", "upper", "middle", "McMeal($)", "Country"])
    
    return data

    
# =============== PREP ===============
# Read in the dataframe
df = pd.read_csv("../input/kaggle-data-science-survey-20172021/kaggle_survey_2017_2021.csv",
                 skiprows=1)

# Clean gender column
col = "What is your gender? - Selected Choice"
df[col] = df[col].replace(["Male", "Female", "Prefer to self-describe", "Prefer not to say", "A different identity", 
                         "Non-binary, genderqueer, or gender non-conforming"], 
                        ["Man", "Woman", "Unspecified", "Unspecified", "Unspecified", "Nonbinary"])

# Total
total = df[["Year", "What is your gender? - Selected Choice"]].value_counts().reset_index()
total.columns = ["Year", "Gender", "Count"]

# save_dataset_artifact(run_name="save_full_data", 
#                       artifact_name="kaggle_survey_2017_2021", 
#                       path="../input/kaggle-data-science-survey-20172021/kaggle_survey_2017_2021.csv")
# wandb.finish()

# # Save to W&B
# run = wandb.init(project='kaggle-survey-2021', name='numbers_eda', 
#                  config=CONFIG, anonymous="allow")
# wandb.log({"2017-2021 rows" : int(df.shape[0]),
#            "2017-2021 cols" : int(df.shape[1])})
# wandb.finish()

<div class="alert success-alert" style="font-family: times-new-roman">
  <h4 style="font-family: times-new-roman"><center>👀 Don't forget to <b>hover</b> your mouse over the charts and <b>interact</b> with them!</center></h4>
</div>

# 1. Ladies & Gentlemen, the floor is open!

> <p style="font-family: times-new-roman"> 🎷 Did you know? 1920s in North America are frequently referred to as the "Roaring Twenties" or the "Jazz Age", while in Europe the period is sometimes referred to as the "Golden Twenties" because of the economic boom following World War I. French speakers refer to the period as the "Années folles" ("Crazy Years"), emphasizing the era's social, artistic, and cultural dynamism (<a href="https://en.wikipedia.org/wiki/1920s">source here</a>).</p>

<center><img src="https://i.imgur.com/nGNgeJw.png" width=800></center>

## 1.1 How many came to the party?

<p style="font-family: times-new-roman">The golden ratio, 80:20.</p>

<p style="font-family: times-new-roman">Throughout the years, no matter the number of overall respondents answering the survey, the percentage of women to men was kept almost the same. Hence, we are looking at a pool of around <b>80% 🎩men and almost 20% 💃women</b>.</p>

<p style="font-family: times-new-roman">We also see that we had the most answers in 2021! Not only is Kaggle becoming more and more popular, but the survey is too!</p>

In [ ]:
col = "What is your gender? - Selected Choice"

gender = df[df[col].isin(["Man", "Woman"])][["Year", col]]
gender.columns = ["Year", "Gender"]

# Gender and Years
gender1 = gender.value_counts().reset_index()
gender1 = pd.pivot(data=gender1, index="Year", columns="Gender", values=0).reset_index()

# Just Gender
gender2 = gender["Gender"].value_counts().reset_index()
gender2.columns = ["Gender", "Count"]
gender2["Gender"] = gender2["Gender"].replace(["Man", "Woman"],
                                              ["Gentlemen", "Ladies"])

gender1.to_csv("gender1.csv", index=False)
gender2.to_csv("gender2.csv", index=False)

# Save PNGs
im = imageio.imread('../input/kaggle-data-science-survey-20172021/images/man.png')
Image.fromarray(im).save("man.png")

im = imageio.imread('../input/kaggle-data-science-survey-20172021/images/lady.png')
Image.fromarray(im).save("lady.png")

im = imageio.imread('../input/kaggle-data-science-survey-20172021/images/bkg.jpg')
Image.fromarray(im).save("bkg.jpg")


htmlt = '''
<!-- Bootstrap -->
<link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.4.1/css/bootstrap.min.css" integrity="sha384-Vkoo8x4CGsO3+Hhxv8T/Q5PaXtkKtu6ug5TOeNV6gBiFeWPGFN9MuhOf23Q9Ifjh" crossorigin="anonymous">
<!-- Style -->
<style>
	.all {
		font-family: "Times New Roman", Times, serif;
		overflow-x: hidden;
		width: 98%;
		background-color:#e0c9a6;
		/* background-image : url("bkg.jpg"); */
	}
	.svg-container {
		display: inline-block;
		position: relative;
		width: 100%;
		padding-bottom: 100%;
		vertical-align: top;
		overflow: hidden;
	}
	.svg-content {
		display: inline-block;
		position: absolute;
		top: 0;
		left: 0;
	}

	.axisHidden path{
	stroke: #e0c9a6;
	}

	.axisHidden line{
	stroke: #e0c9a6;
	}

	.annotation.red text {
		fill: #7D262F;
	}
	.annotation.blue text {
		fill: #184E8B;
	}

	@font-face {
		font-family: parklane;
		src: url(https://h4ks.net/ParkLaneNF.otf);
		}

	div.tooltip-donut {
		position: absolute;
		text-align: center;
		padding: .3rem;
		background: #FFFFFF;
		color: #000000;
		border: 1px solid #000000;
		border-radius: 8px;
		pointer-events: none;
		font-size: 2rem;
		z-index: 99999;
	}

	/* Hackity Hack */
	body {
		position: relative;
		/* background: url("./bkg.jpg") !important; */
	}

	.cell.text_cell, 
	.jp-Cell.jp-MarkdownCell {
		background: rgba(255, 255, 255, 0.2);
	}

	.cell.text_cell > .inner_cell {
		padding-right: 12.5ex;
	}
    
	@media (max-width: 540px) {
		.cell.text_cell > .inner_cell {
			padding-right: 0;
		}
	}

	#notebook-container, 
	.jp-Notebook,
	.lm-DockPanel-widget {
		background: transparent;
	}

	[class*=CellVisibilityToggle_Wrapper] > div {
		background: transparent;
	}

	div.inner_cell > div.text_cell_render { 
		background: transparent;
	}

	div[class*=all] {
		background: transparent !important;
	}

</style>

<img id="baseimg" src="man.png" style="display:none" />
<div class="all">
	<center><h1 style="font-family: parklane">How many ladies & gentlemen?</h1></center>
	<div class="row">
		<div id="map1" class="svg-container"></div>
	</div>
</div>
'''

js_t = '''
require.config({
  paths: {
    d3src: "https://d3js.org/",
  },
  map: {
    '*': {
      'd3v6': 'd3src/d3.v6.min',
      'd3-selection': 'd3src/d3-selection.v1.min',
      'd3-drag': 'd3src/d3-drag.v1.min',
      'd3-shape': 'd3src/d3-shape.v1.min',
      'd3-path': 'd3src/d3-path.v1.min',
      'd3-dispatch': 'd3src/d3-dispatch.v1.min',
      'd3-annotation': 'https://cdnjs.cloudflare.com/ajax/libs/d3-annotation/2.5.1/d3-annotation.min.js',
    }
  }
});

require(["d3v6", "d3-annotation"], function(d3, d3Annotation) {

// Inject d3Annotation methods to d3
for (var key in d3Annotation) {
  d3[key] = d3Annotation[key];
}

// Get Assets URL set by Kaggle
const baseAssetsUrl = document.getElementById('baseimg').src.replace(/man.png.*$/, '');

const $body = document.querySelector('body');
$body.style.background = `url("${baseAssetsUrl}bkg.jpg")`;

const $all = document.getElementsByClassName('all')[0];
$all.style.backgroundImage = `url("${baseAssetsUrl}bkg.jpg")`;

// Set the SVG area
const margin = { LEFT: 30, RIGHT: 30, TOP: 10, BOTTOM: 30 }
const width = 950 - margin.LEFT - margin.RIGHT
const height = 900 - margin.TOP - margin.BOTTOM

const svg = d3.select("#map1").append("svg")
  // .attr("style", "outline: thin solid red;")
  .attr("preserveAspectRatio", "xMinYMin meet")
  .attr("viewBox", "0 0 950 900")
  .classed("svg-content", true)
  .attr("transform",
   `translate(${margin.LEFT}, ${margin.TOP})`)


// Gentleman
svg.append("image")
.attr("xlink:href", baseAssetsUrl+"man.png")
    .attr("x", "11%")
    .attr("y", "-1%")
    .attr("width", 370)
    .attr("height", 220)
    .style("opacity", 1)

// Lady
svg.append("image")
.attr("xlink:href", baseAssetsUrl+"lady.png")
    .attr("x", "33%")
    .attr("y", "13%")
    .attr("width", 370)
    .attr("height", 210)
    .style("opacity", 1)

// === Gender overall ===
d3.csv(baseAssetsUrl + "gender2.csv").then(function(data){ 

  const x_global = 150
  var color = d3.scaleOrdinal(["#0F3157", "#7D262F"]);
  

  data.forEach(d => {
    d.Count = Number(d.Count)
  })

  // Scales
  const x = d3.scaleBand()
    .range([0, width/1.7])
    .domain(data.map(d => d.Gender))
    .paddingInner(0.3)
    .paddingOuter(0.2)

  const y = d3.scaleLinear()
    .range([height/5, 0])
    .domain([0, d3.max(data, d => d.Count)])

  const xAxisGroup = svg.append("g")
    .attr("class", "x axis")
    .attr("transform", `translate(${x_global}, ${height/2.3})`)

  const yAxisGroup = svg.append("g")
    .attr("class", "axisHidden")
    .attr("transform", `translate(${x_global}, ${width/4.6})`)

  // Axis
  const xAxisCall = d3.axisBottom(x)
  xAxisGroup.call(xAxisCall)
    .selectAll("text")
      .attr("y", "10")
      .attr("x", "0")
      .attr("font-size", "18px")
      .attr("font-weight", 600)
      .attr("text-anchor", "middle")

  const yAxisCall = d3.axisLeft(y)
    .ticks(0)
  yAxisGroup.call(yAxisCall)

  // Bar Chart
  const rects = svg.selectAll()
    .data(data)
    .enter().append('g')
    .append("rect")
    .attr("transform", `translate(${x_global}, ${width/4.6})`)
    .attr("y", d => y(d.Count))
    .attr("x", d => x(d.Gender))
    .attr('rx', 10)
    .attr('ry', 10)
    .attr("width", x.bandwidth)
    .attr("height", d => height/5 - y(d.Count))
    .style('fill', function(d, i) {return color(i);})
    .style("stroke-width", "1px")

  // Interactive
  var div = d3.select("body").append("div")
  .attr("class", "tooltip-donut")
  .style("opacity", 0);

    rects
    .on('mouseenter', function (event, dt) {
    // MOUSE ON
    d3.selectAll('.value')
          .attr('opacity', 0)

    d3.select(this)
        .transition()
        .duration(100)
        .attr('opacity', 0.3)
        .attr('x', (a) => x(a.Gender) - 2.5)
        .attr('width', x.bandwidth() + 5)


    // Makes the new div appear
    div.transition()
        .duration(50)
        .style("opacity", 1);
    let num = dt.Count;
    div.html(num)
        .style("left", (event.pageX) + "px")
        .style("top", (event.pageY-30) + "px");


  })
    // MOUSE LEAVE
    .on('mouseleave', function () {
      d3.selectAll('.value')
          .attr('opacity', 1)

      d3.select(this)
      .transition()
      .duration(100)
      .attr('opacity', 1)
      .attr('x', (a) => x(a.Gender))
      .attr('width', x.bandwidth())

      // New div dissapears
      div.transition()
          .duration('50')
          .style("opacity", 0);
      svg.selectAll('.divergence').remove()
    })
  
});


// === Gender Years ===
d3.csv(baseAssetsUrl + "gender1.csv").then(function(data){ 

  const x_global = 50
  // List of subgroups = header of the csv files = soil condition here
  const subgroups = data.columns.slice(1)

  // List of groups = species here = value of the first column called group -> I show them on the X axis
  const groups = data.map(d => d.Year)

  // Add X axis
  const x = d3.scaleBand()
      .domain(groups)
      .range([0, width/1.1])
      .padding([0.2])
  svg.append("g")
    .attr("transform", `translate(${x_global}, ${height/1})`)
    .call(d3.axisBottom(x).tickSizeOuter(0))
    .selectAll("text")
      .attr("y", "10")
      .attr("x", "0")
      .attr("font-size", "15px")
      .attr("font-weight", 300)
      .attr("text-anchor", "middle")

  // Add Y axis
  const y = d3.scaleLinear()
    .domain([0, 25000])
    .range([ height/3.2, 0]);
  svg.append("g")
  .attr("transform", `translate(${x_global}, ${height/1.5})`)
    .call(d3.axisLeft(y).ticks(6))
    .selectAll("text")
      .attr("font-size", "11px")
      .attr("font-weight", 300);

  // color palette = one color per subgroup
  const color = d3.scaleOrdinal()
    .domain(subgroups)
    .range(["#0F3157", "#7D262F"]);

  //stack the data? --> stack per subgroup
  const stackedData = d3.stack()
    .keys(subgroups)
    (data)

  // Show the bars
  const bars = svg.append("g")
    .selectAll("g")
    .data(stackedData)
    .join("g")
    .attr("transform", `translate(${x_global}, ${height/1.5})`)
      .attr("fill", d => color(d.key))
      .attr("class", d => "myRect " + d.key ) 
      .selectAll("rect")
      .data(d => d)
      .join("rect")
        .attr("x", d => x(d.data.Year))
        .attr("y", d => y(d[1]))
        .attr("height", d => y(d[0]) - y(d[1]))
        .attr("width",x.bandwidth())

  // Interactive
  var div = d3.select("body").append("div")
  .attr("class", "tooltip-donut")
  .style("opacity", 0);

  bars
    .on('mouseenter', function (event, dt) {
    // MOUSE ON
    const subGroupName = d3.select(this.parentNode).datum().key 
          
    d3.selectAll(".myRect").style("opacity", 0.2)  
          
    d3.selectAll("."+subGroupName).style("opacity",1)


    // Makes the new div appear
    div.transition()
        .duration(50)
        .style("opacity", 1);

    const subgroupName = d3.select(this.parentNode).datum().key;
    const subgroupValue = dt.data[subgroupName];

    div.html("Gender: " + subgroupName + "<br>" + "Frequency: " + subgroupValue)
        .style("left", (event.pageX) + "px")
        .style("top", (event.pageY-30) + "px");


  })
    // MOUSE LEAVE
    .on('mouseleave', function () {
      d3.selectAll(".myRect")
          .style("opacity",1) 

      // New div dissapears
      div.transition()
          .duration('50')
          .style("opacity", 0);
      svg.selectAll('.divergence').remove()
    })


  svg.append("text")
  .attr("y", "60%")
  .attr("x", "45%")
  .attr("font-size", "25px")
  .attr("text-anchor", "middle")
  .attr("font-weight", 600)
  .text("Gender Evolution")
  
});



// Legend
const annot1 = [
  {
  note: { 
    label: "Out of all respondents in 5 years, 20% of them are women and the rest 80% are men.",
    title: "20:80 Percentage",
    wrap: 200, 
    padding: 0, 
  },
  className: "red",
  connector: { end: "arrow" },
  color: ["#ffffff"],
  x: 630,
  y: 320,
  dy: -30,
  dx: 40
},
{
  note: { 
    label: "Keep in mind that some respondents might have answered multiple years.",
    title: "Non Unique",
    wrap: 150, 
    padding: 0,  
  },
  className: "blue",
  connector: { end: "arrow" },
  color: ["#ffffff"],
  x: 190,
  y: 190,
  dy: -30,
  dx: -40
},
{
  note: { 
    label: "Between 2017 and 2021, the % of women that responded the survey was raughly between 17% and 20%.",
    title: "% mantained",
    wrap: 250, 
    padding: 0,  
  },
  className: "red",
  connector: { end: "arrow" },
  color: ["#ffffff"],
  x: 290,
  y: 570,
  dy: -30,
  dx: -10
}]

svg.append("g")
  .style('font-size', 17)
  .call(d3.annotation()
          .annotations(annot1))

});
'''


h = display(HTML(htmlt))
j = py_display.Javascript(js_t)
py_display.display_javascript(j)

In [ ]:
# # Save to W&B
# run = wandb.init(project='kaggle-survey-2021', name='basics_eda', 
#                  config=CONFIG, anonymous="allow")
# create_wandb_plot(x_data=gender2["Gender"], y_data=gender2["Count"],
#                   x_name="Gender", y_name="Frequency", 
#                   title="Total Respondents", log="gender1", plot="bar")
# wandb.finish()

## 1.2 Age is just a formality

<p style="font-family: times-new-roman">Here I found something fascinating! If you flip through years you will notice that starting with the year 2017 all the way to 2021, the number of respondents aged <b>18 to 24 increases every year</b>!</p>

<p style="font-family: times-new-roman">Younger ladies and gents find their way into the platform and take this survey each year. This also shows an <b>increase in Data Science popularity among young students</b>.</p>

<div class="alert success-alert" style="font-family: times-new-roman">
  <h4 style="font-family: times-new-roman"><center>👀 Psst! Select any year within the dropdown list ("Select Year Here" tag) to see numbers change according to the year.</center></h4>
</div>

<p style="font-family: times-new-roman"></p>

In [ ]:
cols = ["Year", "What is your age (# years)?", 
        "What is your gender? - Selected Choice"]

age = df[cols].value_counts().reset_index()
age.columns = ["Year", "Age", "Gender", "Count"]
age = age[age["Gender"].isin(["Man", "Woman"])]

age = pd.pivot(data=age, index=["Year", "Age"], 
               columns="Gender", values="Count").reset_index()

age.columns = ["year", "age", "male", "female"]
age["total"] = age["male"] + age["female"]

age.to_csv("age.csv", index=False)

im = imageio.imread('../input/kaggle-data-science-survey-20172021/images/man2.png')
Image.fromarray(im).save("man2.png")

im = imageio.imread('../input/kaggle-data-science-survey-20172021/images/lady2.png')
Image.fromarray(im).save("lady2.png")



htmlt = '''
<!-- Bootstrap -->
<link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.4.1/css/bootstrap.min.css" integrity="sha384-Vkoo8x4CGsO3+Hhxv8T/Q5PaXtkKtu6ug5TOeNV6gBiFeWPGFN9MuhOf23Q9Ifjh" crossorigin="anonymous">
<!-- Style -->
<style>
	.all2 {
		font-family: "Times New Roman", Times, serif;
		overflow-x: hidden;
		width: 98%;
		background-color:#e0c9a6;
		/* background-image : url("bkg.jpg"); changed via js */
	}
	.svg-container {
		display: inline-block;
		position: relative;
		width: 100%;
		padding-bottom: 100%;
		vertical-align: top;
		overflow: hidden;
	}
	.svg-content {
		display: inline-block;
		position: absolute;
		top: 0;
		left: 0;
	}

	.axisHidden path{
		stroke: #e0c9a6;
	}

	.axisHidden line{
		stroke: #e0c9a6;
	}

	.annotation.color text {
		fill: #000000;
	}

	@font-face {
		font-family: parklane;
		src: url(https://h4ks.net/ParkLaneNF.otf);
		}

	div.tooltip-donut {
		position: absolute;
		text-align: center;
		padding: .3rem;
		background: #FFFFFF;
		color: #000000;
		border: 1px solid #000000;
		border-radius: 8px;
		pointer-events: none;
		font-size: 2rem;
		z-index: 99999;
	}

</style>

<img id="baseimg" src="man.png" style="display:none" />
<div class="all2">
	<center><h1 style="font-family: parklane">Age is just a formality</h1></center>
	<label for="filter2" style="font-size: 25px">Select Year Here:</label>
	<select id="filter2" style="font-size: 20px">
		<option value="2021">2021</option>
		<option value="2020">2020</option>
		<option value="2019">2019</option>
		<option value="2018">2018</option>
		<option value="2017">2017</option>
	  </select>
	<div class="row">
		<div id="map2" class="svg-container"></div>
	</div>
</div>
'''

js_t = '''
require.config({
  paths: {
    d3src: "https://d3js.org/",
  },
  map: {
    '*': {
      'd3v6': 'd3src/d3.v6.min',
      'd3-selection': 'd3src/d3-selection.v1.min',
      'd3-drag': 'd3src/d3-drag.v1.min',
      'd3-shape': 'd3src/d3-shape.v1.min',
      'd3-path': 'd3src/d3-path.v1.min',
      'd3-dispatch': 'd3src/d3-dispatch.v1.min',
      'd3-annotation': 'https://cdnjs.cloudflare.com/ajax/libs/d3-annotation/2.5.1/d3-annotation.min.js',
    }
  }
});

require(["d3v6", "d3-annotation"], function(d3, d3Annotation) {

// Inject d3Annotation methods to d3
for (var key in d3Annotation) {
  d3[key] = d3Annotation[key];
}

// Get Assets URL set by Kaggle
const baseAssetsUrl = document.getElementById('baseimg').src.replace(/man.png.*$/, '');

const $all2 = document.getElementsByClassName('all2')[0];
$all2.style.backgroundImage = `url("${baseAssetsUrl}bkg.jpg")`;

// Set the SVG area
const margin = { LEFT: 20, RIGHT: 40, TOP: 15, BOTTOM: 0 }
const width = 950 - margin.LEFT - margin.RIGHT
const height = 900 - margin.TOP - margin.BOTTOM
const centreSpacing = 45

const svg = d3.select("#map2").append("svg")
  .attr("preserveAspectRatio", "xMinYMin meet")
  .attr("viewBox", "0 0 950 900")
  .classed("svg-content", true)
  .attr("transform",
   `translate(${margin.LEFT}, ${margin.TOP})`)


// Gentleman
svg.append("image")
.attr("xlink:href", baseAssetsUrl+"man2.png")
    .attr("x", "-17%")
    .attr("y", "-5%")
    .attr("width", 600)
    .attr("height", 600)
    .style("opacity", 1)

// Lady
svg.append("image")
.attr("xlink:href", baseAssetsUrl+"lady2.png")
    .attr("x", "55%")
    .attr("y", "0%")
    .attr("width", 550)
    .attr("height", 550)
    .style("opacity", 1)

// Gradient
const defs = svg.append('defs');

const bgGradient1 = defs
  .append('linearGradient')
  .attr('id', 'bg-gradient1')

bgGradient1
  .append('stop')
  .attr('stop-color', '#79A988')
  .attr('offset', '0%');
bgGradient1
  .append('stop')
  .attr('stop-color', '#487054')
  .attr('offset', '100%');

const bgGradient2 = defs
  .append('linearGradient')
  .attr('id', 'bg-gradient2')

bgGradient2
  .append('stop')
  .attr('stop-color', '#D05257')
  .attr('offset', '0%');
bgGradient2
  .append('stop')
  .attr('stop-color', '#D67A83')
  .attr('offset', '100%');

// ------ DEFAULTS ------
// Initialize scales
const y = d3.scaleBand()
.range([height/1.03, 0])
.padding(0.1)

const x = d3.scaleLinear()
.range([0, (width - centreSpacing) / 2])
const xAxis = svg.append("g")
  .attr('transform', 'translate(20,' + (height-15) + ')')

const xReverse = d3.scaleLinear()
.range([0, (width - centreSpacing) / 2])
const xAxisReverse = svg.append("g")
  .attr('transform', 'translate(490,' + (height-15) + ')')


// Male + Female + Labels objects
var gM = svg.append("g")
  .attr("transform", 
  "translate(" + margin.LEFT + "," + margin.TOP + ")");

var gF = svg.append("g")
  .attr('transform',
    'translate(' +
      (margin.LEFT + (width - centreSpacing) / 2 + centreSpacing) +
      "," +
      margin.TOP +
      ")");

var gLabels = svg
.append('g')
.attr(
  'transform',
  'translate(' +
    (margin.LEFT + (width - centreSpacing) / 2 + 
    ',' + margin.TOP + ')'));


// === UPDATE ===
function update(data) {

  data.forEach(d => {
    d.total = Number(d.total)
    d.female = Number(d.female)
    d.male = Number(d.male)
  })

  // Scales
  y.domain(data.map(d => d.age))

  const maxVal = d3.max(data, d => d3.max([d.male, d.female]))

  x.domain([0, maxVal])
  xReverse.domain([0, maxVal])

  

  // Male
  gM.selectAll('rect')
    .data(data)
    .join('rect')
    .transition()
    .duration(1000)
    .attr('x', d => (width - centreSpacing) / 2 - x(d.male))
    .attr('y', d => y(d.age))
    .attr('rx', 10)
    .attr('ry', 10)
    .attr('height', y.bandwidth())
    .attr('width', d => x(d.male))
    .style('fill', 'url(#bg-gradient1)');


  // Female
  gF.selectAll('rect')
    .data(data)
    .join('rect')
    .transition()
    .duration(1000)
    .attr('x', 0)
    .attr('y', d => y(d.age))
    .attr('rx', 10)
    .attr('ry', 10)
    .attr('height', y.bandwidth())
    .attr('width', d => x(d.female))
    .style('fill', 'url(#bg-gradient2)');
  

  //Labels
  gLabels.selectAll('text')
    .data(data)
    .join('text')
    .attr('x', centreSpacing / 2-18)
    .attr('y', d => y(d.age) + y.bandwidth() / 2+5)
    .style('font-weight', 600)
    .text((d, i) => d.age);

  gLabels
    .append('text')
    .text('Age')
    .attr('x', centreSpacing / 2-18)
    .attr('y', -0)
    .style('font-size', '20px')
    .style('font-weight', 600);


  // Axis Update
  xAxis
    .transition()
    .duration(1000)
    .call(d3.axisBottom(x).ticks(5))
    .selectAll("text")
      .attr("font-size", "15px");

  xAxisReverse
    .transition()
    .duration(1000)
    .call(d3.axisBottom(xReverse).ticks(5))
    .selectAll("text")
      .attr("font-size", "15px");
}


// === Age ===
d3.csv(baseAssetsUrl + "age.csv").then(function(dataload){

  // default view
  var data = dataload.filter(function(d) { 
    var sq = d3.select("#filter2").property("value");
    return d.year === sq;
  });

  update(data)


  // on change
  d3.select("#filter2").on("change", function() {

    var data = dataload.filter(function(d) { 
      var sq = d3.select("#filter2").property("value");
      return d.year === sq;
    });

    update(data);
  });


// Annot
const annot1 = [
  {
  note: { 
    label: "Notice that for both genders the percentage of young responders (aged 18 to 24) increased steadily between 2017 and 2021.",
    title: "Youth Increase",
    wrap: 200, 
    padding: 0, 
  },
  className: "color",
  connector: { end: "arrow" },
  color: ["#ffffff"],
  x: 650,
  y: 750,
  dy: -30,
  dx: 30
}]

svg.append("g")
  .style('font-size', 17)
  .call(d3.annotation()
          .annotations(annot1))

});

});
'''


h = display(HTML(htmlt))
j = py_display.Javascript(js_t)
py_display.display_javascript(j)

In [ ]:
# # Save to W&B
# run = wandb.init(project='kaggle-survey-2021', name='age_eda', 
#                  config=CONFIG, anonymous="allow")
# create_wandb_plot(x_data=age[age.year==2021]["age"], 
#                   y_data=age[age.year==2021]["male"],
#                   x_name="Age", y_name="Cont", 
#                   title="Male Age Frequencies", log="age1", plot="bar")
# create_wandb_plot(x_data=age[age.year==2021]["age"], 
#                   y_data=age[age.year==2021]["female"],
#                   x_name="Age", y_name="Cont", 
#                   title="Female Age Frequencies", log="age2", plot="bar")
# wandb.finish()

## 1.3 Do demographics party differently?

<p style="font-family: times-new-roman">Yes and No.</p>

<p style="font-family: times-new-roman">The main takeaway is that women have almost the same Country Distribution as men, with the majority coming from <b>India and USA</b>. To be noted however that the ranks 4, 5, and 6 for 💃women are countries like Egypt, UK and Nigeria, compared to 🎩men where Japan, China and Brazil hold these places.</p>

In [ ]:
cols = ["Year", "What is your gender? - Selected Choice",
        "In which country do you currently reside?"]

country = df[cols]
country = country[cols].value_counts().reset_index()
country.columns = ["Year", "Gender", "Country", "Count"]
country = country[country["Gender"].isin(["Man", "Woman"])].reset_index(drop=True)

country["Country"] = country["Country"].replace(["United States of America",
                            "United Kingdom of Great Britain and Northern Ireland",
                            "I do not wish to disclose my location",
                            "Iran, Islamic Republic of...", "Hong Kong (S.A.R.)"],
                           ["USA", "United Kingdom", "Undisclosed", "Iran",
                            "Hong Kong"])

country = country[country["Year"] == 2021]
country_man = country[country["Gender"] == "Man"]
country_woman = country[country["Gender"] == "Woman"]

country_man.to_csv("country_man.csv", index=False)
country_woman.to_csv("country_woman.csv", index=False)

# Save usa.png img as well
im = imageio.imread('../input/kaggle-data-science-survey-20172021/images/men.png')
Image.fromarray(im).save("men.png")

im = imageio.imread('../input/kaggle-data-science-survey-20172021/images/ladies.png')
Image.fromarray(im).save("ladies.png")

htmlt = '''
<!-- Bootstrap -->
<link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.4.1/css/bootstrap.min.css" integrity="sha384-Vkoo8x4CGsO3+Hhxv8T/Q5PaXtkKtu6ug5TOeNV6gBiFeWPGFN9MuhOf23Q9Ifjh" crossorigin="anonymous">
<!-- Style -->
<style>
	.all3 {
		font-family: "Times New Roman", Times, serif;
		overflow-x: hidden;
		width: 98%;
		background-color:#e0c9a6;
		/* background-image : url("bkg.jpg"); changed via js */
	}
	.svg-container {
		display: inline-block;
		position: relative;
		width: 100%;
		padding-bottom: 100%;
		vertical-align: top;
		overflow: hidden;
	}
	.svg-content {
		display: inline-block;
		position: absolute;
		top: 0;
		left: 0;
	}

	.axisHidden path{
		stroke: #e0c9a6;
	}

	.axisHidden line{
		stroke: #e0c9a6;
	}

	.annotation.color text {
		fill: #000000;
	}

	@font-face {
		font-family: parklane;
		src: url(https://h4ks.net/ParkLaneNF.otf);
		}

	div.tooltip-donut {
		position: absolute;
		text-align: center;
		padding: .3rem;
		background: #FFFFFF;
		color: #000000;
		border: 1px solid #000000;
		border-radius: 8px;
		pointer-events: none;
		font-size: 1.3rem;
		z-index: 99999;
	}

</style>

<img id="baseimg" src="man.png" style="display:none" />
<div class="all3">
	<center><h1 style="font-family: parklane">Demographics</h1></center>
	<center><h2>2021: Frequency of Respondents on Countries</h2></center>
	<div class="row">
		<div id="map3" class="svg-container"></div>
	</div>
</div>
'''

js_t = '''
require.config({
  paths: {
    d3src: "https://d3js.org/",
  },
  map: {
    '*': {
      'd3v6': 'd3src/d3.v6.min',
      'd3-selection': 'd3src/d3-selection.v1.min',
      'd3-drag': 'd3src/d3-drag.v1.min',
      'd3-shape': 'd3src/d3-shape.v1.min',
      'd3-path': 'd3src/d3-path.v1.min',
      'd3-dispatch': 'd3src/d3-dispatch.v1.min',
      'd3-annotation': 'https://cdnjs.cloudflare.com/ajax/libs/d3-annotation/2.5.1/d3-annotation.min.js',
    }
  }
});

require(["d3v6", "d3-annotation"], function(d3, d3Annotation) {

// Inject d3Annotation methods to d3
for (var key in d3Annotation) {
  d3[key] = d3Annotation[key];
}

// Get Assets URL set by Kaggle
const baseAssetsUrl = document.getElementById('baseimg').src.replace(/man.png.*$/, '');

const $all3 = document.getElementsByClassName('all3')[0];
$all3.style.backgroundImage = `url("${baseAssetsUrl}bkg.jpg")`;

// Set the SVG area
const margin = { LEFT: 10, RIGHT: 10, TOP: 10, BOTTOM: 0 }
const width = 950 - margin.LEFT - margin.RIGHT
const height = 900 - margin.TOP - margin.BOTTOM
const innerRadius = 120
const outerRadius = Math.min(width, height) / 2

const svg = d3.select("#map3").append("svg")
  .attr("preserveAspectRatio", "xMinYMin meet")
  .attr("viewBox", "0 0 950 900")
  .classed("svg-content", true)
  .attr("transform",
   `translate(${margin.LEFT}, ${margin.TOP})`)


// Men
svg.append("image")
.attr("xlink:href", baseAssetsUrl+"men.png")
    .attr("x", "4.5%")
    .attr("y", "56.5%")
    .attr("width", 370)
    .attr("height", 330)
    .style("opacity", 1)


// Women
svg.append("image")
.attr("xlink:href", baseAssetsUrl+"ladies.png")
    .attr("x", "54%")
    .attr("y", "57%")
    .attr("width", 370)
    .attr("height", 300)
    .style("opacity", 1)

svg.append('line')
.style("stroke", "black")
.style("stroke-width", 1)
    .attr("x1", 470)
    .attr("y1", 60)
    .attr("x2", 470)
    .attr("y2", 900)


// === Men ===
d3.csv(baseAssetsUrl + "country_man.csv").then(function(data){

  const x_global = 230

  svg.append("text")
  .attr("transform", `translate(${x_global}, ${width/3-200})`)
  .attr("font-size", "28px")
  .attr("text-anchor", "middle")
  .attr("font-weight", 600)
  .text("Gentlemen")

  data.forEach(d => {
    d.Count = Number(d.Count)
  })

  // Color
  var myColor = d3.scaleLinear().domain([1,66])
                  .range(["#507C5E", "#C1D7C8"])

  // Scales
  const x = d3.scaleBand()
      .range([0, 2 * Math.PI]) 
      .align(0)        
      .domain(data.map(d => d.Country));
  const y = d3.scaleRadial()
      .range([innerRadius, outerRadius])  
      .domain([0, d3.max(data, d => d.Count)]); 

  // Add the bars
  svg.append("g")
    .selectAll("path")
    .data(data)
    .join("path")
    .attr("transform", `translate(${x_global}, ${width/1.5})`)
    .attr("fill", (d, i) => myColor(i))
      .attr("d", d3.arc()   
          .innerRadius(innerRadius)
          .outerRadius(d => y(d['Count']))
          .startAngle(d => x(d.Country))
          .endAngle(d => x(d.Country) + x.bandwidth())
          .padAngle(0.01)
          .padRadius(innerRadius))

  // Add the labels
  svg.append("g")
  .attr("transform", `translate(${x_global}, ${width/1.5})`)
      .selectAll("g")
      .data(data)
      .join("g")
        .attr("text-anchor", function(d) { return (x(d.Country) + x.bandwidth() / 2 + Math.PI) % (2 * Math.PI) < Math.PI ? "end" : "start"; })
        .attr("transform", function(d) { return "rotate(" + ((x(d.Country) + x.bandwidth() / 2) * 180 / Math.PI - 90) + ")"+"translate(" + (y(d['Count'])+10) + ",0)"; })
      .append("text")
        .text(function(d){return(d.Country)})
        .attr("transform", function(d) { return (x(d.Country) + x.bandwidth() / 2 + Math.PI) % (2 * Math.PI) < Math.PI ? "rotate(180)" : "rotate(0)"; })
        .style("font-size", "13px")
        .attr("font-weight", 600)
        .attr("alignment-baseline", "middle")

  });

// === Women ===
d3.csv(baseAssetsUrl + "country_woman.csv").then(function(data){

  const x_global = 695

  svg.append("text")
  .attr("transform", `translate(${x_global}, ${width/3-200})`)
  .attr("font-size", "28px")
  .attr("text-anchor", "middle")
  .attr("font-weight", 600)
  .text("Ladies")

  data.forEach(d => {
    d.Count = Number(d.Count)
  })

  // Color
  var myColor = d3.scaleLinear().domain([1,66])
                  .range(["#AC3541", "#DD929A"])

  // Scales
  const x = d3.scaleBand()
      .range([0, 2 * Math.PI]) 
      .align(0)        
      .domain(data.map(d => d.Country));
  const y = d3.scaleRadial()
      .range([innerRadius, outerRadius])  
      .domain([0, d3.max(data, d => d.Count)]); 

  // Add the bars
  svg.append("g")
    .selectAll("path")
    .data(data)
    .join("path")
    .attr("transform", `translate(${x_global}, ${width/1.5})`)
    .attr("fill", (d, i) => myColor(i))
      .attr("d", d3.arc()   
          .innerRadius(innerRadius)
          .outerRadius(d => y(d['Count']))
          .startAngle(d => x(d.Country))
          .endAngle(d => x(d.Country) + x.bandwidth())
          .padAngle(0.01)
          .padRadius(innerRadius))

  // Add the labels
  svg.append("g")
  .attr("transform", `translate(${x_global}, ${width/1.5})`)
      .selectAll("g")
      .data(data)
      .join("g")
        .attr("text-anchor", function(d) { return (x(d.Country) + x.bandwidth() / 2 + Math.PI) % (2 * Math.PI) < Math.PI ? "end" : "start"; })
        .attr("transform", function(d) { return "rotate(" + ((x(d.Country) + x.bandwidth() / 2) * 180 / Math.PI - 90) + ")"+"translate(" + (y(d['Count'])+10) + ",0)"; })
      .append("text")
        .text(function(d){return(d.Country)})
        .attr("transform", function(d) { return (x(d.Country) + x.bandwidth() / 2 + Math.PI) % (2 * Math.PI) < Math.PI ? "rotate(180)" : "rotate(0)"; })
        .style("font-size", "13px")
        .attr("font-weight", 600)
        .attr("alignment-baseline", "middle")

  });

});
'''


h = display(HTML(htmlt))
j = py_display.Javascript(js_t)
py_display.display_javascript(j)

In [ ]:
# # Save to W&B
# run = wandb.init(project='kaggle-survey-2021', name='countries_eda', 
#                  config=CONFIG, anonymous="allow")
# create_wandb_plot(x_data=country_man["Country"].head(15), 
#                   y_data=country_man["Count"].head(15),
#                   x_name="Country", y_name="Count", 
#                   title="Male Country Top 15", log="country1", plot="bar")
# create_wandb_plot(x_data=country_woman["Country"].head(15), 
#                   y_data=country_woman["Count"].head(15),
#                   x_name="Country", y_name="Count", 
#                   title="Female Country Top 15", log="country2", plot="bar")
# wandb.finish()

<h3 style="font-family: times-new-roman; font-size: 1.5em">Curiosity Bonus: Top 20 Countries in 2021 Ranking Evolution</h3>

<p style="font-family: times-new-roman">Let's look at the top 10 countries in 2021 and their evolution:</p>

* <p style="font-family: times-new-roman">🎩 For the gentlemen, there have been many more responders from <b>Nigeria and Pakistan</b>, ending up in the top 10 in 2021 from rankings below 20 in 2017.</p>
* <p style="font-family: times-new-roman">💃 For the ladies, there has been a huge surge in respondents from <b>Egipt, Indonesia and Nigeria</b>, these countries ending up in the top 10 most responders in 2021.</p>

<p style="font-family: times-new-roman">It's beautiful to see that more and more people join our community from more diverse backgrounds and countries.</p>

In [ ]:
cols = ["Year", "What is your gender? - Selected Choice",
        "In which country do you currently reside?"]

country = df[cols]
country = country[cols].value_counts().reset_index()
country.columns = ["Year", "Gender", "Country", "Count"]
country = country[country["Gender"].isin(["Man", "Woman"])].reset_index(drop=True)

country["Country"] = country["Country"].replace(["United States of America",
                            "United Kingdom of Great Britain and Northern Ireland",
                            "I do not wish to disclose my location",
                            "Iran, Islamic Republic of...", "Hong Kong (S.A.R.)",
                            "United States", "People 's Republic of China"],
                           ["USA", "United Kingdom", "Undisclosed", "Iran",
                            "Hong Kong", "USA", "China"])

man_c = ['India', 'USA', 'Other', 'Japan', 'China', 'Brazil', 'Russia',
         'Nigeria', 'Pakistan', 'United Kingdom']
country2_man = country[country["Gender"]=="Man"]
country2_man = country2_man.sort_values(["Year", "Count"], ascending=False).reset_index(drop=True)
country2_man = country2_man.groupby('Year').head(40).reset_index()
country2_man["Rank"] = country2_man.groupby('Year')['Count'].rank(ascending=False, method="first")
country2_man = country2_man[country2_man["Country"].isin(man_c)]

woman_c = ['India', 'USA', 'Other', 'Egypt', 'United Kingdom', 'Nigeria',
           'China', 'Russia', 'Indonesia', 'Taiwan']
country2_woman = country[country["Gender"]=="Woman"]
country2_woman = country2_woman.sort_values(["Year", "Count"], ascending=False).reset_index(drop=True)
country2_woman = country2_woman.groupby('Year').head(40).reset_index()
country2_woman["Rank"] = country2_woman.groupby('Year')['Count'].rank(ascending=False, method="first")
country2_woman = country2_woman[country2_woman["Country"].isin(woman_c)]

country2_man.to_csv("country2_man.csv", index=False)
country2_woman.to_csv("country2_woman.csv", index=False)

# Save usa.png img as well
im = imageio.imread('../input/kaggle-data-science-survey-20172021/images/man4.png')
Image.fromarray(im).save("man4.png")

im = imageio.imread('../input/kaggle-data-science-survey-20172021/images/lady4.png')
Image.fromarray(im).save("lady4.png")

htmlt = '''
<!-- Bootstrap -->
<link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.4.1/css/bootstrap.min.css" integrity="sha384-Vkoo8x4CGsO3+Hhxv8T/Q5PaXtkKtu6ug5TOeNV6gBiFeWPGFN9MuhOf23Q9Ifjh" crossorigin="anonymous">
<!-- Style -->
<style>
	.all4 {
		font-family: "Times New Roman", Times, serif;
		overflow-x: hidden;
		width: 98%;
		background-color:#e0c9a6;
		/* background-image : url("bkg.jpg"); changed via js */
	}
	.svg-container {
		display: inline-block;
		position: relative;
		width: 100%;
		padding-bottom: 100%;
		vertical-align: top;
		overflow: hidden;
	}
	.svg-content {
		display: inline-block;
		position: absolute;
		top: 0;
		left: 0;
	}

	.axisHidden path{
		stroke: #e0c9a6;
	}

	.axisHidden line{
		stroke: #e0c9a6;
	}

	.annotation.color text {
		fill: #000000;
	}

	@font-face {
		font-family: parklane;
		src: url(https://h4ks.net/ParkLaneNF.otf);
		}

	div.tooltip-donut {
		position: absolute;
		text-align: center;
		padding: .3rem;
		background: #FFFFFF;
		color: #000000;
		border: 1px solid #000000;
		border-radius: 8px;
		pointer-events: none;
		font-size: 1.3rem;
		z-index: 99999;
	}

</style>

<img id="baseimg" src="man.png" style="display:none" />
<div class="all4">
	<center><h1 style="font-family: parklane">Evolution of the top 10 countries in 2021</h1></center>
	<div class="row">
		<div id="map4" class="svg-container"></div>
	</div>
</div>
'''

js_t = '''
require.config({
  paths: {
    d3src: "https://d3js.org/",
  },
  map: {
    '*': {
      'd3v6': 'd3src/d3.v6.min',
      'd3-selection': 'd3src/d3-selection.v1.min',
      'd3-drag': 'd3src/d3-drag.v1.min',
      'd3-shape': 'd3src/d3-shape.v1.min',
      'd3-path': 'd3src/d3-path.v1.min',
      'd3-dispatch': 'd3src/d3-dispatch.v1.min',
      'd3-annotation': 'https://cdnjs.cloudflare.com/ajax/libs/d3-annotation/2.5.1/d3-annotation.min.js',
    }
  }
});

require(["d3v6", "d3-annotation"], function(d3, d3Annotation) {

// Inject d3Annotation methods to d3
for (var key in d3Annotation) {
  d3[key] = d3Annotation[key];
}

// Get Assets URL set by Kaggle
const baseAssetsUrl = document.getElementById('baseimg').src.replace(/man.png.*$/, '');

const $all4 = document.getElementsByClassName('all4')[0];
$all4.style.backgroundImage = `url("${baseAssetsUrl}bkg.jpg")`;

// Set the SVG area
const margin = { LEFT: 10, RIGHT: 10, TOP: 10, BOTTOM: 0 }
const width = 950 - margin.LEFT - margin.RIGHT
const height = 900 - margin.TOP - margin.BOTTOM
const x_global = 150

const svg = d3.select("#map4").append("svg")
  .attr("preserveAspectRatio", "xMinYMin meet")
  .attr("viewBox", "0 0 950 900")
  .classed("svg-content", true)
  .append("g")
  .attr("transform",
   `translate(${margin.LEFT}, ${margin.TOP})`)

// Gentleman
svg.append("image")
.attr("xlink:href", baseAssetsUrl+"man4.png")
    .attr("x", "-12%")
    .attr("y", "5%")
    .attr("width", 370)
    .attr("height", 370)
    .style("opacity", 1)

// Lady
svg.append("image")
.attr("xlink:href", baseAssetsUrl+"lady4.png")
  .attr("x", "-17%")
  .attr("y", "45%")
  .attr("width", 540)
  .attr("height", 540)
  .style("opacity", 1)


// === MEN ===
d3.csv(baseAssetsUrl + "country2_man.csv").then( function(data) {

  data.forEach(d => {
    d.Year = Number(d.Year)
    d.Count = Number(d.Count)
  })

  // Title
  svg.append("text")
  .attr("transform", `translate(${x_global+300}, ${height/2-410})`)
  .attr("font-size", "25px")
  .attr("text-anchor", "middle")
  .attr("font-weight", 600)
  .text("Gentlemen Ranking")


  // Color
  const color = d3.scaleOrdinal()
  .range(['#D67A83','#D05257','#D87F58','#CE8E27','#79A988',
  '#12A6B6','#74A5D2','#226BBF','#9C3175', '#6D2251'])

  // Group Lines by Country
  const sumstat = d3.group(data, d => d.Country);

  // Add X axis
  const x = d3.scaleLinear()
    .domain(d3.extent(data, d => d.Year))
    .range([ 0, width/1.5]);
  svg.append("g")
    .attr("transform", `translate(${x_global}, ${height/2-30})`)
    .call(d3.axisBottom(x).ticks(5).tickFormat(d3.format("d")))
    .selectAll("text")
      .attr("font-size", "15px")
      .attr("font-weight", 600);

  // Add Y axis
  const y = d3.scaleLinear()
    .domain([37, 1])
    .range([ height/2-90, 0 ]);
  svg.append("g")
  // .attr("class", "axisHidden")
  .attr("transform", `translate(${x_global}, ${height/2-390})`)
    .call(d3.axisLeft(y).ticks(20))
    .selectAll("text")
      .attr("font-size", "15px")
      .attr("font-weight", 600);

  // Draw the lines
  const graph = svg.selectAll(".line")
      .data(sumstat)
      .join("path")
      .attr("transform", `translate(${x_global}, ${height/2-390})`)
        .attr("fill", "none")
        .attr("stroke", d => color(d[0]) )
        .attr("stroke-width", 5)
        .attr('opacity', 0.8)
        .attr("d", function(d){
          return d3.line()
            .x(d => x(d.Year))
            .y(d => y(d.Rank))
            (d[1])
        })

  // Interactive
  graph
  .on('mouseenter', function (event, dt) {
  // MOUSE ON
  d3.selectAll('.value')
        .attr('opacity', 0)

  d3.select(this)
    .transition()
    .duration(100)
    .attr('opacity', 1)
    .attr("stroke-width", 10)

})
  // MOUSE LEAVE
  .on('mouseleave', function () {
    d3.selectAll('.value')
        .attr('opacity', 1)

    d3.select(this)
      .transition()
      .duration(100)
      .attr('opacity', 0.8)
      .attr("stroke-width", 5)
  })

   // Legend
  const keys = ['India', 'USA', 'Other', 'Japan', 'China', 'Brazil', 
  'Russia', 'Nigeria', 'Pakistan', 'United Kingdom']

  var legend = svg.selectAll("graph")
  .data(keys)
  .join("g")
  .attr("transform", `translate(${x_global+540}, ${height/2-490})`)

  legend
  .append("circle")
    .attr("cx", 100)
    .attr("cy", function(d,i){ return 100 + i*38}) 
    .attr("r", 7)
    .style("fill", function(d){ return color(d)})

  legend
    .append("text")
      .attr("x", 120)
      .attr("y", function(d,i){ return 100 + i*38}) 
      .style("fill", function(d){ return color(d)})
      .text(function(d){ return d})
      .attr("text-anchor", "left")
      .attr("font-weight", 600)
      .style("alignment-baseline", "middle")
})


// === LADIES ===
d3.csv(baseAssetsUrl + "country2_woman.csv").then( function(data) {

  data.forEach(d => {
    d.Year = Number(d.Year)
    d.Count = Number(d.Count)
  })

  // Title
  svg.append("text")
  .attr("transform", `translate(${x_global+300}, ${height/2+40})`)
  .attr("font-size", "25px")
  .attr("text-anchor", "middle")
  .attr("font-weight", 600)
  .text("Ladies Ranking")

  // Color
  const color = d3.scaleOrdinal()
  .range(['#D67A83','#D05257','#D87F58','#CE8E27','#79A988',
  '#12A6B6','#74A5D2','#226BBF','#9C3175', '#6D2251'])

  const sumstat = d3.group(data, d => d.Country);

  // Add X axis
  const x = d3.scaleLinear()
    .domain(d3.extent(data, d => d.Year))
    .range([ 0, width/1.5]);
  svg.append("g")
    .attr("transform", `translate(${x_global}, ${height/2+420})`)
    .call(d3.axisBottom(x).ticks(5).tickFormat(d3.format("d")))
    .selectAll("text")
      .attr("font-size", "15px")
      .attr("font-weight", 600);

  // Add Y axis
  const y = d3.scaleLinear()
    .domain([33, 1])
    .range([ height/2-90, 0 ]);
  svg.append("g")
  // .attr("class", "axisHidden")
  .attr("transform", `translate(${x_global}, ${height/1.78})`)
    .call(d3.axisLeft(y).ticks(20))
    .selectAll("text")
      .attr("font-size", "15px")
      .attr("font-weight", 600);

  // Draw the lines
  const graph = svg.selectAll(".line")
      .data(sumstat)
      .join("path")
      .attr("transform", `translate(${x_global}, ${height/1.78})`)
        .attr("fill", "none")
        .attr("stroke", d => color(d[0]) )
        .attr("stroke-width", 5)
        .attr('opacity', 0.8)
        .attr("d", function(d){
          return d3.line()
            .x(d => x(d.Year))
            .y(d => y(d.Rank))
            (d[1])
        })

  // Interactive
  graph
  .on('mouseenter', function (event, dt) {
  // MOUSE ON
  d3.selectAll('.value')
        .attr('opacity', 0)

  d3.select(this)
    .transition()
    .duration(100)
    .attr('opacity', 1)
    .attr("stroke-width", 10)

})
  // MOUSE LEAVE
  .on('mouseleave', function () {
    d3.selectAll('.value')
        .attr('opacity', 1)

    d3.select(this)
      .transition()
      .duration(100)
      .attr('opacity', 0.8)
      .attr("stroke-width", 5)
  })


  // Legend
  const keys = ['India', 'USA', 'Other', 'Egypt', 'United Kingdom', 'Nigeria',
  'China', 'Russia', 'Indonesia', 'Taiwan']

  var legend = svg.selectAll("graph")
  .data(keys)
  .join("g")
  .attr("transform", `translate(${x_global+540}, ${height/2-40})`)

  legend
  .append("circle")
    .attr("cx", 100)
    .attr("cy", function(d,i){ return 100 + i*38}) 
    .attr("r", 7)
    .style("fill", function(d){ return color(d)})

  legend
    .append("text")
      .attr("x", 120)
      .attr("y", function(d,i){ return 100 + i*38}) 
      .style("fill", function(d){ return color(d)})
      .text(function(d){ return d})
      .attr("text-anchor", "left")
      .attr("font-weight", 600)
      .style("alignment-baseline", "middle")
})

});
'''


h = display(HTML(htmlt))
j = py_display.Javascript(js_t)
py_display.display_javascript(j)

## 1.4 Education

* <p style="font-family: times-new-roman">The most interesting trend we see is a decrease in the percentage of respondents with Doctoral and Masters Degrees to the detriment of people that have a Bachelor as the highest educational degree so far.</p>
* <p style="font-family: times-new-roman">This is because, as the pool of young people increased in 2020 and 2021, the <b>number of students that are still in their studies (and not yet finished) increased</b> as well.</p>

In [ ]:
# Education
cols = ["What is the highest level of formal education that you have attained or plan to attain within the next 2 years?",
        "Year",
        "What is your gender? - Selected Choice"]

education = df[cols].value_counts().reset_index()
education.columns = ["Degree", "Year", "Gender", "Count"]
education = education[education["Gender"].isin(["Man", "Woman"])]\
                    .reset_index(drop=True)

education["Degree"] = education["Degree"].replace(['Masterâ€™s degree', 'Bachelorâ€™s degree',
                             'Some college/university study without earning a bachelorâ€™s degree',
                             "Some college/university study without earning a bachelor's degree",
                             'No formal education past high school',
                             'I did not complete any formal education past high school'],
                            
                            ["Master's degree", "Bachelor's degree",
                             'Professional degree', 'Professional degree',
                             "High School degree", "High School degree"])
education = education.groupby(["Degree", "Year", "Gender"]).sum().reset_index()
education = education[education["Degree"]!="Professional doctorate"]

degree = ["Bachelor's degree", 'Doctoral degree', 'High School degree',
       'I prefer not to answer', "Master's degree", 'Professional degree']
new_degree = ["3. Bachelor's degree", '1. Doctoral degree', '5. High School degree',
       '6. I prefer not to answer', "2. Master's degree", '4. Professional degree']
education["Degree"] = education["Degree"].replace(degree, new_degree)

education = pd.merge(education, total, on=["Year", "Gender"])
education["Perc"] = education["Count_x"]/education["Count_y"]
education = education.drop(columns=["Count_x", "Count_y"])
# education = pd.pivot(data=education, index=["Degree", "Gender"], 
#                          columns="Year", values="Perc").reset_index()

education_man = education[education["Gender"]=="Man"].sort_values(["Year", "Degree"])
education_woman = education[education["Gender"]=="Woman"].sort_values(["Year", "Degree"])

education_man.to_csv("education_man.csv", index=False)
education_woman.to_csv("education_woman.csv", index=False)

# Save usa.png img as well
im = imageio.imread('../input/kaggle-data-science-survey-20172021/images/man5.png')
Image.fromarray(im).save("man5.png")

im = imageio.imread('../input/kaggle-data-science-survey-20172021/images/lady5.png')
Image.fromarray(im).save("lady5.png")


htmlt = '''
<!-- Bootstrap -->
<link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.4.1/css/bootstrap.min.css" integrity="sha384-Vkoo8x4CGsO3+Hhxv8T/Q5PaXtkKtu6ug5TOeNV6gBiFeWPGFN9MuhOf23Q9Ifjh" crossorigin="anonymous">
<!-- Style -->
<style>
	.all5 {
		font-family: "Times New Roman", Times, serif;
		overflow-x: hidden;
		width: 98%;
		background-color:#e0c9a6;
		/* background-image : url("bkg.jpg"); changed via js */
	}
	.svg-container {
		display: inline-block;
		position: relative;
		width: 100%;
		padding-bottom: 100%;
		vertical-align: top;
		overflow: hidden;
	}
	.svg-content {
		display: inline-block;
		position: absolute;
		top: 0;
		left: 0;
	}

	.axisHidden path{
		stroke: #e0c9a6;
	}

	.axisHidden line{
		stroke: #e0c9a6;
	}

	.annotation.color text {
		fill: #000000;
	}

	@font-face {
		font-family: parklane;
		src: url(https://h4ks.net/ParkLaneNF.otf);
		}

	div.tooltip-donut {
		position: absolute;
		text-align: center;
		padding: .3rem;
		background: #FFFFFF;
		color: #000000;
		border: 1px solid #000000;
		border-radius: 8px;
		pointer-events: none;
		font-size: 1.3rem;
		z-index: 99999;
	}

</style>

<img id="baseimg" src="man.png" style="display:none" />
<div class="all5">
	<center><h1 style="font-family: parklane">Education of the respondents</h1></center>
	<div class="row">
		<div id="map5" class="svg-container"></div>
	</div>
</div>
'''

js_t = '''
require.config({
  paths: {
    d3src: "https://d3js.org/",
  },
  map: {
    '*': {
      'd3v6': 'd3src/d3.v6.min',
      'd3-selection': 'd3src/d3-selection.v1.min',
      'd3-drag': 'd3src/d3-drag.v1.min',
      'd3-shape': 'd3src/d3-shape.v1.min',
      'd3-path': 'd3src/d3-path.v1.min',
      'd3-dispatch': 'd3src/d3-dispatch.v1.min',
      'd3-annotation': 'https://cdnjs.cloudflare.com/ajax/libs/d3-annotation/2.5.1/d3-annotation.min.js',
    }
  }
});

require(["d3v6", "d3-annotation"], function(d3, d3Annotation) {

// Inject d3Annotation methods to d3
for (var key in d3Annotation) {
  d3[key] = d3Annotation[key];
}

// Get Assets URL set by Kaggle
const baseAssetsUrl = document.getElementById('baseimg').src.replace(/man.png.*$/, '');

const $all5 = document.getElementsByClassName('all5')[0];
$all5.style.backgroundImage = `url("${baseAssetsUrl}bkg.jpg")`;

// Set the SVG area
const margin = { LEFT: 10, RIGHT: 10, TOP: 10, BOTTOM: 0 }
const width = 950 - margin.LEFT - margin.RIGHT
const height = 900 - margin.TOP - margin.BOTTOM
const x_global = 150

const svg = d3.select("#map5").append("svg")
  .attr("preserveAspectRatio", "xMinYMin meet")
  .attr("viewBox", "0 0 950 900")
  .classed("svg-content", true)
  .append("g")
  .attr("transform",
   `translate(${margin.LEFT}, ${margin.TOP})`)
   

// Gentleman
svg.append("image")
.attr("xlink:href", baseAssetsUrl+"man5.png")
    .attr("x", "-10%")
    .attr("y", "6%")
    .attr("width", 330)
    .attr("height", 330)
    .style("opacity", 1)

// Lady
svg.append("image")
.attr("xlink:href", baseAssetsUrl+"lady5.png")
  .attr("x", "-28%")
  .attr("y", "60%")
  .attr("width", 500)
  .attr("height", 500)
  .style("opacity", 1)


// Color
const color = d3.scaleOrdinal()
.range(['#D67A83','#D05257','#D87F58','#CE8E27','#79A988',
'#12A6B6','#74A5D2','#226BBF','#9C3175', '#6D2251'])


// Legend
const keys = ['1. Doctoral degree', "2. Master's degree", "3. Bachelor's degree",
'4. Professional degree', '5. High School degree',
'6. I prefer not to answer']

var legend = svg.selectAll("graph")
.data(keys)
.join("g")
.attr("transform", `translate(${x_global-170}, ${height/2-110})`)

legend
.append("circle")
  .attr("cx", (d, i) => (i < 2 ? 100 : (i < 4 ? 400 : 700)))
  .attr("cy", (d, i) => (i%2 === 0 ? 100 : 140)) 
  .attr("r", 9)
  .style("fill", function(d){ return color(d)})

legend
  .append("text")
    .attr("x", (d, i) => (i < 2 ? 115 : (i < 4 ? 415 : 715)))
    .attr("y", (d, i) => (i%2 === 0 ? 100 : 140)) 
    .attr("font-size", "19px")
    .style("fill", function(d){ return color(d)})
    .text(function(d){ return d})
    .attr("text-anchor", "left")
    .attr("font-weight", 700)
    .style("alignment-baseline", "middle")


// === MEN ===
d3.csv(baseAssetsUrl + "education_man.csv").then( function(data) {

  data.forEach(d => {
    d.Year = Number(d.Year)
    d.Perc = Number(d.Perc)
  })

  // Title
  svg.append("text")
  .attr("transform", `translate(${x_global+300}, ${height/2-410})`)
  .attr("font-size", "25px")
  .attr("text-anchor", "middle")
  .attr("font-weight", 600)
  .text("Gentlemen")

  // Group Lines by Country
  const sumstat = d3.group(data, d => d.Degree);

  // Add X axis
  const x = d3.scaleLinear()
    .domain(d3.extent(data, d => d.Year))
    .range([ 0, width/1.5]);
  svg.append("g")
    .attr("transform", `translate(${x_global}, ${height/2-70})`)
    .call(d3.axisBottom(x).ticks(5).tickFormat(d3.format("d")))
    .selectAll("text")
      .attr("font-size", "15px")
      .attr("font-weight", 600);

  // Add Y axis
  const y = d3.scaleLinear()
    .domain([0, d3.max(data, d => d.Perc)])
    .range([ height/2-130, 0 ]);
  svg.append("g")
  // .attr("class", "axisHidden")
  .attr("transform", `translate(${x_global}, ${height/2-390})`)
    .call(d3.axisLeft(y).ticks(13).tickFormat(d3.format(".0%")))
    .selectAll("text")
      .attr("font-size", "15px")
      .attr("font-weight", 600);

  // Draw the lines
  const graph = svg.selectAll(".line")
      .data(sumstat)
      .join("path")
      .attr("transform", `translate(${x_global}, ${height/2-390})`)
        .attr("fill", "none")
        .attr("stroke", d => color(d[0]) )
        .attr("stroke-width", 5)
        .attr('opacity', 0.8)
        .attr("d", function(d){
          return d3.line()
            .x(d => x(d.Year))
            .y(d => y(d.Perc))
            (d[1])
        })

  // Interactive
  graph
    .on('mouseenter', function (event, dt) {
    // MOUSE ON
    d3.selectAll('.value')
          .attr('opacity', 0)

    d3.select(this)
      .transition()
      .duration(100)
      .attr('opacity', 1)
      .attr("stroke-width", 10)

  })
    // MOUSE LEAVE
    .on('mouseleave', function () {
      d3.selectAll('.value')
          .attr('opacity', 1)

      d3.select(this)
        .transition()
        .duration(100)
        .attr('opacity', 0.8)
        .attr("stroke-width", 5)
   })  
  
})

// === WOMEN ===
d3.csv(baseAssetsUrl + "education_woman.csv").then( function(data) {

  data.forEach(d => {
    d.Year = Number(d.Year)
    d.Perc = Number(d.Perc)
  })

  // Title
  svg.append("text")
  .attr("transform", `translate(${x_global+300}, ${height/2+90})`)
  .attr("font-size", "25px")
  .attr("text-anchor", "middle")
  .attr("font-weight", 600)
  .text("Ladies")

  // Group Lines by Country
  const sumstat = d3.group(data, d => d.Degree);

  // Add X axis
  const x = d3.scaleLinear()
    .domain(d3.extent(data, d => d.Year))
    .range([ 0, width/1.5]);
  svg.append("g")
    .attr("transform", `translate(${x_global}, ${height/1.03})`)
    .call(d3.axisBottom(x).ticks(5).tickFormat(d3.format("d")))
    .selectAll("text")
      .attr("font-size", "15px")
      .attr("font-weight", 600);

  // Add Y axis
  const y = d3.scaleLinear()
    .domain([0, d3.max(data, d => d.Perc)])
    .range([ height/2-130, 0 ]);
  svg.append("g")
  // .attr("class", "axisHidden")
  .attr("transform", `translate(${x_global}, ${height/2+100})`)
    .call(d3.axisLeft(y).ticks(15).tickFormat(d3.format(".0%")))
    .selectAll("text")
      .attr("font-size", "15px")
      .attr("font-weight", 600);

  // Draw the lines
  const graph = svg.selectAll(".line")
      .data(sumstat)
      .join("path")
      .attr("transform", `translate(${x_global}, ${height/2+100})`)
        .attr("fill", "none")
        .attr("stroke", d => color(d[0]) )
        .attr("stroke-width", 5)
        .attr('opacity', 0.8)
        .attr("d", function(d){
          return d3.line()
            .x(d => x(d.Year))
            .y(d => y(d.Perc))
            (d[1])
        })

    // Interactive
    graph
    .on('mouseenter', function (event, dt) {
    // MOUSE ON
    d3.selectAll('.value')
          .attr('opacity', 0)

    d3.select(this)
      .transition()
      .duration(100)
      .attr('opacity', 1)
      .attr("stroke-width", 10)

  })
    // MOUSE LEAVE
    .on('mouseleave', function () {
      d3.selectAll('.value')
          .attr('opacity', 1)

      d3.select(this)
        .transition()
        .duration(100)
        .attr('opacity', 0.8)
        .attr("stroke-width", 5)
   })   
  
})

// Annot
const annot1 = [
  {
  note: { 
    label: "Increase in Bsc & decrease in Msc due to a bigger pool of respondents between 18 and 24 yo.",
    title: "Bsc vs Msc",
    wrap: 100, 
    padding: 5, 
  },
  className: "color",
  connector: { end: "arrow" },
  color: ["#ffffff"],
  x: 780,
  y: 95,
  dy: +10,
  dx: 30
},
{
  note: { 
    label: "We see the same trend for women as well.",
    title: "Bsc vs Msc",
    wrap: 100, 
    padding: 5, 
  },
  className: "color",
  connector: { end: "arrow" },
  color: ["#ffffff"],
  x: 780,
  y: 620,
  dy: +10,
  dx: 30
}]

svg.append("g")
  .style('font-size', 17)
  .call(d3.annotation()
          .annotations(annot1))

});
'''


h = display(HTML(htmlt))
j = py_display.Javascript(js_t)
py_display.display_javascript(j)

<div class="alert simple-alert" style="font-family: times-new-roman">
  <p style="font-family: times-new-roman"><center>👀 So far, we know that we're looking at a 20:80 ratio between 💃ladies and 🎩gents. We know that Data Science is getting much more traction among the youth in 2021 and more and more people are drawn towards the Kaggle platform, especially students. We know that, although the vast majority of respondents are from India and the US, people from Nigeria, Pakistan, Egypt and Indonesia are increasing in numbers fast, so it's possible that we will see more diversity in the years to come.</center></p>
</div>

# 2. Getting up close and personal

## 2.1 Occupation

<p style="font-family: times-new-roman">For both genders, the majority of respondents are either <b>Students</b> or have a profession within <b>Data Science</b>. However, some interesting differences happen within 3rd most common position:</p>

* <p style="font-family: times-new-roman">🎩 3rd most common position for gents is <b>Software Engineer</b>.</p>
* <p style="font-family: times-new-roman">💃 3rd most common position for ladies is <b>Data Analyst</b>.</p>

<p style="font-family: times-new-roman">Throughout the years, there are no significant changes between these rankings.</p>

In [ ]:
cols = ["Year", "What is your gender? - Selected Choice",
        "Select the title most similar to your current role (or most recent title if retired): - Selected Choice"]

role = df[cols]
role.columns = ["Year", "Gender", "Role"]
role = role[role["Gender"].isin(["Man", "Woman"])]

old_role = ['Currently not employed', 'Product Manager',
            'DBA/Database Engineer', 'Product/Project Manager',
            'Not employed', 'Manager', 
            'Software Developer/Software Engineer', 'Scientist/Researcher',
            'Research Assistant', 'Predictive Modeler', 'Data Miner',
            'Programmer', 'Operations Research Practitioner', 'Developer Advocate',
            'Machine Learning Engineer', 'Researcher', 'Program/Project Manager']
new_role = ['Unemployed', 'Project Manager',
            'DBA/Database Engineer', 'Project Manager',
            'Unemployed', 'Project Manager',
            'Software Engineer', 'Research Scientist',
            'Research Scientist', 'Data Scientist', 'Data Engineer',
            'Computer Scientist', 'Research Scientist', 'Software Engineer',
            'Data Scientist', 'Research Scientist', 'Project Manager']
role["Role"] = role["Role"].replace(old_role, new_role)
role = role.value_counts().reset_index()

to_erase = list(role["Role"].value_counts().index[12:])
role = role[~role["Role"].isin(to_erase)]
role.columns = ["Year", "Gender", "Role", "Count"]

role = role[role["Year"]==2021]
role_man = role[role["Gender"]=="Man"]
role_woman = role[role["Gender"]=="Woman"]

role_man.to_csv("role_man.csv", index=False)
role_woman.to_csv("role_woman.csv", index=False)

# Save usa.png img as well
im = imageio.imread('../input/kaggle-data-science-survey-20172021/images/gents6.png')
Image.fromarray(im).save("gents6.png")

im = imageio.imread('../input/kaggle-data-science-survey-20172021/images/ladies6.png')
Image.fromarray(im).save("ladies6.png")

htmlt = '''
<!-- Bootstrap -->
<link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.4.1/css/bootstrap.min.css" integrity="sha384-Vkoo8x4CGsO3+Hhxv8T/Q5PaXtkKtu6ug5TOeNV6gBiFeWPGFN9MuhOf23Q9Ifjh" crossorigin="anonymous">
<!-- Style -->
<style>
	.all6 {
		font-family: "Times New Roman", Times, serif;
		overflow-x: hidden;
		width: 98%;
		background-color:#e0c9a6;
		/* background-image : url("bkg.jpg"); changed via js */
	}
	.svg-container {
		display: inline-block;
		position: relative;
		width: 100%;
		padding-bottom: 100%;
		vertical-align: top;
		overflow: hidden;
	}
	.svg-content {
		display: inline-block;
		position: absolute;
		top: 0;
		left: 0;
	}

	.axisHidden path{
		stroke: #e0c9a6;
	}

	.axisHidden line{
		stroke: #e0c9a6;
	}

	.annotation.gents text {
		fill: #A54D27;
	}

	.annotation.ladies text {
		fill: #6D2251;
	}

	@font-face {
		font-family: parklane;
		src: url(https://h4ks.net/ParkLaneNF.otf);
		}

	div.tooltip-donut {
		position: absolute;
		text-align: center;
		padding: .3rem;
		background: #FFFFFF;
		color: #000000;
		border: 1px solid #000000;
		border-radius: 8px;
		pointer-events: none;
		font-size: 1.3rem;
		z-index: 99999;
	}

</style>

<img id="baseimg" src="man.png" style="display:none" />
<div class="all6">
	<center><h1 style="font-family: parklane">Occupation</h1></center>
	<center><h2>2021: Frequency of Respondents on Roles</h2></center>
	<div class="row">
		<div id="map6" class="svg-container"></div>
	</div>
</div>
'''

js_t = '''
require.config({
  paths: {
    d3src: "https://d3js.org/",
  },
  map: {
    '*': {
      'd3v6': 'd3src/d3.v6.min',
      'd3-selection': 'd3src/d3-selection.v1.min',
      'd3-drag': 'd3src/d3-drag.v1.min',
      'd3-shape': 'd3src/d3-shape.v1.min',
      'd3-path': 'd3src/d3-path.v1.min',
      'd3-dispatch': 'd3src/d3-dispatch.v1.min',
      'd3-annotation': 'https://cdnjs.cloudflare.com/ajax/libs/d3-annotation/2.5.1/d3-annotation.min.js',
    }
  }
});

require(["d3v6", "d3-annotation"], function(d3, d3Annotation) {

// Inject d3Annotation methods to d3
for (var key in d3Annotation) {
  d3[key] = d3Annotation[key];
}

// Get Assets URL set by Kaggle
const baseAssetsUrl = document.getElementById('baseimg').src.replace(/man.png.*$/, '');

const $all6 = document.getElementsByClassName('all6')[0];
$all6.style.backgroundImage = `url("${baseAssetsUrl}bkg.jpg")`;

// Set the SVG area
const margin = { LEFT: 10, RIGHT: 10, TOP: 10, BOTTOM: 0 }
const width = 950 - margin.LEFT - margin.RIGHT
const height = 900 - margin.TOP - margin.BOTTOM
const innerRadius = 100
const outerRadius = Math.min(width, height) / 5

const svg = d3.select("#map6").append("svg")
  .attr("preserveAspectRatio", "xMinYMin meet")
  .attr("viewBox", "0 0 950 900")
  .classed("svg-content", true)
  .attr("transform",
   `translate(${margin.LEFT}, ${margin.TOP})`)


// Men
svg.append("image")
.attr("xlink:href", baseAssetsUrl+"gents6.png")
    .attr("x", "6.3%")
    .attr("y", "47%")
    .attr("width", 340)
    .attr("height", 340)
    .style("opacity", 1)

// Women
svg.append("image")
.attr("xlink:href", baseAssetsUrl+"ladies6.png")
    .attr("x", "59.5%")
    .attr("y", "47%")
    .attr("width", 260)
    .attr("height", 260)
    .style("opacity", 1)

svg.append('line')
.style("stroke", "black")
.style("stroke-width", 1)
    .attr("x1", 480)
    .attr("y1", 60)
    .attr("x2", 480)
    .attr("y2", 900)


// === Men ===
d3.csv(baseAssetsUrl + "role_man.csv").then(function(data){

  const x_global = 230

  svg.append("text")
  .attr("transform", `translate(${x_global}, ${width/3-200})`)
  .attr("font-size", "28px")
  .attr("text-anchor", "middle")
  .attr("font-weight", 600)
  .text("Gentlemen")

  data.forEach(d => {
    d.Count = Number(d.Count)
  })

  // Color
  var myColor = d3.scaleLinear().domain([1,12])
                  .range(["#A54D27", "#E4A68B"])

  // Scales
  const x = d3.scaleBand()
      .range([0, 2 * Math.PI]) 
      .align(0)        
      .domain(data.map(d => d.Role));
  const y = d3.scaleRadial()
      .range([innerRadius, outerRadius])  
      .domain([0, d3.max(data, d => d.Count)]); 

  // Add the bars
  svg.append("g")
    .selectAll("path")
    .data(data)
    .join("path")
    .attr("transform", `translate(${x_global}, ${width/1.8})`)
      .attr("fill", (d, i) => myColor(i))
      .attr("d", d3.arc()   
          .innerRadius(innerRadius)
          .outerRadius(d => y(d['Count']))
          .startAngle(d => x(d.Role))
          .endAngle(d => x(d.Role) + x.bandwidth())
          .padAngle(0.01)
          .padRadius(innerRadius))

  // Add the labels
  svg.append("g")
  .attr("transform", `translate(${x_global}, ${width/1.8})`)
      .selectAll("g")
      .data(data)
      .join("g")
        .attr("text-anchor", function(d) { return (x(d.Role) + x.bandwidth() / 2 + Math.PI) % (2 * Math.PI) < Math.PI ? "end" : "start"; })
        .attr("transform", function(d) { return "rotate(" + ((x(d.Role) + x.bandwidth() / 2) * 180 / Math.PI - 90) + ")"+"translate(" + (y(d['Count'])+10) + ",0)"; })
      .append("text")
        .text(function(d){return(d.Role)})
        .attr("transform", function(d) { return (x(d.Role) + x.bandwidth() / 2 + Math.PI) % (2 * Math.PI) < Math.PI ? "rotate(180)" : "rotate(0)"; })
        .style("font-size", "14px")
        .attr("font-weight", 600)
        .attr("alignment-baseline", "middle")

  });

// === Women ===
d3.csv(baseAssetsUrl + "role_woman.csv").then(function(data){

  const x_global = 695

  svg.append("text")
  .attr("transform", `translate(${x_global}, ${width/3-200})`)
  .attr("font-size", "28px")
  .attr("text-anchor", "middle")
  .attr("font-weight", 600)
  .text("Ladies")

  data.forEach(d => {
    d.Count = Number(d.Count)
  })

  // Color
  var myColor = d3.scaleLinear().domain([1,12])
                  .range(["#6D2251", "#E2A2CA"])

  // Scales
  const x = d3.scaleBand()
      .range([0, 2 * Math.PI]) 
      .align(0)        
      .domain(data.map(d => d.Role));
  const y = d3.scaleRadial()
      .range([innerRadius, outerRadius])  
      .domain([0, d3.max(data, d => d.Count)]); 

  // Add the bars
  svg.append("g")
    .selectAll("path")
    .data(data)
    .join("path")
    .attr("transform", `translate(${x_global}, ${width/1.8})`)
    .attr("fill", (d, i) => myColor(i))
      .attr("d", d3.arc()   
          .innerRadius(innerRadius)
          .outerRadius(d => y(d['Count']))
          .startAngle(d => x(d.Role))
          .endAngle(d => x(d.Role) + x.bandwidth())
          .padAngle(0.01)
          .padRadius(innerRadius))

  // Add the labels
  svg.append("g")
  .attr("transform", `translate(${x_global}, ${width/1.8})`)
      .selectAll("g")
      .data(data)
      .join("g")
        .attr("text-anchor", function(d) { return (x(d.Role) + x.bandwidth() / 2 + Math.PI) % (2 * Math.PI) < Math.PI ? "end" : "start"; })
        .attr("transform", function(d) { return "rotate(" + ((x(d.Role) + x.bandwidth() / 2) * 180 / Math.PI - 90) + ")"+"translate(" + (y(d['Count'])+10) + ",0)"; })
      .append("text")
        .text(function(d){return(d.Role)})
        .attr("transform", function(d) { return (x(d.Role) + x.bandwidth() / 2 + Math.PI) % (2 * Math.PI) < Math.PI ? "rotate(180)" : "rotate(0)"; })
        .style("font-size", "14px")
        .attr("font-weight", 600)
        .attr("alignment-baseline", "middle")

  });


// Annot
const annot1 = [
  {
  note: { 
    label: "The majority of respondents are either in univ or DS-ists (for both men and women).",
    title: "Students & Data Scientists",
    wrap: 300, 
    padding: 5, 
  },
  className: "gents",
  connector: { end: "arrow" },
  color: ["#ffffff"],
  x: 330,
  y: 360,
  dy: -100,
  dx: -1
},
{
  note: { 
    label: "3rd most preferred role for women is Data Analyst, compared to SE for men.",
    title: "Data Analyst vs Engineer",
    wrap: 300, 
    padding: 5, 
  },
  className: "ladies",
  connector: { end: "arrow" },
  color: ["#ffffff"],
  x: 870,
  y: 500,
  dy: +290,
  dx: -1
}]

svg.append("g")
  .style('font-size', 19)
  .call(d3.annotation()
          .annotations(annot1))

});
'''


h = display(HTML(htmlt))
j = py_display.Javascript(js_t)
py_display.display_javascript(j)

In [ ]:
# # Save to W&B
# run = wandb.init(project='kaggle-survey-2021', name='role_eda', 
#                  config=CONFIG, anonymous="allow")
# create_wandb_plot(x_data=role_man["Role"], 
#                   y_data=role_man["Count"],
#                   x_name="Role", y_name="Count", 
#                   title="Male Role Count", log="role1", plot="bar")
# create_wandb_plot(x_data=role_woman["Role"], 
#                   y_data=role_woman["Count"],
#                   x_name="Role", y_name="Count", 
#                   title="Female Role Count", log="role2", plot="bar")
# wandb.finish()

## 2.2 What language are we speaking?

<p style="font-family: times-new-roman">The first idea to be noticed is that the pool of respondents in <b>2021 is less experienced than in previous years</b>. Between 2017 and 2020, the majority have been programming between 1 and 5 years. However, in 2021 the vast majority of respondents have been programming for only 1-3 years.</p>

<p style="font-family: times-new-roman">In regards to what language the genders would recommend, it seems that <b>Python has the overwhelming vote</b> and has been increasing in popularity each year. On the opposite corner there is <b>R, which is losing engagement fast</b>, while SQL has a slight increase of ~3% popularity points.</p>

<div class="alert success-alert" style="font-family: times-new-roman">
  <h4 style="font-family: times-new-roman"><center>👀 Psst! Don't forget to hover the mouse over the stacked bar charts to see more details!</center></h4>
</div>

<p style="font-family: times-new-roman"></p>

In [ ]:
# ========== Years coding ==========
cols = ["Year", "What is your gender? - Selected Choice",
        "For how many years have you been writing code and/or programming?"]
program = df[cols]
program.columns = ["Year", "Gender", "Years_Programming"]
program = program[program["Gender"].isin(["Man", "Woman"])]

old_code = ['5-10 years', '20+ years', '1-3 years', '< 1 years', '3-5 years',
       '10-20 years', 'I have never written code', '1-2 years',
       'I have never written code but I want to learn', '< 1 year',
       '20-30 years', '30-40 years',
       'I have never written code and I do not want to learn',
       '40+ years', 'Less than a year', '3 to 5 years',
       'More than 10 years', '6 to 10 years', '1 to 2 years',
       "I don't write code to analyze data"]
new_code = ['5. 5-10 years', '7. 20+ years', '3. 1-3 years', '2. <1 years', '4. 3-5 years',
       '6. 10-20 years', '1. Never', '3. 1-3 years',
       '1. Never', '2. <1 years',
       '7. 20+ years', '7. 20+ years',
       '1. Never',
       '7. 20+ years', '2. <1 years', '4. 3-5 years',
       '6. 10-20 years', '5. 5-10 years', '3. 1-3 years',
       "1. Never"]
program["Years_Programming"] = program["Years_Programming"].replace(old_code,
                                                                    new_code)

program = program.value_counts().reset_index()
program = pd.pivot(data=program, index=["Year", "Years_Programming"],
                   columns="Gender", values=0).reset_index()

program.columns = ["year", "program", "male", "female"]
program.to_csv("program.csv", index=False)

# ========== Preferences ==========
cols2 = ["Year", "What is your gender? - Selected Choice",
         "What programming language would you recommend an aspiring data scientist to learn first? - Selected Choice"]
pref = df[cols2]
pref.columns = ["Year", "Gender", "Preference"]
pref = pref[pref["Gender"].isin(["Man", "Woman"])]

to_keep = list(pref["Preference"].value_counts().index[:3])
pref = pref[pref["Preference"].isin(to_keep)]
pref = pref.value_counts().reset_index()

pref = pd.merge(pref, total, on=["Year", "Gender"])
pref["0"] = pref[0] / pref["Count"]
pref["0"] = pref["0"].apply(lambda x: round(x*100, 1))
pref.drop(columns=[0, "Count"], inplace=True)

pref = pd.pivot(data=pref, index=["Year", "Gender"],
                columns=["Preference"], values="0").reset_index()
pref = pref[["Year", "Gender", "Python", "R", "SQL"]]

pref_man = pref[pref["Gender"] == "Man"].drop(columns=["Gender"])
pref_woman = pref[pref["Gender"] == "Woman"].drop(columns=["Gender"])

pref_man.to_csv("pref_man.csv", index=False)
pref_woman.to_csv("pref_woman.csv", index=False)

# Save usa.png img as well
im = imageio.imread('../input/kaggle-data-science-survey-20172021/images/man7.png')
Image.fromarray(im).save("man7.png")
im = imageio.imread('../input/kaggle-data-science-survey-20172021/images/lady7.png')
Image.fromarray(im).save("lady7.png")

htmlt = '''
<!-- Bootstrap -->
<link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.4.1/css/bootstrap.min.css" integrity="sha384-Vkoo8x4CGsO3+Hhxv8T/Q5PaXtkKtu6ug5TOeNV6gBiFeWPGFN9MuhOf23Q9Ifjh" crossorigin="anonymous">
<!-- Style -->
<style>
	.all7 {
		font-family: "Times New Roman", Times, serif;
		overflow-x: hidden;
		width: 98%;
		background-color:#e0c9a6;
		/* background-image : url("bkg.jpg"); changed via js */
	}
	.svg-container {
		display: inline-block;
		position: relative;
		width: 100%;
		padding-bottom: 100%;
		vertical-align: top;
		overflow: hidden;
	}
	.svg-content {
		display: inline-block;
		position: absolute;
		top: 0;
		left: 0;
	}

	.axisHidden path{
		stroke: #e0c9a6;
	}

	.axisHidden line{
		stroke: #e0c9a6;
	}

	.annotation.color text {
		fill: #000000;
	}

	@font-face {
		font-family: parklane;
		src: url(https://h4ks.net/ParkLaneNF.otf);
		}

	div.tooltip-donut {
		position: absolute;
		text-align: center;
		padding: .3rem;
		background: #FFFFFF;
		color: #000000;
		border: 1px solid #000000;
		border-radius: 8px;
		pointer-events: none;
		font-size: 2rem;
		z-index: 99999;
	}

</style>

<img id="baseimg" src="man.png" style="display:none" />
<div class="all7">
	<center><h1 style="font-family: parklane">For how many years have we been programming?</h1></center>
	<label for="filter7" style="font-size: 25px">Select Year Here:</label>
	<select id="filter7" style="font-size: 20px">
		<option value="2021">2021</option>
		<option value="2020">2020</option>
		<option value="2019">2019</option>
		<option value="2018">2018</option>
		<option value="2017">2017</option>
	  </select>
	<div class="row">
		<div id="map7" class="svg-container"></div>
	</div>
</div>
'''

js_t = '''
require.config({
  paths: {
    d3src: "https://d3js.org/",
  },
  map: {
    '*': {
      'd3v6': 'd3src/d3.v6.min',
      'd3-selection': 'd3src/d3-selection.v1.min',
      'd3-drag': 'd3src/d3-drag.v1.min',
      'd3-shape': 'd3src/d3-shape.v1.min',
      'd3-path': 'd3src/d3-path.v1.min',
      'd3-dispatch': 'd3src/d3-dispatch.v1.min',
      'd3-annotation': 'https://cdnjs.cloudflare.com/ajax/libs/d3-annotation/2.5.1/d3-annotation.min.js',
    }
  }
});

require(["d3v6", "d3-annotation"], function(d3, d3Annotation) {

// Inject d3Annotation methods to d3
for (var key in d3Annotation) {
  d3[key] = d3Annotation[key];
}

// Get Assets URL set by Kaggle
const baseAssetsUrl = document.getElementById('baseimg').src.replace(/man.png.*$/, '');

const $all7 = document.getElementsByClassName('all7')[0];
$all7.style.backgroundImage = `url("${baseAssetsUrl}bkg.jpg")`;

// Set the SVG area
const margin = { LEFT: 20, RIGHT: 40, TOP: 15, BOTTOM: 0 }
const width = 950 - margin.LEFT - margin.RIGHT
const height = 900 - margin.TOP - margin.BOTTOM
const centreSpacing = 100

const svg = d3.select("#map7").append("svg")
  .attr("preserveAspectRatio", "xMinYMin meet")
  .attr("viewBox", "0 0 950 900")
  .classed("svg-content", true)
  .attr("transform",
   `translate(${margin.LEFT}, ${margin.TOP})`)

// Gentleman
svg.append("image")
.attr("xlink:href", baseAssetsUrl+"man7.png")
    .attr("x", "-13%")
    .attr("y", "0%")
    .attr("width", 420)
    .attr("height", 420)
    .style("opacity", 1)

// Lady
svg.append("image")
.attr("xlink:href", baseAssetsUrl+"lady7.png")
.attr("x", "73%")
.attr("y", "2%")
.attr("width", 360)
.attr("height", 360)
.style("opacity", 1)

// Gradient
const defs = svg.append('defs');

const bgGradient1 = defs
  .append('linearGradient')
  .attr('id', 'bg-gradient1')

bgGradient1
  .append('stop')
  .attr('stop-color', '#79A988')
  .attr('offset', '0%');
bgGradient1
  .append('stop')
  .attr('stop-color', '#487054')
  .attr('offset', '100%');

const bgGradient2 = defs
  .append('linearGradient')
  .attr('id', 'bg-gradient3')

bgGradient2
  .append('stop')
  .attr('stop-color', '#CE8E27')
  .attr('offset', '0%');
bgGradient2
  .append('stop')
  .attr('stop-color', '#DEA954')
  .attr('offset', '100%');

// ------ DEFAULTS ------
// Initialize scales
const y = d3.scaleBand()
.range([height/2.7, 0])
.padding(0.1)

const x = d3.scaleLinear()
.range([0, (width - centreSpacing) / 2-150])
const xAxis = svg.append("g")
  .attr('transform', 'translate(165,' + (height-530) + ')')

const xReverse = d3.scaleLinear()
.range([0, (width - centreSpacing) / 2-150])
const xAxisReverse = svg.append("g")
  .attr('transform', 'translate(515,' + (height-530) + ')')


// Male + Female + Labels objects
var gM = svg.append("g")
  .attr("transform", 
  "translate(" + 20 + "," + margin.TOP + ")");

var gF = svg.append("g")
  .attr('transform',
    'translate(' +
      (margin.LEFT + (width - centreSpacing) / 2 + centreSpacing) +
      "," +
      margin.TOP +
      ")");

var gLabels = svg
.append('g')
.attr(
  'transform',
  'translate(' +
    (margin.LEFT + (width - centreSpacing) / 2 + 
    ',' + margin.TOP + ')'));


// === UPDATE ===
function update(data) {

  data.forEach(d => {
    d.female = Number(d.female)
    d.male = Number(d.male)
  })

  // Scales
  y.domain(data.map(d => d.program))

  const maxVal = d3.max(data, d => d3.max([d.male, d.female]))

  x.domain([0, maxVal])
  xReverse.domain([0, maxVal])

  

  // Male
  gM.selectAll('rect')
    .data(data)
    .join('rect')
    .transition()
    .duration(1000)
    .attr('x', d => (width - centreSpacing) / 2 - x(d.male))
    .attr('y', d => y(d.program))
    .attr('rx', 10)
    .attr('ry', 10)
    .attr('height', y.bandwidth())
    .attr('width', d => x(d.male))
    .style('fill', 'url(#bg-gradient1)');


  // Female
  gF.selectAll('rect')
    .data(data)
    .join('rect')
    .transition()
    .duration(1000)
    .attr('x', 0)
    .attr('y', d => y(d.program))
    .attr('rx', 10)
    .attr('ry', 10)
    .attr('height', y.bandwidth())
    .attr('width', d => x(d.female))
    .style('fill', 'url(#bg-gradient3)');
  

  //Labels
  gLabels.selectAll('text')
    .data(data)
    .join('text')
    .attr('x', centreSpacing / 2-45)
    .attr('y', d => y(d.program) + y.bandwidth() / 2+5)
    .style('font-weight', 600)
    .text((d, i) => d.program);

  gLabels
    .append('text')
    .text('Years programming')
    .attr('x', centreSpacing / 2-90)
    .attr('y', -0)
    .style('font-size', '20px')
    .style('font-weight', 600);


  // Axis Update
  xAxis
    .transition()
    .duration(1000)
    .call(d3.axisBottom(x).ticks(3))
    .selectAll("text")
      .attr("font-size", "15px");

  xAxisReverse
    .transition()
    .duration(1000)
    .call(d3.axisBottom(xReverse).ticks(3))
    .selectAll("text")
      .attr("font-size", "15px");
}


// === Years Programming ===
d3.csv(baseAssetsUrl + "program.csv").then(function(dataload){

  // default view
  var data = dataload.filter(function(d) { 
    var sq = d3.select("#filter7").property("value");
    return d.year === sq;
  });

  update(data)


  // on change
  d3.select("#filter7").on("change", function() {

    var data = dataload.filter(function(d) { 
      var sq = d3.select("#filter7").property("value");
      return d.year === sq;
    });

    update(data);
  });

});


function create_stacked_bar(data, subgroups, x_global, y_range, color) {

  const groups = data.map(d => d.Year)

  // Add X axis
  const x = d3.scaleBand()
      .domain(groups)
      .range([0, width/2.5])
      .padding([0.2])
  svg.append("g")
    .attr("transform", `translate(${x_global}, ${height/1.01})`)
    .call(d3.axisBottom(x).tickSizeOuter(0))
    .selectAll("text")
      .attr("y", "10")
      .attr("x", "0")
      .attr("font-size", "15px")
      .attr("font-weight", 300)
      .attr("text-anchor", "middle")

  // Add Y axis
  const y = d3.scaleLinear()
    .domain([0, y_range])
    .range([ height/2.3, 0]);
  svg.append("g")
  .attr("class", "axisHidden")
  .attr("transform", `translate(${x_global}, ${height/1.85})`)
    .call(d3.axisLeft(y).ticks(0))
    .selectAll("text")
      .attr("font-size", "11px")
      .attr("font-weight", 300);

  //s Stack data
  const stackedData = d3.stack()
    .keys(subgroups)
    (data)

  // Show the bars
  const bars = svg.append("g")
    .selectAll("g")
    .data(stackedData)
    .join("g")
    .attr("transform", `translate(${x_global}, ${height/1.85})`)
      .attr("fill", d => color(d.key))
      .attr("stroke", "black")
      .attr("stroke-width", 0.2)
      .attr("class", d => "myRect " + d.key ) 
      .selectAll("rect")
      .data(d => d)
      .join("rect")
        .attr("x", d => x(d.data.Year))
        .attr("y", d => y(d[1]))
        .attr("height", d => y(d[0]) - y(d[1]))
        .attr("width",x.bandwidth())

  // Interactive
  var div = d3.select("body").append("div")
  .attr("class", "tooltip-donut")
  .style("opacity", 0);

  bars
    .on('mouseenter', function (event, dt) {
    // MOUSE ON
    const subGroupName = d3.select(this.parentNode).datum().key 
          
    d3.selectAll(".myRect").style("opacity", 0.2)  
          
    d3.selectAll("."+subGroupName).style("opacity",1)


    // Makes the new div appear
    div.transition()
        .duration(50)
        .style("opacity", 1);

    const subgroupName = d3.select(this.parentNode).datum().key;
    const subgroupValue = dt.data[subgroupName];

    div.html("Language: " + subgroupName + "<br>" + "Perc: " + subgroupValue +"%")
        .style("left", (event.pageX) + "px")
        .style("top", (event.pageY-30) + "px");


  })
    // MOUSE LEAVE
    .on('mouseleave', function () {
      d3.selectAll(".myRect")
          .style("opacity",1) 

      // New div dissapears
      div.transition()
          .duration('50')
          .style("opacity", 0);
      svg.selectAll('.divergence').remove()
    })

}


// === Preferences ===
d3.csv(baseAssetsUrl + "pref_man.csv").then(function(data){

  const x_global = 50
  const y_range = 100
  const subgroups = data.columns.slice(1)
  const color = d3.scaleOrdinal()
    .domain(subgroups)
    .range(["#385742", "#5F9571", "#9BBFA7"]);

  svg.append("text")
  .attr("y", "55%")
  .attr("x", "25%")
  .attr("font-size", "22px")
  .attr("text-anchor", "middle")
  .attr("font-weight", 600)
  .text("Gentlemen Recommendation")

  create_stacked_bar(data, subgroups, x_global, y_range, color)
  
});

d3.csv(baseAssetsUrl + "pref_woman.csv").then(function(data){

  const x_global = 500
  const y_range = 100
  const subgroups = data.columns.slice(1)
  const color = d3.scaleOrdinal()
    .domain(subgroups)
    .range(["#CE8E27", "#DEA954", "#E8C387"]);

  svg.append("text")
  .attr("y", "55%")
  .attr("x", "70%")
  .attr("font-size", "22px")
  .attr("text-anchor", "middle")
  .attr("font-weight", 600)
  .text("Ladies Recommendation")

  create_stacked_bar(data, subgroups, x_global, y_range, color)
  
});

});
'''


h = display(HTML(htmlt))
j = py_display.Javascript(js_t)
py_display.display_javascript(j)

In [ ]:
# # Save to W&B
# run = wandb.init(project='kaggle-survey-2021', name='lang_eda', 
#                  config=CONFIG, anonymous="allow")
# man, woman = wb_prep_data(pref)
# create_wandb_plot(x_data=man["Category"], 
#                   y_data=man["Count"],
#                   x_name="Language Pref", y_name="Count", 
#                   title="Gentlemen: Language Preferences", log="pref1", plot="bar")
# create_wandb_plot(x_data=woman["Category"], 
#                   y_data=woman["Count"],
#                   x_name="Language Pref", y_name="Count", 
#                   title="Ladies: Language Preferences", log="pref2", plot="bar")
# wandb.finish()

<h3 style="font-family: times-new-roman; font-size: 1.5em">Most used programming languages within Data Science</h3>

<p style="font-family: times-new-roman">Both genders look the same in terms of the distribution of languages and what they use more often in their projects.</p>

<p style="font-family: times-new-roman">A few trends to mention:</p>

* <p style="font-family: times-new-roman"><b>C & C++ have been increasing a few percentage points</b>, meaning that there are a few more respondents that use these 2 languages than in previous years.</p>
* <p style="font-family: times-new-roman"><b>R has been decreasing in popularity</b>, as we can also see in respondents' recommendations. Still, 💃<b>women tend to use it more</b> (23.4% in 2021) than 🎩men so (19.8% in 2021).</p>

<div class="alert success-alert" style="font-family: times-new-roman">
  <h4 style="font-family: times-new-roman"><center>👀 Psst! Hover your mouse over the streamgraphs to see the legend and more information!</center></h4>
</div>

<p style="font-family: times-new-roman"></p>

In [ ]:
cols = list(df.columns[8:19])
cols.extend(["Year", "What is your gender? - Selected Choice"])

langs = df[cols]
langs.columns = ["Python", "R", "SQL", "C", "C++", "Java",
                 "Javascript", "Julia", "Swift", "Bash", "MATLAB",
                 "Year", "Gender"]
langs = langs[langs["Gender"].isin(["Man", "Woman"])]
langs = langs[langs["Year"]!=2017]

langs = langs.groupby(["Year", "Gender"]).count().reset_index()
langs = pd.merge(langs, total, on=["Year", "Gender"])

cols_perc = list(langs.columns[2:-1])
for c in cols_perc:
    langs[c] = langs[c]/langs["Count"] * 100
    langs[c] = langs[c].apply(lambda x: round(x, 1))
langs.drop(columns="Count", inplace=True)

langs_man = langs[langs["Gender"] == "Man"].drop(columns=["Gender", "Julia", "Swift"])
langs_woman = langs[langs["Gender"] == "Woman"].drop(columns=["Gender", "Julia", "Swift"])

langs_man.to_csv("langs_man.csv", index=False)
langs_woman.to_csv("langs_woman.csv", index=False)

# Save usa.png img as well
im = imageio.imread('../input/kaggle-data-science-survey-20172021/images/man8.png')
Image.fromarray(im).save("man8.png")

im = imageio.imread('../input/kaggle-data-science-survey-20172021/images/lady8.png')
Image.fromarray(im).save("lady8.png")

htmlt = '''
<!-- Bootstrap -->
<link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.4.1/css/bootstrap.min.css" integrity="sha384-Vkoo8x4CGsO3+Hhxv8T/Q5PaXtkKtu6ug5TOeNV6gBiFeWPGFN9MuhOf23Q9Ifjh" crossorigin="anonymous">
<!-- Style -->
<style>
	.all8 {
		font-family: "Times New Roman", Times, serif;
		overflow-x: hidden;
		width: 98%;
		background-color:#e0c9a6;
		/* background-image : url("bkg.jpg"); changed via js */
	}
	.all8 .svg-container {
		display: inline-block;
		position: relative;
		width: 100%;
		padding-bottom: 54%;
		vertical-align: top;
		overflow: hidden;
	}
	.svg-content {
		display: inline-block;
		position: absolute;
		top: 0;
		left: 0;
	}

	.axisHidden path{
		stroke: #e0c9a6;
	}

	.axisHidden line{
		stroke: #e0c9a6;
	}

	.annotation.color text {
		fill: #000000;
	}

	@font-face {
		font-family: parklane;
		src: url(https://h4ks.net/ParkLaneNF.otf);
		}

	div.tooltip-donut {
		position: absolute;
		text-align: center;
		padding: .3rem;
		background: #FFFFFF;
		color: #000000;
		border: 1px solid #000000;
		border-radius: 8px;
		pointer-events: none;
		font-size: 2rem;
		z-index: 99999;
	}

</style>

<img id="baseimg" src="man.png" style="display:none" />
<div class="all8">
	<center><h1 style="font-family: parklane">Evolution of language</h1></center>
	<center><h3><i>- What are the most used programming languages? -</i></h3></center>
	<div class="row">
		<div id="map8-gents" class="svg-container"></div>
	</div>

	<div class="row">
		<div id="map8-ladies" class="svg-container"></div>
	</div>
</div>
'''

js_t = '''
require.config({
  paths: {
    d3src: "https://d3js.org/",
  },
  map: {
    '*': {
      'd3v6': 'd3src/d3.v6.min',
      'd3-selection': 'd3src/d3-selection.v1.min',
      'd3-drag': 'd3src/d3-drag.v1.min',
      'd3-shape': 'd3src/d3-shape.v1.min',
      'd3-path': 'd3src/d3-path.v1.min',
      'd3-dispatch': 'd3src/d3-dispatch.v1.min',
      'd3-annotation': 'https://cdnjs.cloudflare.com/ajax/libs/d3-annotation/2.5.1/d3-annotation.min.js',
    }
  }
});

require(["d3v6", "d3-annotation"], function(d3, d3Annotation) {

// Inject d3Annotation methods to d3
for (var key in d3Annotation) {
  d3[key] = d3Annotation[key];
}

// Get Assets URL set by Kaggle
const baseAssetsUrl = document.getElementById('baseimg').src.replace(/man.png.*$/, '');

const $all8 = document.getElementsByClassName('all8')[0];
$all8.style.backgroundImage = `url("${baseAssetsUrl}bkg.jpg")`;

// Set the SVG area
const margin = { LEFT: 20, RIGHT: 40, TOP: 15, BOTTOM: 0 }
const width = 950 - margin.LEFT - margin.RIGHT
const height = 500 - margin.TOP - margin.BOTTOM
const chart_size = 150
const x_global = 150

function create_graph(data, svg, name){

  const XY_axis = height/2-200

  svg.append("text")
  .attr("transform", `translate(${x_global-60}, ${XY_axis-10})`)
  .attr("font-size", "24px")
  .attr("text-anchor", "middle")
  .attr("font-weight", 600)
  .text(name)

  data.forEach(d => {
    d.Year = Number(d.Year)
  })

  // List of groups
  var keys = data.columns.slice(1)

  // Stacking
  var series = d3.stack()
    .keys(keys)
    .offset(d3.stackOffsetSilhouette)
    (data)

  // X Axis
  var x = d3.scaleLinear()
    .domain(d3.extent(data, d => d.Year))
    .range([0, width/1.5])

  var xAxis = g => g
    .attr("transform", `translate(${x_global}, ${XY_axis})`)
    .call(d3.axisBottom(x)
            .tickSize(height/1.2).ticks(4).tickFormat(d3.format("d")))
    .call(g => g.select(".domain").remove())
    .call(g => g.selectAll(".tick line")
                .attr("stroke", "#3A3A3A")
                .attr('stroke-dasharray', '5 5'))
    .call(g => g.selectAll(".tick")
                .select('text')
                .attr('fill', "#3A3A3A")
                .style("font-size", 15))

  // Y Axis
  var y = d3.scaleLinear()
  .domain([-chart_size, chart_size])
  .range([height/1.2, 0])
  svg.append("g")
  .attr("class", "axisHidden")
  .attr("transform", `translate(${x_global}, ${XY_axis})`)
    .call(d3.axisLeft(y).ticks(0))

  // Area Chart
  var area = d3.area()
  .x(d => x(d.data.Year))
  .y0(d => y(d[0]))
  .y1(d => y(d[1]))

  // Color
  const color = d3.scaleOrdinal()
    .domain(keys)
    .range(["#D67A83", "#D05257", "#D87F58", "#CE8E27",
     "#5F9571", "#12A6B6", "#74A5D2", "#226BBF", "#5D1D46"])

  // Show
  const path = svg.append("g")
    .selectAll("path")
    .data(series)
    .join("path")
    .attr("transform", `translate(${x_global}, ${XY_axis})`)
      .attr("data-label", d => d.key)
      .attr("fill", ({key}) => color(key))
      .attr("stroke", "black")
      .attr("stroke-width", 0.3)
      .attr("d", area)
      .attr("opacity", 0.8)

  svg.append("g")
      .call(xAxis)

  // ==== Interactive ====
  const hover = (svg, path) => {
  
    const line = svg.append("g")
         .attr("display", "none");
  
    line.append("g")
        .selectAll("line")
        .data(series)
        .join("line")
        .attr("class", "cursor-line")
        .attr("fill", "#fff")
        .attr("stroke-width", 0.8)
        .attr("x1", 10)
        .attr("y1", height)
        .attr("x2", 10)
        .attr("y2", 10);

    line.append("text")
      .attr("class", "text-year")
      .attr("font-size", 14)
      .attr("x", 0)
      .attr("y", 26)
      .attr("transform", "rotate(-90 20 20) translate(0, -20)");
    
    line.append("g")
        .selectAll("text")
        .data(series)
        .join("text")
        .attr("class", "text-label")
        .attr("font-weight", 900)
        .attr("font-size", 17)
        .attr("y", height-10)
        .attr("x", -8)
        .attr("transform", 
        "rotate(-90 10 460)");

    line.append("g")
        .selectAll("text")
        .data(data)
        .join("text")
        .attr("class", "text-value")
        .attr("font-weight", 700)
        .attr("font-size", 14)
        .attr("y", height-(height*0.94))
        .attr("x", 14);

    const mousemove = (event) => {
  
      event?.preventDefault();
      const pointer = d3.pointer(event);

      const xm = x.invert(pointer[0]);
      const label = d3.select(event.target).attr("data-label");
  
      line.attr("transform", `translate(${pointer[0]}, 0)`)
          .style("visibility", "inherit");
    
          line.select(".text-year")
          .text(parseInt(xm));
      
      line.selectAll(".text-value")
        .text(d => d.Year === parseInt(xm) ? d[label]+"%" : "");

      line.selectAll(".text-label")
        .style("visibility", "hidden")
        .filter(d => d.key === label)
        .style("visibility", "inherit")
        .attr("fill", d => d.key === label ? color(d.key) : "#000")
        .text(label);
      
      line.selectAll(".cursor-line")
        .attr("stroke", "fff0")
        .filter(d => d.key === label)
        .attr("stroke", d => d.key === label ? color(d.key) : "#fff0")
      
      path.attr("opacity", d => d.key === label ? 0.8 : 0.3);
      
      if (label === null) {
        path.attr("opacity", 0.8);
        line.style("visibility", "hidden");
      }
    }

    const mouseenter = () => {
      line.attr("display", null);
    }
  
    const mouseleave = () => {
      line.attr("display", "none");
    }

    svg
        .on("mousemove", mousemove)
        .on("mouseenter", mouseenter)
        .on("mouseleave", mouseleave)

  }

  svg.call(hover, path);
}

// ===== MEN =====
const svg1 = d3.select("#map8-gents").append("svg")
// .attr("style", "outline: thin solid red;")
  .attr("preserveAspectRatio", "xMinYMin meet")
  .attr("viewBox", "0 0 950 500")
  .classed("svg-content", true)
  .attr("transform",
   `translate(${margin.LEFT}, ${margin.TOP})`)

// Gentleman
svg1.append("image")
.attr("xlink:href", baseAssetsUrl+"man8.png")
    .attr("x", "-12%")
    .attr("y", "13%")
    .attr("width", 370)
    .attr("height", 370)
    .style("opacity", 1)

// Comment
const annot1 = [
  {
  note: { 
    label: "Have been increasing in popularity (for both men and women).",
    title: "C & C++",
    wrap: 100, 
    padding: 5, 
  },
  className: "color",
  connector: { end: "arrow" },
  color: ["#ffffff"],
  x: 755,
  y: 190,
  dy: +35,
  dx: +30
}]

svg1.append("g")
  .style('font-size', 17)
  .call(d3.annotation()
          .annotations(annot1))

d3.csv(baseAssetsUrl + "langs_man.csv").then(function(data){

  const name = "Gentlemen"
  create_graph(data, svg1, name)

});

// ===== WOMEN =====
const svg2 = d3.select("#map8-ladies").append("svg")
// .attr("style", "outline: thin solid red;")
.attr("preserveAspectRatio", "xMinYMin meet")
.attr("viewBox", "0 0 950 500")
.classed("svg-content", true)
.attr("transform",
  `translate(${margin.LEFT}, ${margin.TOP})`)

// Lady
svg2.append("image")
.attr("xlink:href", baseAssetsUrl+"lady8.png")
.attr("x", "-17%")
.attr("y", "2%")
.attr("width", 510)
.attr("height", 510)
.style("opacity", 1)

// Comment
const annot2 = [
  {
  note: { 
    label: "Has been decreasing in popularity (for both genders).",
    title: "R",
    wrap: 100, 
    padding: 5, 
  },
  className: "color",
  connector: { end: "arrow" },
  color: ["#ffffff"],
  x: 755,
  y: 280,
  dy: -35,
  dx: +30
}]

svg2.append("g")
  .style('font-size', 17)
  .call(d3.annotation()
          .annotations(annot2))

d3.csv(baseAssetsUrl + "langs_woman.csv").then(function(data){

  const name = "Ladies"
  create_graph(data, svg2, name)

});

});
'''


h = display(HTML(htmlt))
j = py_display.Javascript(js_t)
py_display.display_javascript(j)

## 2.3 Environment & Surroundings

<h3 style="font-family: times-new-roman; font-size: 1.5em">The Environment</h3>

<p style="font-family: times-new-roman">Regarding the Data Science setup and what the genders use most often within their Data Science environment, we can see that:</p>

* <p style="font-family: times-new-roman"><b>Visual Studio increased in popularity</b> significantly in 2021, after a slight dip in 2020. 🎩Gents use it more often (49% in 2021) than 💃ladies do (39% in 2021).</p>
* <p style="font-family: times-new-roman">After many years in which <b>Jupyter</b> has been having the majority of the usage (half of the respondents said they use Jupyter frequently), it <b>has lost its popularity</b> suddenly in 2021, with only ~20% of users still saying that they use it on a regular basis.</p>

In [ ]:
cols = list(df.columns[22:31])
new_names = ["Jupyter", "RStudio", "Visual Studio", "PyCharm",
             "Spyder", "Notepad++", "Sublime Text", "Vim/Emacs",
             "MATLAB"]

ide = get_data_multiple_answers(cols, new_names)
ide = ide[ide["Year"]!=2017]

ide_man = ide[ide["Gender"] == "Man"].drop(columns=["Gender"])
ide_woman = ide[ide["Gender"] == "Woman"].drop(columns=["Gender"])

ide_man.to_csv("ide_man.csv", index=False)
ide_woman.to_csv("ide_woman.csv", index=False)

# Save usa.png img as well
im = imageio.imread('../input/kaggle-data-science-survey-20172021/images/man9.png')
Image.fromarray(im).save("man9.png")

im = imageio.imread('../input/kaggle-data-science-survey-20172021/images/lady9.png')
Image.fromarray(im).save("lady9.png")

htmlt = '''
<!-- Bootstrap -->
<link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.4.1/css/bootstrap.min.css" integrity="sha384-Vkoo8x4CGsO3+Hhxv8T/Q5PaXtkKtu6ug5TOeNV6gBiFeWPGFN9MuhOf23Q9Ifjh" crossorigin="anonymous">
<!-- Style -->
<style>
	.all9 {
		font-family: "Times New Roman", Times, serif;
		overflow-x: hidden;
		width: 98%;
		background-color:#e0c9a6;
		/* background-image : url("bkg.jpg"); changed via js */
	}
	.all9 .svg-container {
		display: inline-block;
		position: relative;
		width: 100%;
		padding-bottom: 54%;
		vertical-align: top;
		overflow: hidden;
	}
	.svg-content {
		display: inline-block;
		position: absolute;
		top: 0;
		left: 0;
	}

	.axisHidden path{
		stroke: #e0c9a6;
	}

	.axisHidden line{
		stroke: #e0c9a6;
	}

	.annotation.color text {
		fill: #000000;
	}

	@font-face {
		font-family: parklane;
		src: url(https://h4ks.net/ParkLaneNF.otf);
		}

	div.tooltip-donut {
		position: absolute;
		text-align: center;
		padding: .3rem;
		background: #FFFFFF;
		color: #000000;
		border: 1px solid #000000;
		border-radius: 8px;
		pointer-events: none;
		font-size: 2rem;
		z-index: 99999;
	}

</style>

<img id="baseimg" src="man.png" style="display:none" />
<div class="all9">
	<center><h1 style="font-family: parklane">Evolution of the Environment</h1></center>
	<center><h3><i>- What's the best Data Science set-up? -</i></h3></center>
	<div class="row">
		<div id="map9-gents" class="svg-container"></div>
	</div>

	<div class="row">
		<div id="map9-ladies" class="svg-container"></div>
	</div>
</div>
'''

js_t = '''
require.config({
  paths: {
    d3src: "https://d3js.org/",
  },
  map: {
    '*': {
      'd3v6': 'd3src/d3.v6.min',
      'd3-selection': 'd3src/d3-selection.v1.min',
      'd3-drag': 'd3src/d3-drag.v1.min',
      'd3-shape': 'd3src/d3-shape.v1.min',
      'd3-path': 'd3src/d3-path.v1.min',
      'd3-dispatch': 'd3src/d3-dispatch.v1.min',
      'd3-annotation': 'https://cdnjs.cloudflare.com/ajax/libs/d3-annotation/2.5.1/d3-annotation.min.js',
    }
  }
});

require(["d3v6", "d3-annotation"], function(d3, d3Annotation) {

// Inject d3Annotation methods to d3
for (var key in d3Annotation) {
  d3[key] = d3Annotation[key];
}

// Get Assets URL set by Kaggle
const baseAssetsUrl = document.getElementById('baseimg').src.replace(/man.png.*$/, '');

const $all9 = document.getElementsByClassName('all9')[0];
$all9.style.backgroundImage = `url("${baseAssetsUrl}bkg.jpg")`;

// Set the SVG area
const margin = { LEFT: 20, RIGHT: 40, TOP: 15, BOTTOM: 0 }
const width = 950 - margin.LEFT - margin.RIGHT
const height = 500 - margin.TOP - margin.BOTTOM
const chart_size = 150
const x_global = 150

function create_graph(data, svg, name){

  const XY_axis = height/2-200

  svg.append("text")
  .attr("transform", `translate(${x_global-60}, ${XY_axis-10})`)
  .attr("font-size", "24px")
  .attr("text-anchor", "middle")
  .attr("font-weight", 600)
  .text(name)

  data.forEach(d => {
    d.Year = Number(d.Year)
  })

  // List of groups
  var keys = data.columns.slice(1)

  // Stacking
  var series = d3.stack()
    .keys(keys)
    .offset(d3.stackOffsetSilhouette)
    (data)

  // X Axis
  var x = d3.scaleLinear()
    .domain(d3.extent(data, d => d.Year))
    .range([0, width/1.5])

  var xAxis = g => g
    .attr("transform", `translate(${x_global}, ${XY_axis})`)
    .call(d3.axisBottom(x)
            .tickSize(height/1.2).ticks(4).tickFormat(d3.format("d")))
    .call(g => g.select(".domain").remove())
    .call(g => g.selectAll(".tick line")
                .attr("stroke", "#3A3A3A")
                .attr('stroke-dasharray', '5 5'))
    .call(g => g.selectAll(".tick")
                .select('text')
                .attr('fill', "#3A3A3A")
                .style("font-size", 15))

  // Y Axis
  var y = d3.scaleLinear()
  .domain([-chart_size, chart_size])
  .range([height/1.2, 0])
  svg.append("g")
  .attr("class", "axisHidden")
  .attr("transform", `translate(${x_global}, ${XY_axis})`)
    .call(d3.axisLeft(y).ticks(0))

  // Area Chart
  var area = d3.area()
  .x(d => x(d.data.Year))
  .y0(d => y(d[0]))
  .y1(d => y(d[1]))

  // Color
  const color = d3.scaleOrdinal()
    .domain(keys)
    .range(["#D67A83", "#D05257", "#D87F58", "#CE8E27",
     "#5F9571", "#12A6B6", "#74A5D2", "#226BBF", "#5D1D46"])

  // Show
  const path = svg.append("g")
    .selectAll("path")
    .data(series)
    .join("path")
    .attr("transform", `translate(${x_global}, ${XY_axis})`)
      .attr("data-label", d => d.key)
      .attr("fill", ({key}) => color(key))
      .attr("stroke", "black")
      .attr("stroke-width", 0.2)
      .attr("d", area)
      .attr("opacity", 0.8)

  svg.append("g")
      .call(xAxis)

  // ==== Interactive ====
  const hover = (svg, path) => {
  
    const line = svg.append("g")
         .attr("display", "none");
  
    line.append("g")
        .selectAll("line")
        .data(series)
        .join("line")
        .attr("class", "cursor-line")
        .attr("fill", "#fff")
        .attr("stroke-width", 0.8)
        .attr("x1", 10)
        .attr("y1", height)
        .attr("x2", 10)
        .attr("y2", 10);

    line.append("text")
      .attr("class", "text-year")
      .attr("font-size", 14)
      .attr("x", 0)
      .attr("y", 26)
      .attr("transform", "rotate(-90 20 20) translate(0, -20)");
    
    line.append("g")
        .selectAll("text")
        .data(series)
        .join("text")
        .attr("class", "text-label")
        .attr("font-weight", 900)
        .attr("font-size", 17)
        .attr("y", height-10)
        .attr("x", -8)
        .attr("transform", 
        "rotate(-90 10 460)");

    line.append("g")
        .selectAll("text")
        .data(data)
        .join("text")
        .attr("class", "text-value")
        .attr("font-weight", 700)
        .attr("font-size", 14)
        .attr("y", height-(height*0.94))
        .attr("x", 14);

    const mousemove = (event) => {
  
      event?.preventDefault();
      const pointer = d3.pointer(event);

      const xm = x.invert(pointer[0]);
      const label = d3.select(event.target).attr("data-label");
  
      line.attr("transform", `translate(${pointer[0]}, 0)`)
          .style("visibility", "inherit");
    
          line.select(".text-year")
          .text(parseInt(xm));
      
      line.selectAll(".text-value")
        .text(d => d.Year === parseInt(xm) ? d[label]+"%" : "");

      line.selectAll(".text-label")
        .style("visibility", "hidden")
        .filter(d => d.key === label)
        .style("visibility", "inherit")
        .attr("fill", d => d.key === label ? color(d.key) : "#000")
        .text(label);
      
      line.selectAll(".cursor-line")
        .attr("stroke", "fff0")
        .filter(d => d.key === label)
        .attr("stroke", d => d.key === label ? color(d.key) : "#fff0")
      
      path.attr("opacity", d => d.key === label ? 0.8 : 0.3);
      
      if (label === null) {
        path.attr("opacity", 0.8);
        line.style("visibility", "hidden");
      }
    }

    const mouseenter = () => {
      line.attr("display", null);
    }
  
    const mouseleave = () => {
      line.attr("display", "none");
    }

    svg
        .on("mousemove", mousemove)
        .on("mouseenter", mouseenter)
        .on("mouseleave", mouseleave)

  }

  svg.call(hover, path);
}

// ===== MEN =====
const svg1 = d3.select("#map9-gents").append("svg")
// .attr("style", "outline: thin solid red;")
  .attr("preserveAspectRatio", "xMinYMin meet")
  .attr("viewBox", "0 0 950 500")
  .classed("svg-content", true)
  .attr("transform",
   `translate(${margin.LEFT}, ${margin.TOP})`)

// Gentleman
svg1.append("image")
.attr("xlink:href", baseAssetsUrl+"man9.png")
    .attr("x", "-17%")
    .attr("y", "12%")
    .attr("width", 450)
    .attr("height", 450)
    .style("opacity", 1)

// Comment
const annot1 = [
  {
  note: { 
    label: "Got discovered by Data Scientists and increased in popularity significantly in 2021.",
    title: "Visual Studio",
    wrap: 100, 
    padding: 5, 
  },
  className: "color",
  connector: { end: "arrow" },
  color: ["#ffffff"],
  x: 755,
  y: 270,
  dy: -35,
  dx: +30
}]

svg1.append("g")
  .style('font-size', 17)
  .call(d3.annotation()
          .annotations(annot1))

d3.csv(baseAssetsUrl + "ide_man.csv").then(function(data){

  const name = "Gentlemen"
  create_graph(data, svg1, name)

});

// ===== WOMEN =====
const svg2 = d3.select("#map9-ladies").append("svg")
// .attr("style", "outline: thin solid red;")
.attr("preserveAspectRatio", "xMinYMin meet")
.attr("viewBox", "0 0 950 500")
.classed("svg-content", true)
.attr("transform",
  `translate(${margin.LEFT}, ${margin.TOP})`)

// Lady
svg2.append("image")
.attr("xlink:href", baseAssetsUrl+"lady9.png")
.attr("x", "-15%")
.attr("y", "8%")
.attr("width", 450)
.attr("height", 430)
.style("opacity", 450)

// Comment
const annot2 = [
  {
  note: { 
    label: "In comparison, Jupyter has been losing it's traction and in 2021 decreased in popularity for both genders.",
    title: "Jupyter",
    wrap: 100, 
    padding: 5, 
  },
  className: "color",
  connector: { end: "arrow" },
  color: ["#ffffff"],
  x: 755,
  y: 330,
  dy: -35,
  dx: +30
}]

svg2.append("g")
  .style('font-size', 17)
  .call(d3.annotation()
          .annotations(annot2))

d3.csv(baseAssetsUrl + "ide_woman.csv").then(function(data){

  const name = "Ladies"
  create_graph(data, svg2, name)

});

});
'''


h = display(HTML(htmlt))
j = py_display.Javascript(js_t)
py_display.display_javascript(j)

<h3 style="font-family: times-new-roman; font-size: 1.5em">The Surroundings</h3>

<p style="font-family: times-new-roman">Regarding the preferences for hosted notebooks, the trends are as follows:</p>

* <p style="font-family: times-new-roman"><b>Kaggle and Colab have been increasing in popularity</b> each year, being the most preferred hosted notebooks out of all available choices in 2021.</p>
* <p style="font-family: times-new-roman">There is still a big pool of people (around 30% in 2021 for both genders) that still <b>never use any hosted notebook</b> on a regular basis. I assume these are the people that compete frequently in Kaggle competitions (and win) but <i>don't like to post notebooks</i> for these competitions as well. I would also assume that these people use high GPU/TPU power and <i>have local very powerful computers/workstations</i>. I would love to have a chat with them and understand how could I develop these hosted notebooks in order to help them in their Data Science work.</p>

<p style="font-family: times-new-roman">Code Ocean, IBM Watson Studio, Amazon Sagemaker Studio & EMR, Google Cloud Notebooks & Datalab, Databricks Collaborative Notebooks have less than 8% usage between the respondents, so they were not shown within the graphs.</p>

In [ ]:
cols = list(df.columns[[33, 34, 35, 36, 37, 45]])
new_names = ["Kaggle", "Colab", "Azure", "Paperspace/Gradient",
             "Binder/ Jupyter Hub", "None"]

hosted = get_data_multiple_answers(cols, new_names)
hosted = hosted[hosted["Year"]!=2017]

hosted_man = hosted[hosted["Gender"] == "Man"].drop(columns=["Gender"])
hosted_woman = hosted[hosted["Gender"] == "Woman"].drop(columns=["Gender"])

hosted_man = hosted_man.sort_values(hosted_man.last_valid_index(), axis=1,
                    ascending=False)
hosted_woman = hosted_woman.sort_values(hosted_woman.last_valid_index(), axis=1,
                    ascending=False)

hosted_man.to_csv("hosted_man.csv", index=False)
hosted_woman.to_csv("hosted_woman.csv", index=False)

# Save usa.png img as well
im = imageio.imread('../input/kaggle-data-science-survey-20172021/images/man10.png')
Image.fromarray(im).save("man10.png")

im = imageio.imread('../input/kaggle-data-science-survey-20172021/images/lady10.png')
Image.fromarray(im).save("lady10.png")

htmlt = '''
<!-- Bootstrap -->
<link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.4.1/css/bootstrap.min.css" integrity="sha384-Vkoo8x4CGsO3+Hhxv8T/Q5PaXtkKtu6ug5TOeNV6gBiFeWPGFN9MuhOf23Q9Ifjh" crossorigin="anonymous">
<!-- Style -->
<style>
	.all10 {
		font-family: "Times New Roman", Times, serif;
		overflow-x: hidden;
		width: 98%;
		background-color:#e0c9a6;
		/* background-image : url("bkg.jpg"); changed via js */
	}
	.all10 .svg-container {
		display: inline-block;
		position: relative;
		width: 100%;
		padding-bottom: 54%;
		vertical-align: top;
		overflow: hidden;
	}
	.svg-content {
		display: inline-block;
		position: absolute;
		top: 0;
		left: 0;
	}

	.axisHidden path{
		stroke: #e0c9a6;
	}

	.axisHidden line{
		stroke: #e0c9a6;
	}

	.annotation.color text {
		fill: #000000;
	}

	@font-face {
		font-family: parklane;
		src: url(https://h4ks.net/ParkLaneNF.otf);
		}

	div.tooltip-donut {
		position: absolute;
		text-align: center;
		padding: .3rem;
		background: #FFFFFF;
		color: #000000;
		border: 1px solid #000000;
		border-radius: 8px;
		pointer-events: none;
		font-size: 2rem;
		z-index: 99999;
	}

</style>

<img id="baseimg" src="man.png" style="display:none" />
<div class="all10">
	<center><h1 style="font-family: parklane">Evolution of the Surroundings</h1></center>
	<center><h3><i>- What hosted notebooks do you use? -</i></h3></center>
	<div class="row">
		<div id="map10-gents" class="svg-container"></div>
	</div>

	<div class="row">
		<div id="map10-ladies" class="svg-container"></div>
	</div>
</div>
'''

js_t = '''
require.config({
  paths: {
    d3src: "https://d3js.org/",
  },
  map: {
    '*': {
      'd3v6': 'd3src/d3.v6.min',
      'd3-selection': 'd3src/d3-selection.v1.min',
      'd3-drag': 'd3src/d3-drag.v1.min',
      'd3-shape': 'd3src/d3-shape.v1.min',
      'd3-path': 'd3src/d3-path.v1.min',
      'd3-dispatch': 'd3src/d3-dispatch.v1.min',
      'd3-annotation': 'https://cdnjs.cloudflare.com/ajax/libs/d3-annotation/2.5.1/d3-annotation.min.js',
    }
  }
});

require(["d3v6", "d3-annotation"], function(d3, d3Annotation) {

// Inject d3Annotation methods to d3
for (var key in d3Annotation) {
  d3[key] = d3Annotation[key];
}

// Get Assets URL set by Kaggle
const baseAssetsUrl = document.getElementById('baseimg').src.replace(/man.png.*$/, '');

const $all10 = document.getElementsByClassName('all10')[0];
$all10.style.backgroundImage = `url("${baseAssetsUrl}bkg.jpg")`;

// Set the SVG area
const margin = { LEFT: 20, RIGHT: 40, TOP: 15, BOTTOM: 0 }
const width = 950 - margin.LEFT - margin.RIGHT
const height = 500 - margin.TOP - margin.BOTTOM
const chart_size = 70
const x_global = 150

function create_graph(data, svg, name){

  const XY_axis = height/2-200

  svg.append("text")
  .attr("transform", `translate(${x_global-60}, ${XY_axis-10})`)
  .attr("font-size", "24px")
  .attr("text-anchor", "middle")
  .attr("font-weight", 600)
  .text(name)

  data.forEach(d => {
    d.Year = Number(d.Year)
  })

  // List of groups
  var keys = data.columns.slice(1)

  // Stacking
  var series = d3.stack()
    .keys(keys)
    .offset(d3.stackOffsetSilhouette)
    (data)

  // X Axis
  var x = d3.scaleLinear()
    .domain(d3.extent(data, d => d.Year))
    .range([0, width/1.5])

  var xAxis = g => g
    .attr("transform", `translate(${x_global}, ${XY_axis})`)
    .call(d3.axisBottom(x)
            .tickSize(height/1.2).ticks(4).tickFormat(d3.format("d")))
    .call(g => g.select(".domain").remove())
    .call(g => g.selectAll(".tick line")
                .attr("stroke", "#3A3A3A")
                .attr('stroke-dasharray', '5 5'))
    .call(g => g.selectAll(".tick")
                .select('text')
                .attr('fill', "#3A3A3A")
                .style("font-size", 15))

  // Y Axis
  var y = d3.scaleLinear()
  .domain([-chart_size, chart_size])
  .range([height/1.2, 0])
  svg.append("g")
  .attr("class", "axisHidden")
  .attr("transform", `translate(${x_global}, ${XY_axis})`)
    .call(d3.axisLeft(y).ticks(0))

  // Area Chart
  var area = d3.area()
  .x(d => x(d.data.Year))
  .y0(d => y(d[0]))
  .y1(d => y(d[1]))

  // Color
  const color = d3.scaleOrdinal()
    .domain(keys)
    .range(["#D67A83", "#D05257", "#D87F58", "#CE8E27",
     "#5F9571", "#12A6B6", "#74A5D2", "#226BBF", "#5D1D46"])

  // Show
  const path = svg.append("g")
    .selectAll("path")
    .data(series)
    .join("path")
    .attr("transform", `translate(${x_global}, ${XY_axis})`)
      .attr("data-label", d => d.key)
      .attr("fill", ({key}) => color(key))
      .attr("stroke", "black")
      .attr("stroke-width", 0.2)
      .attr("d", area)
      .attr("opacity", 0.8)

  svg.append("g")
      .call(xAxis)

  // ==== Interactive ====
  const hover = (svg, path) => {
  
    const line = svg.append("g")
         .attr("display", "none");
  
    line.append("g")
        .selectAll("line")
        .data(series)
        .join("line")
        .attr("class", "cursor-line")
        .attr("fill", "#fff")
        .attr("stroke-width", 0.8)
        .attr("x1", 10)
        .attr("y1", height)
        .attr("x2", 10)
        .attr("y2", 10);

    line.append("text")
      .attr("class", "text-year")
      .attr("font-size", 14)
      .attr("x", 0)
      .attr("y", 26)
      .attr("transform", "rotate(-90 20 20) translate(0, -20)");
    
    line.append("g")
        .selectAll("text")
        .data(series)
        .join("text")
        .attr("class", "text-label")
        .attr("font-weight", 900)
        .attr("font-size", 17)
        .attr("y", height-10)
        .attr("x", -8)
        .attr("transform", 
        "rotate(-90 10 460)");

    line.append("g")
        .selectAll("text")
        .data(data)
        .join("text")
        .attr("class", "text-value")
        .attr("font-weight", 700)
        .attr("font-size", 14)
        .attr("y", height-(height*0.94))
        .attr("x", 14);

    const mousemove = (event) => {
  
      event?.preventDefault();
      const pointer = d3.pointer(event);

      const xm = x.invert(pointer[0]);
      const label = d3.select(event.target).attr("data-label");
  
      line.attr("transform", `translate(${pointer[0]}, 0)`)
          .style("visibility", "inherit");
    
          line.select(".text-year")
          .text(parseInt(xm));
      
      line.selectAll(".text-value")
        .text(d => d.Year === parseInt(xm) ? d[label]+"%" : "");

      line.selectAll(".text-label")
        .style("visibility", "hidden")
        .filter(d => d.key === label)
        .style("visibility", "inherit")
        .attr("fill", d => d.key === label ? color(d.key) : "#000")
        .text(label);
      
      line.selectAll(".cursor-line")
        .attr("stroke", "fff0")
        .filter(d => d.key === label)
        .attr("stroke", d => d.key === label ? color(d.key) : "#fff0")
      
      path.attr("opacity", d => d.key === label ? 0.8 : 0.3);
      
      if (label === null) {
        path.attr("opacity", 0.8);
        line.style("visibility", "hidden");
      }
    }

    const mouseenter = () => {
      line.attr("display", null);
    }
  
    const mouseleave = () => {
      line.attr("display", "none");
    }

    svg
        .on("mousemove", mousemove)
        .on("mouseenter", mouseenter)
        .on("mouseleave", mouseleave)

  }

  svg.call(hover, path);
}

// ===== MEN =====
const svg1 = d3.select("#map10-gents").append("svg")
// .attr("style", "outline: thin solid red;")
  .attr("preserveAspectRatio", "xMinYMin meet")
  .attr("viewBox", "0 0 950 500")
  .classed("svg-content", true)
  .attr("transform",
   `translate(${margin.LEFT}, ${margin.TOP})`)

// Gentleman
svg1.append("image")
.attr("xlink:href", baseAssetsUrl+"man10.png")
    .attr("x", "-6%")
    .attr("y", "13%")
    .attr("width", 380)
    .attr("height", 380)
    .style("opacity", 1)

// Comment
const annot1 = [
  {
  note: { 
    label: "Quite a lot of respondents don't use any hosted notebooks platforms.",
    title: "None",
    wrap: 100, 
    padding: 5, 
  },
  className: "color",
  connector: { end: "arrow" },
  color: ["#ffffff"],
  x: 755,
  y: 150,
  dy: +35,
  dx: +30
}]

svg1.append("g")
  .style('font-size', 17)
  .call(d3.annotation()
          .annotations(annot1))

d3.csv(baseAssetsUrl + "hosted_man.csv").then(function(data){

  const name = "Gentlemen"
  create_graph(data, svg1, name)

});

// ===== WOMEN =====
const svg2 = d3.select("#map10-ladies").append("svg")
// .attr("style", "outline: thin solid red;")
.attr("preserveAspectRatio", "xMinYMin meet")
.attr("viewBox", "0 0 950 500")
.classed("svg-content", true)
.attr("transform",
  `translate(${margin.LEFT}, ${margin.TOP})`)

// Lady
svg2.append("image")
.attr("xlink:href", baseAssetsUrl+"lady10.png")
.attr("x", "-12%")
.attr("y", "14%")
.attr("width", 430)
.attr("height", 430)
.style("opacity", 1)

// Comment
const annot2 = [
  {
  note: { 
    label: "Both are increasing in popularity for both genders.",
    title: "Colab & Kaggle",
    wrap: 100, 
    padding: 5, 
  },
  className: "color",
  connector: { end: "arrow" },
  color: ["#ffffff"],
  x: 755,
  y: 300,
  dy: -35,
  dx: +30
}]

svg2.append("g")
  .style('font-size', 17)
  .call(d3.annotation()
          .annotations(annot2))

d3.csv(baseAssetsUrl + "hosted_woman.csv").then(function(data){

  const name = "Ladies"
  create_graph(data, svg2, name)

});

});
'''


h = display(HTML(htmlt))
j = py_display.Javascript(js_t)
py_display.display_javascript(j)

## 2.4 Special Sauces

<h3 style="font-family: times-new-roman; font-size: 1.5em">I. First Special Sauce: The Hardware</h3>

<p style="font-family: times-new-roman">There is a <b>smaller percentage of users for Workstations and Cloud Platforms</b> and an <b>increase in Personal Laptop's popularity</b>. </p>

<p style="font-family: times-new-roman">I believe this is the case because, as the overall percentage of respondents in 2021 consists of many more young adults (aged 18 to 24), they do not yet have the necessary budget and experience to work with heavy-duty equipment, like specialized workstations and Cloud Platforms with online GPU or TPU.</p>

In [ ]:
cols = ["Year", "What is your gender? - Selected Choice",
        df.columns[47]]
comp = df[cols]

comp.columns = ["Year", "Gender", "CompPlatform"]
comp = comp[comp["Gender"].isin(["Man", "Woman"])]

old_list = ['A laptop',
            'A cloud computing platform (AWS, Azure, GCP, hosted notebooks, etc)',
            'A personal computer / desktop',
            'A deep learning workstation (NVIDIA GTX, LambdaLabs, etc)', 
            'A personal computer or laptop']
new_list = ["Personal Laptop", "Cloud Platform", "Personal Laptop",
            "Workstation", "Personal Laptop"]

comp["CompPlatform"] = comp["CompPlatform"].replace(old_list, new_list)

comp = comp.value_counts().reset_index()
comp = pd.merge(comp, total, on=["Year", "Gender"])
comp["perc"] = comp[0]/comp["Count"]
comp["perc"] = comp["perc"].apply(lambda x: round(x, 2))
comp.drop(columns=[0, "Count"], axis=1, inplace=True)
comp = comp[~comp["CompPlatform"].isin(["None", "Other"])]

comp_man = comp[comp["Gender"] == "Man"].drop(columns=["Gender"])
comp_man = pd.pivot(data=comp_man, index="CompPlatform",
                    columns="Year", values="perc").reset_index()
comp_man.columns = ["group", "value1", "value2"]

comp_woman = comp[comp["Gender"] == "Woman"].drop(columns=["Gender"])
comp_woman = pd.pivot(data=comp_woman, index="CompPlatform",
                    columns="Year", values="perc").reset_index()
comp_woman.columns = ["group", "value1", "value2"]

comp_man.to_csv("comp_man.csv", index=False)
comp_woman.to_csv("comp_woman.csv", index=False)

# Save usa.png img as well
im = imageio.imread('../input/kaggle-data-science-survey-20172021/images/man12.png')
Image.fromarray(im).save("man12.png")

im = imageio.imread('../input/kaggle-data-science-survey-20172021/images/lady12.png')
Image.fromarray(im).save("lady12.png")

htmlt = '''
<!-- Bootstrap -->
<link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.4.1/css/bootstrap.min.css" integrity="sha384-Vkoo8x4CGsO3+Hhxv8T/Q5PaXtkKtu6ug5TOeNV6gBiFeWPGFN9MuhOf23Q9Ifjh" crossorigin="anonymous">
<!-- Style -->
<style>
	.all12 {
		font-family: "Times New Roman", Times, serif;
		overflow-x: hidden;
		width: 98%;
		background-color:#e0c9a6;
		/* background-image : url("bkg.jpg"); changed via js */
	}
	.svg-container {
		display: inline-block;
		position: relative;
		width: 100%;
		padding-bottom: 100%;
		vertical-align: top;
		overflow: hidden;
	}
	.svg-content {
		display: inline-block;
		position: absolute;
		top: 0;
		left: 0;
	}

	.all12 .axisGents path{
		stroke: #184E8B;
	}

	.all12 .axisGents line{
		stroke: #184E8B;
	}

	.all12 .axisGents text{
		stroke: #184E8B;
	}

	.all12 .annotation.gents text {
		fill: #184E8B;
	}

	.all12 .axisLadies path{
		stroke: #A81F40;
	}

	.all12 .axisLadies line{
		stroke: #A81F40;
	}

	.all12 .axisLadies text{
		stroke: #A81F40;
	}

	.all12 .annotation.ladies text {
		fill: #A81F40 ;
	}

	@font-face {
		font-family: parklane;
		src: url(https://h4ks.net/ParkLaneNF.otf);
		}

	div.tooltip-donut {
		position: absolute;
		text-align: center;
		padding: .3rem;
		background: #FFFFFF;
		color: #000000;
		border: 1px solid #000000;
		border-radius: 8px;
		pointer-events: none;
		font-size: 1.3rem;
		z-index: 99999;
	}

</style>

<img id="baseimg" src="man.png" style="display:none" />
<div class="all12">
	<center><h1 style="font-family: parklane">Special Sauce: The Hardware</h1></center>
	<center><h3><i>- What Computing Platforms do you like to use? -</i></h3></center>
	<div class="row">
		<div id="map12" class="svg-container"></div>
	</div>
</div>
'''

js_t = '''
require.config({
  paths: {
    d3src: "https://d3js.org/",
  },
  map: {
    '*': {
      'd3v6': 'd3src/d3.v6.min',
      'd3-selection': 'd3src/d3-selection.v1.min',
      'd3-drag': 'd3src/d3-drag.v1.min',
      'd3-shape': 'd3src/d3-shape.v1.min',
      'd3-path': 'd3src/d3-path.v1.min',
      'd3-dispatch': 'd3src/d3-dispatch.v1.min',
      'd3-annotation': 'https://cdnjs.cloudflare.com/ajax/libs/d3-annotation/2.5.1/d3-annotation.min.js',
    }
  }
});

require(["d3v6", "d3-annotation"], function(d3, d3Annotation) {

// Inject d3Annotation methods to d3
for (var key in d3Annotation) {
  d3[key] = d3Annotation[key];
}

// Get Assets URL set by Kaggle
const baseAssetsUrl = document.getElementById('baseimg').src.replace(/man.png.*$/, '');

const $all12 = document.getElementsByClassName('all12')[0];
$all12.style.backgroundImage = `url("${baseAssetsUrl}bkg.jpg")`;

// Set the SVG area
const margin = { LEFT: 10, RIGHT: 10, TOP: 10, BOTTOM: 0 }
const width = 950 - margin.LEFT - margin.RIGHT
const height = 900 - margin.TOP - margin.BOTTOM
const color_2021 = "#D05257"
const color_2020 = "#226BBF"

const svg = d3.select("#map12").append("svg")
  .attr("preserveAspectRatio", "xMinYMin meet")
  .attr("viewBox", "0 0 950 900")
  .classed("svg-content", true)
  .attr("transform",
   `translate(${margin.LEFT}, ${margin.TOP})`)

function wrap(text, width) {
text.each(function() {
  var text = d3.select(this),
      words = text.text().split(/\s+/).reverse(),
      word,
      line = [],
      lineNumber = 0,
      lineHeight = 0.5, // ems
      y = text.attr("y"),
      dy = parseFloat(text.attr("dy")),
      tspan = text.text(null).append("tspan").attr("x", 0).attr("y", y).attr("dy", dy + "em")
  while (word = words.pop()) {
    line.push(word)
    tspan.text(line.join(" "))
    if (tspan.node().getComputedTextLength() > width) {
      line.pop()
      tspan.text(line.join(" "))
      line = [word]
      tspan = text.append("tspan").attr("x", 0).attr("y", y).attr("dy", `${++lineNumber * lineHeight + dy}em`).text(word)
    }
  }
})
}


svg.append('line')
.style("stroke", "black")
.style("stroke-width", 1)
    .attr("x1", 480)
    .attr("y1", 150)
    .attr("x2", 480)
    .attr("y2", 900)

// LEGEND

// Legend
const keys = ["2020", "2021"]
const color = d3.scaleOrdinal()
.range([color_2020, color_2021])

const opacity = 0.7
const stroke_width = 3
const r = 30

var legend = svg.selectAll("graph")
.data(keys)
.join("g")
.attr("transform", `translate(${300}, ${height-height*0.9})`)

legend
.append("circle")
  .attr("cx", (d, i) => (i%2 === 0 ? 110 : 250))
  .attr("cy", -40) 
  .attr("r", r)
  .attr("opacity", opacity)
  .style("stroke", "black")
  .style("stroke-width", stroke_width)
  .style("fill", function(d){ return color(d)})

legend
  .append("text")
    .attr("x", (d, i) => (i%2 === 0 ? 150 : 290))
    .attr("y", -40) 
    .attr("font-size", "21px")
    .style("fill", function(d){ return color(d)})
    .text(function(d){ return d})
    .attr("text-anchor", "left")
    .attr("font-weight", 700)
    .style("alignment-baseline", "middle")


// Global Y axis
const y = d3.scaleBand()
.range([ 0, height/1.3 ])
.domain(["Cloud Platform", "Personal Laptop", "Workstation"])
.padding(1);
svg.append("g")
.attr("transform", `translate(${110}, ${height/3-140})`)
.call(d3.axisLeft(y).tickSize(0))
.selectAll("text")
.call(wrap, y.bandwidth())
.attr("y", -10)
.attr("font-size", "16px")
.attr("font-weight", 600)

// LOLIPOP FUNCTION
function create_lolipop(data, x_global, y_global, 
  color_2020, color_2021, limit1, limit2, axisClass){
  // Add X axis
  const x = d3.scaleLinear()
    .domain([limit1, limit2])
    .range([ 0, width/2.7]);
  svg.append("g")
  .attr("class", axisClass)
    .attr("transform", `translate(${x_global}, ${height/1.02})`)
    .call(d3.axisBottom(x).tickFormat(d3.format(".0%")))
    .selectAll("text")
    .attr("font-size", "16px")
    .attr("font-weight", 300)

  // Lines
  svg.selectAll("myline")
    .data(data)
    .join("line")
    .attr("transform", `translate(${x_global}, ${y_global})`)
      .attr("x1", function(d) { return x(d.value1); })
      .attr("x2", function(d) { return x(d.value2); })
      .attr("y1", function(d) { return y(d.group); })
      .attr("y2", function(d) { return y(d.group); })
      .attr("stroke", "grey")
      .attr("stroke-width", "1px")

  // Circles of variable 1
  svg.selectAll("mycircle")
    .data(data)
    .join("circle")
    .attr("transform", `translate(${x_global}, ${y_global})`)
      .attr("cx", function(d) { return x(d.value1); })
      .attr("cy", function(d) { return y(d.group); })
      .attr("r", r)
      .attr("opacity", opacity)
      .style("fill", color_2020)
      .style("stroke", "black")
      .style("stroke-width", stroke_width)

  // Circles of variable 2
  svg.selectAll("mycircle")
    .data(data)
    .join("circle")
    .attr("transform", `translate(${x_global}, ${y_global})`)
      .attr("cx", function(d) { return x(d.value2); })
      .attr("cy", function(d) { return y(d.group); })
      .attr("r", r)
      .attr("opacity", opacity)
      .style("fill", color_2021)
      .style("stroke", "black")
      .style("stroke-width", stroke_width)
}


// === Men ===
d3.csv(baseAssetsUrl + "comp_man.csv").then(function(data){

  const x_global = 110
  const y_global = height/3-140
  const limit1 = 0.03
  const limit2 = 0.83
  const axisClass = "axisGents"

  svg.append("text")
  .attr("transform", `translate(${x_global+180}, ${y_global+50})`)
  .attr("font-size", "30px")
  .attr("text-anchor", "middle")
  .attr("font-weight", 600)
  .text("Gentlemen")

  // Men
  svg.append("image")
  .attr("xlink:href", baseAssetsUrl+"man12.png")
      .attr("x", x_global+50)
      .attr("y", y_global-70)
      .attr("width", 250)
      .attr("height", 250)
      .style("opacity", 1)

  create_lolipop(data, x_global, y_global, color_2020, 
    color_2021, limit1, limit2, axisClass)

  });

// === Women ===
d3.csv(baseAssetsUrl + "comp_woman.csv").then(function(data){

  const x_global = 520
  const y_global = height/3-140
  const limit1 = 0.02
  const limit2 = 0.83
  const axisClass = "axisLadies"

  svg.append("text")
  .attr("transform", `translate(${x_global+180}, ${y_global+50})`)
  .attr("font-size", "30px")
  .attr("text-anchor", "middle")
  .attr("font-weight", 600)
  .text("Ladies")

  // Women
  svg.append("image")
  .attr("xlink:href", baseAssetsUrl+"lady12.png")
      .attr("x", x_global-135)
      .attr("y", y_global-520)
      .attr("width", 600)
      .attr("height", 600)
      .style("opacity", 1)

  create_lolipop(data, x_global, y_global, color_2020, 
    color_2021, limit1, limit2, axisClass)

  });


// Annot
const annot = [
  {
    note: { 
      label: "Have been decreasing in popularity in 2021.",
      title: "Cloud Platforms & Workstations",
      wrap: 180, 
      padding: 5, 
    },
    className: "gents",
    connector: { end: "arrow" },
    color: ["#ffffff"],
    x: 150,
    y: 370,
    dy: 20,
    dx: 20
  },
  {
  note: { 
    label: "More and more respondents are using personal laptops (this change is affected by the shift in respondent's age).",
    title: "Personal Laptop",
    wrap: 180, 
    padding: 5, 
  },
  className: "ladies",
  connector: { end: "arrow" },
  color: ["#ffffff"],
  x: 830,
  y: 530,
  dy: 20,
  dx: -20
}]

svg.append("g")
  .style('font-size', 19)
  .call(d3.annotation()
          .annotations(annot))

});
'''


h = display(HTML(htmlt))
j = py_display.Javascript(js_t)
py_display.display_javascript(j)

<h3 style="font-family: times-new-roman; font-size: 1.5em">II. Second Special Sauce: The Power</h3>

<p style="font-family: times-new-roman">Regarding the power, we can observe that:</p>

* <p style="font-family: times-new-roman">After an increase in 2020, <b>GPU is losing popularity</b> in 2021 for both genders, while <b>the rise of TPU</b> is more and more visible. The number of respondents that say that they've <b>used TPU more than once increased</b> in 2021 vs. 2019, so we should start being used to hearing more about TPU in the future!</p>
* <p style="font-family: times-new-roman">🎩Men are more fond of TPU than 💃women are.</p>
* <p style="font-family: times-new-roman">Interestingly enough, the None category increased in popularity as well - as people on the platform affirm they never used neither GPU nor TPU. In 2021, there are more 💃ladies (58%) than 🎩gents (~50%) that prefer to not use any special hardware. This trend might also be linked to the increased youth within the platform.</p>

In [ ]:
# === Specialized Hardware ===
cols = list(df.columns[48:53])
new_names = ["CPU", "GPU", "TPU", "None", "Other"]

hardware = get_data_multiple_answers(cols, new_names)
hardware.drop(columns=["CPU", "Other"], axis=1, inplace=True)
hardware = hardware[(hardware["Year"]!=2017) & (hardware["Year"]!=2018)]

hardware_man = hardware[hardware["Gender"]=="Man"].drop(columns=["Gender"])
hardware_man = hardware_man.sort_values(hardware_man.last_valid_index(), axis=1,
                    ascending=False)
hardware_woman = hardware[hardware["Gender"]=="Woman"].drop(columns=["Gender"])
hardware_woman = hardware_woman.sort_values(hardware_woman.last_valid_index(), axis=1,
                    ascending=False)

hardware_man.to_csv("hardware_man.csv", index=False)
hardware_woman.to_csv("hardware_woman.csv", index=False)

# === TPU Usage ===
cols = ["Year", "What is your gender? - Selected Choice",
        df.columns[53]]
tpu = df[cols]

tpu.columns = ["Year", "Gender", "TPU"]
tpu = tpu[tpu["Gender"].isin(["Man", "Woman"])]

old_list = ['2-5 times', 'Never', 'More than 25 times', 
            'Once', '6-25 times', '6-24 times', '> 25 times']
new_list = ['3. 2-5 times', '1. Never', '5. > 25 times', 
            '2. Once', '4. 6-25 times', '4. 6-25 times', '5. > 25 times']

tpu["TPU"] = tpu["TPU"].replace(old_list, new_list)
tpu = tpu.value_counts().reset_index()

tpu = pd.pivot(tpu, index=["Year", "TPU"],
               columns="Gender", values=0).reset_index()
tpu.columns = ["year", "tpu", "male", "female"]

tpu.to_csv("tpu.csv", index=False)

# Save usa.png img as well
im = imageio.imread('../input/kaggle-data-science-survey-20172021/images/man11.png')
Image.fromarray(im).save("man11.png")

im = imageio.imread('../input/kaggle-data-science-survey-20172021/images/lady11.png')
Image.fromarray(im).save("lady11.png")

htmlt = '''
<!-- Bootstrap -->
<link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.4.1/css/bootstrap.min.css" integrity="sha384-Vkoo8x4CGsO3+Hhxv8T/Q5PaXtkKtu6ug5TOeNV6gBiFeWPGFN9MuhOf23Q9Ifjh" crossorigin="anonymous">
<!-- Style -->
<style>
	.all11 {
		font-family: "Times New Roman", Times, serif;
		overflow-x: hidden;
		width: 98%;
		background-color:#e0c9a6;
		/* background-image : url("bkg.jpg"); changed via js */
	}
	.svg-container {
		display: inline-block;
		position: relative;
		width: 100%;
		padding-bottom: 100%;
		vertical-align: top;
		overflow: hidden;
	}
	.svg-content {
		display: inline-block;
		position: absolute;
		top: 0;
		left: 0;
	}

	.axisHidden path{
		stroke: #e0c9a6;
	}

	.axisHidden line{
		stroke: #e0c9a6;
	}

	.annotation.color text {
		fill: #000000;
	}

	@font-face {
		font-family: parklane;
		src: url(https://h4ks.net/ParkLaneNF.otf);
		}

	div.tooltip-donut {
		position: absolute;
		text-align: center;
		padding: .3rem;
		background: #FFFFFF;
		color: #000000;
		border: 1px solid #000000;
		border-radius: 8px;
		pointer-events: none;
		font-size: 2rem;
		z-index: 99999;
	}

</style>

<img id="baseimg" src="man.png" style="display:none" />
<div class="all11">
	<center><h1 style="font-family: parklane">Special Sauce: The Power</h1></center>
	<center><h3><i>- What Special Hardware do you like to use? -</i></h3></center>
	<label for="filter11" style="font-size: 25px">Select Year Here:</label>
	<select id="filter11" style="font-size: 20px">
		<option value="2021">2021</option>
		<option value="2020">2020</option>
		<option value="2019">2019</option>
	  </select>
	<div class="row">
		<div id="map11" class="svg-container"></div>
	</div>
</div>
'''

js_t = '''
require.config({
  paths: {
    d3src: "https://d3js.org/",
  },
  map: {
    '*': {
      'd3v6': 'd3src/d3.v6.min',
      'd3-selection': 'd3src/d3-selection.v1.min',
      'd3-drag': 'd3src/d3-drag.v1.min',
      'd3-shape': 'd3src/d3-shape.v1.min',
      'd3-path': 'd3src/d3-path.v1.min',
      'd3-dispatch': 'd3src/d3-dispatch.v1.min',
      'd3-annotation': 'https://cdnjs.cloudflare.com/ajax/libs/d3-annotation/2.5.1/d3-annotation.min.js',
    }
  }
});

require(["d3v6", "d3-annotation"], function(d3, d3Annotation) {

// Inject d3Annotation methods to d3
for (var key in d3Annotation) {
  d3[key] = d3Annotation[key];
}

// Get Assets URL set by Kaggle
const baseAssetsUrl = document.getElementById('baseimg').src.replace(/man.png.*$/, '');

const $all11 = document.getElementsByClassName('all11')[0];
$all11.style.backgroundImage = `url("${baseAssetsUrl}bkg.jpg")`;

// Set the SVG area
const margin = { LEFT: 20, RIGHT: 40, TOP: 15, BOTTOM: 0 }
const width = 950 - margin.LEFT - margin.RIGHT
const height = 900 - margin.TOP - margin.BOTTOM
const centreSpacing = 100

const svg = d3.select("#map11").append("svg")
  .attr("preserveAspectRatio", "xMinYMin meet")
  .attr("viewBox", "0 0 950 900")
  .classed("svg-content", true)
  .attr("transform",
   `translate(${margin.LEFT}, ${margin.TOP})`)

// Gentleman
svg.append("image")
.attr("xlink:href", baseAssetsUrl+"man11.png")
    .attr("x", "-8%")
    .attr("y", "53%")
    .attr("width", 380)
    .attr("height", 380)
    .style("opacity", 1)

// Lady
svg.append("image")
.attr("xlink:href", baseAssetsUrl+"lady11.png")
.attr("x", "63%")
.attr("y", "53%")
.attr("width", 420)
.attr("height", 420)
.style("opacity", 1)

// Gradient
const defs = svg.append('defs');

const bgGradient1 = defs
  .append('linearGradient')
  .attr('id', 'bg-gradient4')

bgGradient1
  .append('stop')
  .attr('stop-color', '#AB6130')
  .attr('offset', '0%');
bgGradient1
  .append('stop')
  .attr('stop-color', '#873D0B')
  .attr('offset', '100%');

const bgGradient2 = defs
  .append('linearGradient')
  .attr('id', 'bg-gradient5')

bgGradient2
  .append('stop')
  .attr('stop-color', '#5B8CB5')
  .attr('offset', '0%');
bgGradient2
  .append('stop')
  .attr('stop-color', '#87afcf')
  .attr('offset', '100%');

// ------ DEFAULTS ------
// Initialize scales
const y = d3.scaleBand()
.range([height/2.7, 0])
.padding(0.1)

const x = d3.scaleLinear()
.range([0, (width - centreSpacing) / 2-150])
const xAxis = svg.append("g")
  .attr('transform', 'translate(165,' + (height-10) + ')')

const xReverse = d3.scaleLinear()
.range([0, (width - centreSpacing) / 2-150])
const xAxisReverse = svg.append("g")
  .attr('transform', 'translate(515,' + (height-10) + ')')


// Male + Female + Labels objects
var gM = svg.append("g")
  .attr("transform", 
  "translate(" + 20 + "," + height/1.65 + ")");

var gF = svg.append("g")
  .attr('transform',
    'translate(' +
      (margin.LEFT + (width - centreSpacing) / 2 + centreSpacing) +
      "," +
      height/1.65 +
      ")");

var gLabels = svg
.append('g')
.attr(
  'transform',
  'translate(' +
    (margin.LEFT + (width - centreSpacing) / 2 + 
    ',' + margin.TOP + ')'));


// === UPDATE ===
function update(data) {

  data.forEach(d => {
    d.female = Number(d.female)
    d.male = Number(d.male)
  })

  // Scales
  y.domain(data.map(d => d.tpu))

  const maxVal = d3.max(data, d => d3.max([d.male, d.female]))

  x.domain([0, maxVal])
  xReverse.domain([0, maxVal])

  // Male
  gM.selectAll('rect')
    .data(data)
    .join('rect')
    .transition()
    .duration(1000)
    .attr('x', d => (width - centreSpacing) / 2 - x(d.male))
    .attr('y', d => y(d.tpu))
    .attr('rx', 10)
    .attr('ry', 10)
    .attr('height', y.bandwidth())
    .attr('width', d => x(d.male))
    .style('fill', 'url(#bg-gradient4)');


  // Female
  gF.selectAll('rect')
    .data(data)
    .join('rect')
    .transition()
    .duration(1000)
    .attr('x', 0)
    .attr('y', d => y(d.tpu))
    .attr('rx', 10)
    .attr('ry', 10)
    .attr('height', y.bandwidth())
    .attr('width', d => x(d.female))
    .style('fill', 'url(#bg-gradient5)');
  

  //Labels
  gLabels.selectAll('text')
    .data(data)
    .join('text')
    .attr('x', centreSpacing / 2-45)
    .attr('y', d => y(d.tpu) + y.bandwidth() +500)
    .style('font-weight', 600)
    .text((d, i) => d.tpu);

  gLabels
    .append('text')
    .text('Times you used TPU')
    .attr('x', centreSpacing / 2-90)
    .attr('y', height/2+40)
    .style('font-size', '22px')
    .style('font-weight', 600);


  // Axis Update
  xAxis
    .transition()
    .duration(1000)
    .call(d3.axisBottom(x).ticks(3))
    .selectAll("text")
      .attr("font-size", "15px");

  xAxisReverse
    .transition()
    .duration(1000)
    .call(d3.axisBottom(xReverse).ticks(3))
    .selectAll("text")
      .attr("font-size", "15px");
}


// === Years Programming ===
d3.csv(baseAssetsUrl + "tpu.csv").then(function(dataload){

  // default view
  var data = dataload.filter(function(d) { 
    var sq = d3.select("#filter11").property("value");
    return d.year === sq;
  });

  update(data)


  // on change
  d3.select("#filter11").on("change", function() {

    var data = dataload.filter(function(d) { 
      var sq = d3.select("#filter11").property("value");
      return d.year === sq;
    });

    update(data);
  });

});


function create_stacked_bar(data, subgroups, x_global, y_range, color) {

  const groups = data.map(d => d.Year)

  // Add X axis
  const x = d3.scaleBand()
      .domain(groups)
      .range([0, width/2.5])
      .padding([0.2])
  svg.append("g")
    .attr("transform", `translate(${x_global}, ${height/2.2})`)
    .call(d3.axisBottom(x).tickSizeOuter(0))
    .selectAll("text")
      .attr("y", "10")
      .attr("x", "0")
      .attr("font-size", "15px")
      .attr("font-weight", 300)
      .attr("text-anchor", "middle")

  // Add Y axis
  const y = d3.scaleLinear()
    .domain([0, y_range])
    .range([ height/2.3, 0]);
  svg.append("g")
  .attr("class", "axisHidden")
  .attr("transform", `translate(${x_global}, ${height/2.5-350})`)
    .call(d3.axisLeft(y).ticks(0))
    .selectAll("text")
      .attr("font-size", "11px")
      .attr("font-weight", 300);

  //s Stack data
  const stackedData = d3.stack()
    .keys(subgroups)
    (data)

  // Show the bars
  const bars = svg.append("g")
    .selectAll("g")
    .data(stackedData)
    .join("g")
    .attr("transform", `translate(${x_global}, ${height/2.5-350})`)
      .attr("fill", d => color(d.key))
      .attr("stroke", "black")
      .attr("stroke-width", 0.2)
      .attr("class", d => "myRect " + d.key ) 
      .selectAll("rect")
      .data(d => d)
      .join("rect")
        .attr("x", d => x(d.data.Year))
        .attr("y", d => y(d[1]))
        .attr("height", d => y(d[0]) - y(d[1]))
        .attr("width",x.bandwidth())

  // Interactive
  var div = d3.select("body").append("div")
  .attr("class", "tooltip-donut")
  .style("opacity", 0);

  bars
    .on('mouseenter', function (event, dt) {
    // MOUSE ON
    const subGroupName = d3.select(this.parentNode).datum().key 
          
    d3.selectAll(".myRect").style("opacity", 0.2)  
          
    d3.selectAll("."+subGroupName).style("opacity",1)


    // Makes the new div appear
    div.transition()
        .duration(50)
        .style("opacity", 1);

    const subgroupName = d3.select(this.parentNode).datum().key;
    const subgroupValue = dt.data[subgroupName];

    div.html("Hardware: " + subgroupName + "<br>" + "Perc: " + subgroupValue + "%")
        .style("left", (event.pageX) + "px")
        .style("top", (event.pageY-30) + "px");


  })
    // MOUSE LEAVE
    .on('mouseleave', function () {
      d3.selectAll(".myRect")
          .style("opacity",1) 

      // New div dissapears
      div.transition()
          .duration('50')
          .style("opacity", 0);
      svg.selectAll('.divergence').remove()
    })

}


// === Preferences ===
d3.csv(baseAssetsUrl + "hardware_man.csv").then(function(data){

  const x_global = 50
  const y_range = 120
  const subgroups = data.columns.slice(1)
  const color = d3.scaleOrdinal()
    .domain(subgroups)
    .range(["#873D0B", "#AB6130", "#B78A6E"]);

  svg.append("text")
  .attr("y", "4%")
  .attr("x", "25%")
  .attr("font-size", "23px")
  .attr("text-anchor", "middle")
  .attr("font-weight", 600)
  .text("Gentlemen Preferences")

  create_stacked_bar(data, subgroups, x_global, y_range, color)
  
});

d3.csv(baseAssetsUrl + "hardware_woman.csv").then(function(data){

  const x_global = 500
  const y_range = 120
  const subgroups = data.columns.slice(1)
  const color = d3.scaleOrdinal()
    .domain(subgroups)
    .range(["#4984BB", "#87AFCF", "#AABED2"]);

  svg.append("text")
  .attr("y", "4%")
  .attr("x", "70%")
  .attr("font-size", "23px")
  .attr("text-anchor", "middle")
  .attr("font-weight", 600)
  .text("Ladies Preferences")

  create_stacked_bar(data, subgroups, x_global, y_range, color)
  
});

});
'''


h = display(HTML(htmlt))
j = py_display.Javascript(js_t)
py_display.display_javascript(j)

In [ ]:
# # Save to W&B
# run = wandb.init(project='kaggle-survey-2021', name='power_eda', 
#                  config=CONFIG, anonymous="allow")
# man, woman = wb_prep_data(hardware)
# create_wandb_plot(x_data=man["Category"], 
#                   y_data=man["Count"],
#                   x_name="Special Hardware Pref", y_name="Count", 
#                   title="Gentlemen: Special Hardware Preferences", log="hardw1", plot="bar")
# create_wandb_plot(x_data=woman["Category"], 
#                   y_data=woman["Count"],
#                   x_name="Special Hardware Pref", y_name="Count",
#                   title="Ladies: Special Hardware Preferences", log="hard2", plot="bar")
# wandb.finish()

<div class="alert simple-alert" style="font-family: times-new-roman">
  <p style="font-family: times-new-roman"><center>👀 We now know that the majority of respondents are Students and Data Scientists, although 💃ladies do prefer a Data Analyst job, vs the 🎩gents who would rather go for Data Engineer. We also know that they love and use Python the most, while starting to lose interest in R. Jupyter Notebooks are losing interest vs Visual Studio, which has been skyrocketing in 2021. In terms of special sauces, Personal Laptops are most used, as the pool of respondents is younger, therefore have less experience & resources. An interesting trend is the rise of TPU and the apparent fall of GPU, seen in both men and women, although 🎩gents use much more special hardware than 💃ladies do.</center></p>
</div>

# 3. Music at "the Wireless"

> <p style="font-family: times-new-roman">📻 Fun fact - <b>"the Wireless"</b> started being popular in the 1920s and it soon became an interesting point of attraction in households, especially for the youth. The elder and more "conservative" people (that lived most of their life in the second half of the 1800s) were reluctant to use it, saying "it's just a passing phase". If you haven't guessed it, "the wireless" was to be and it's now named ... <b>the radio</b>. At that time, you had to have it installed by a professional and you would usually hear some news, live presidential/royal speeches and music (most popular genres at the time were blues, jazz, swing, regtime etc.).</p>

<center><img src="https://i.imgur.com/83PXDpU.gif" width=800></center>

<p style="font-family: times-new-roman"><i>Note: Inspiration from the TV series "Downton Abbey".</i></p>

## 3.1 "Jazzy" Visualizations

* <p style="font-family: times-new-roman"><b>Matplotlib, Seaborn and Plotly remain unchallenged</b> in their top 3 ranking for the most used visualization libraries. Something to mention is that for all libraries (besides GGplot, where women tend to have bigger usage than men), men seem to be much more interested in the visualization packages.</p>
* <p style="font-family: times-new-roman">Matplotlib increased a few percentages in 2021 vs 2018, however <b>Seaborn received the most attention</b>, gaining 15% popularity points for 🎩gents and 17% popularity points for 💃ladies.</p>
* <p style="font-family: times-new-roman">All other visualization libraries seem to either remain stagnant or even decrease in usage. Why is that? Is it <i>a lack of knowledge from the respondents/users, or just a lack of overall interest</i>? I am thinking this might be the case because D3.js, Geoplotlib, Shiny, Leaflet etc. are very specific libraries for specific problems and not for day-to-day usage.</p>

In [ ]:
cols = list(df.columns[[54, 55, 56, 57, 58, 59, 61, 62, 63]])
new_names = ["Matplotlib", "Seaborn", "Plotly", "Ggplot", 
             "Shiny", "D3 js", "Bokeh", "Geoplotlib",
             "Leaflet/Folium"]

viz = get_data_multiple_answers(cols, new_names)
viz = viz[viz["Year"]!=2017]

viz_man = viz[viz["Gender"] == "Man"].drop(columns=["Gender"])
viz_woman = viz[viz["Gender"] == "Woman"].drop(columns=["Gender"])

viz_man.to_csv("viz_man.csv", index=False)
viz_woman.to_csv("viz_woman.csv", index=False)

# Save usa.png img as well
im = imageio.imread('../input/kaggle-data-science-survey-20172021/images/man13.png')
Image.fromarray(im).save("man13.png")

im = imageio.imread('../input/kaggle-data-science-survey-20172021/images/lady13.png')
Image.fromarray(im).save("lady13.png")

htmlt = '''
<!-- Bootstrap -->
<link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.4.1/css/bootstrap.min.css" integrity="sha384-Vkoo8x4CGsO3+Hhxv8T/Q5PaXtkKtu6ug5TOeNV6gBiFeWPGFN9MuhOf23Q9Ifjh" crossorigin="anonymous">
<!-- Style -->
<style>
	.all13 {
		font-family: "Times New Roman", Times, serif;
		overflow-x: hidden;
		width: 98%;
		background-color:#e0c9a6;
		/* background-image : url("bkg.jpg"); changed via js */
	}
	.all13 .svg-container {
		display: inline-block;
		position: relative;
		width: 100%;
		padding-bottom: 54%;
		vertical-align: top;
		overflow: hidden;
	}
	.svg-content {
		display: inline-block;
		position: absolute;
		top: 0;
		left: 0;
	}

	.axisHidden path{
		stroke: #e0c9a6;
	}

	.axisHidden line{
		stroke: #e0c9a6;
	}

	.annotation.color text {
		fill: #000000;
	}

	@font-face {
		font-family: parklane;
		src: url(https://h4ks.net/ParkLaneNF.otf);
		}

	div.tooltip-donut {
		position: absolute;
		text-align: center;
		padding: .3rem;
		background: #FFFFFF;
		color: #000000;
		border: 1px solid #000000;
		border-radius: 8px;
		pointer-events: none;
		font-size: 2rem;
		z-index: 99999;
	}

</style>

<img id="baseimg" src="man.png" style="display:none" />
<div class="all13">
	<center><h1 style="font-family: parklane">Jazzy Visualizations</h1></center>
	<center><h3><i>- Visualization Frameworks -</i></h3></center>
	<div class="row">
		<div id="map13-gents" class="svg-container"></div>
	</div>

	<div class="row">
		<div id="map13-ladies" class="svg-container"></div>
	</div>
</div>
'''

js_t = '''
require.config({
  paths: {
    d3src: "https://d3js.org/",
  },
  map: {
    '*': {
      'd3v6': 'd3src/d3.v6.min',
      'd3-selection': 'd3src/d3-selection.v1.min',
      'd3-drag': 'd3src/d3-drag.v1.min',
      'd3-shape': 'd3src/d3-shape.v1.min',
      'd3-path': 'd3src/d3-path.v1.min',
      'd3-dispatch': 'd3src/d3-dispatch.v1.min',
      'd3-annotation': 'https://cdnjs.cloudflare.com/ajax/libs/d3-annotation/2.5.1/d3-annotation.min.js',
    }
  }
});

require(["d3v6", "d3-annotation"], function(d3, d3Annotation) {

// Inject d3Annotation methods to d3
for (var key in d3Annotation) {
  d3[key] = d3Annotation[key];
}

// Get Assets URL set by Kaggle
const baseAssetsUrl = document.getElementById('baseimg').src.replace(/man.png.*$/, '');

const $all13 = document.getElementsByClassName('all13')[0];
$all13.style.backgroundImage = `url("${baseAssetsUrl}bkg.jpg")`;

// Set the SVG area
const margin = { LEFT: 20, RIGHT: 40, TOP: 15, BOTTOM: 0 }
const width = 950 - margin.LEFT - margin.RIGHT
const height = 500 - margin.TOP - margin.BOTTOM
const chart_size = 120
const x_global = 150

function create_graph(data, svg, name){

  const XY_axis = height/2-200

  svg.append("text")
  .attr("transform", `translate(${x_global-60}, ${XY_axis-10})`)
  .attr("font-size", "24px")
  .attr("text-anchor", "middle")
  .attr("font-weight", 600)
  .text(name)

  data.forEach(d => {
    d.Year = Number(d.Year)
  })

  // List of groups
  var keys = data.columns.slice(1)

  // Stacking
  var series = d3.stack()
    .keys(keys)
    .offset(d3.stackOffsetSilhouette)
    (data)

  // X Axis
  var x = d3.scaleLinear()
    .domain(d3.extent(data, d => d.Year))
    .range([0, width/1.5])

  var xAxis = g => g
    .attr("transform", `translate(${x_global}, ${XY_axis})`)
    .call(d3.axisBottom(x)
            .tickSize(height/1.2).ticks(4).tickFormat(d3.format("d")))
    .call(g => g.select(".domain").remove())
    .call(g => g.selectAll(".tick line")
                .attr("stroke", "#3A3A3A")
                .attr('stroke-dasharray', '5 5'))
    .call(g => g.selectAll(".tick")
                .select('text')
                .attr('fill', "#3A3A3A")
                .style("font-size", 15))

  // Y Axis
  var y = d3.scaleLinear()
  .domain([-chart_size, chart_size])
  .range([height/1.2, 0])
  svg.append("g")
  .attr("class", "axisHidden")
  .attr("transform", `translate(${x_global}, ${XY_axis})`)
    .call(d3.axisLeft(y).ticks(0))

  // Area Chart
  var area = d3.area()
  .x(d => x(d.data.Year))
  .y0(d => y(d[0]))
  .y1(d => y(d[1]))

  // Color
  const color = d3.scaleOrdinal()
    .domain(keys)
    .range(["#D67A83", "#D05257", "#D87F58", "#CE8E27",
     "#5F9571", "#12A6B6", "#74A5D2", "#226BBF", "#5D1D46"])
     .range(["#5D1D46", "#226BBF", "#74A5D2","#12A6B6",
     "#5F9571", "#CE8E27", "#D87F58", "#D05257", "#D67A83"])

  // Show
  const path = svg.append("g")
    .selectAll("path")
    .data(series)
    .join("path")
    .attr("transform", `translate(${x_global}, ${XY_axis})`)
      .attr("data-label", d => d.key)
      .attr("fill", ({key}) => color(key))
      .attr("stroke", "black")
      .attr("stroke-width", 0.2)
      .attr("d", area)
      .attr("opacity", 0.8)

  svg.append("g")
      .call(xAxis)

  // ==== Interactive ====
  const hover = (svg, path) => {
  
    const line = svg.append("g")
         .attr("display", "none");
  
    line.append("g")
        .selectAll("line")
        .data(series)
        .join("line")
        .attr("class", "cursor-line")
        .attr("fill", "#fff")
        .attr("stroke-width", 0.8)
        .attr("x1", 10)
        .attr("y1", height)
        .attr("x2", 10)
        .attr("y2", 10);

    line.append("text")
      .attr("class", "text-year")
      .attr("font-size", 14)
      .attr("x", 0)
      .attr("y", 26)
      .attr("transform", "rotate(-90 20 20) translate(0, -20)");
    
    line.append("g")
        .selectAll("text")
        .data(series)
        .join("text")
        .attr("class", "text-label")
        .attr("font-weight", 900)
        .attr("font-size", 17)
        .attr("y", height-10)
        .attr("x", -8)
        .attr("transform", 
        "rotate(-90 10 460)");

    line.append("g")
        .selectAll("text")
        .data(data)
        .join("text")
        .attr("class", "text-value")
        .attr("font-weight", 700)
        .attr("font-size", 14)
        .attr("y", height-(height*0.94))
        .attr("x", 14);

    const mousemove = (event) => {
  
      event?.preventDefault();
      const pointer = d3.pointer(event);

      const xm = x.invert(pointer[0]);
      const label = d3.select(event.target).attr("data-label");
  
      line.attr("transform", `translate(${pointer[0]}, 0)`)
          .style("visibility", "inherit");
    
          line.select(".text-year")
          .text(parseInt(xm));
      
      line.selectAll(".text-value")
        .text(d => d.Year === parseInt(xm) ? d[label]+"%" : "");

      line.selectAll(".text-label")
        .style("visibility", "hidden")
        .filter(d => d.key === label)
        .style("visibility", "inherit")
        .attr("fill", d => d.key === label ? color(d.key) : "#000")
        .text(label);
      
      line.selectAll(".cursor-line")
        .attr("stroke", "fff0")
        .filter(d => d.key === label)
        .attr("stroke", d => d.key === label ? color(d.key) : "#fff0")
      
      path.attr("opacity", d => d.key === label ? 0.8 : 0.3);
      
      if (label === null) {
        path.attr("opacity", 0.8);
        line.style("visibility", "hidden");
      }
    }

    const mouseenter = () => {
      line.attr("display", null);
    }
  
    const mouseleave = () => {
      line.attr("display", "none");
    }

    svg
        .on("mousemove", mousemove)
        .on("mouseenter", mouseenter)
        .on("mouseleave", mouseleave)

  }

  svg.call(hover, path);
}

// ===== MEN =====
const svg1 = d3.select("#map13-gents").append("svg")
// .attr("style", "outline: thin solid red;")
  .attr("preserveAspectRatio", "xMinYMin meet")
  .attr("viewBox", "0 0 950 500")
  .classed("svg-content", true)
  .attr("transform",
   `translate(${margin.LEFT}, ${margin.TOP})`)

// Gentleman
svg1.append("image")
.attr("xlink:href", baseAssetsUrl+"man13.png")
    .attr("x", "-6%")
    .attr("y", "13%")
    .attr("width", 380)
    .attr("height", 380)
    .style("opacity", 1)

// Comment
const annot1 = [
  {
  note: { 
    label: "Remain the most popular libraries out of all, with trend increase from 2018 to present.",
    title: "Matplotlib/ Seaborn/ Plotly",
    wrap: 100, 
    padding: 5, 
  },
  className: "color",
  connector: { end: "arrow" },
  color: ["#ffffff"],
  x: 755,
  y: 300,
  dy: -35,
  dx: +30
}]

svg1.append("g")
  .style('font-size', 17)
  .call(d3.annotation()
          .annotations(annot1))

d3.csv(baseAssetsUrl + "viz_man.csv").then(function(data){

  const name = "Gentlemen"
  create_graph(data, svg1, name)

});

// ===== WOMEN =====
const svg2 = d3.select("#map13-ladies").append("svg")
// .attr("style", "outline: thin solid red;")
.attr("preserveAspectRatio", "xMinYMin meet")
.attr("viewBox", "0 0 950 500")
.classed("svg-content", true)
.attr("transform",
  `translate(${margin.LEFT}, ${margin.TOP})`)

// Lady
svg2.append("image")
.attr("xlink:href", baseAssetsUrl+"lady13.png")
.attr("x", "-22%")
.attr("y", "9%")
.attr("width", 460)
.attr("height", 460)
.style("opacity", 1)

// Comment
const annot2 = [
  {
  note: { 
    label: "All these libraries remain yet to be discovered (lack of interest or lack of awareness?).",
    title: "Shiny/ D3/ Bokeh/ Geo",
    wrap: 100, 
    padding: 5, 
  },
  className: "color",
  connector: { end: "arrow" },
  color: ["#ffffff"],
  x: 755,
  y: 130,
  dy: +35,
  dx: +30
}]

svg2.append("g")
  .style('font-size', 17)
  .call(d3.annotation()
          .annotations(annot2))

d3.csv(baseAssetsUrl + "viz_woman.csv").then(function(data){

  const name = "Ladies"
  create_graph(data, svg2, name)

});

});
'''


h = display(HTML(htmlt))
j = py_display.Javascript(js_t)
py_display.display_javascript(j)

## 3.2 Machine Learning on the "Blues"

<h3 style="font-family: times-new-roman; font-size: 1.5em">I. The Algorithms</h3>

* <p style="font-family: times-new-roman">If you shift through years, there is an <b>apparent decrease in specialized respondents</b> (that have more than 2 years of experience in doing ML). This links very well with the fact that we know that there are more youngsters on the platform, that are using their personal computers in order to navigate and learn Machine Learning. Nevertheless, <i>these youngsters will increase the percentage of experienced coders, usage of specialized hardware and overall "heavy-duty" tools and libraries in the years to come</i>. Just wait for it! 🧐</p>
* <p style="font-family: times-new-roman">🎩<b>Gents tend to have more numbers within the 3+ years of ML Experience</b> than 💃ladies do. This is linked to the fact that more men come from an Engineering background (which requires coding aptitudes) vs. women that come more from Data/Business Analytics (which requires less or even no coding).</p>
* <p style="font-family: times-new-roman">The most used algorithms for both genders are <b>Trees Ensembles and Random Forests, Gradient Boosting Algorithms and CNNs</b> (Convolutional Neural Networks). This matches quite well the overall scheme of competitions we have been having on Kaggle, which tends to be either tabular or requires Computer Vision (of course, there have been many more other types, forecasting and NLP to name a few - however most competitions do require in most of the cases at least one of the above-mentioned algorithms).</p>

In [ ]:
# ===== Years been doing ML =====
cols = ["Year", "What is your gender? - Selected Choice",
        df.columns[66]]
ml_yrs = df[cols]

ml_yrs.columns = ["Year", "Gender", "MachineLearning"]
ml_yrs = ml_yrs[ml_yrs["Gender"].isin(["Man", "Woman"])]

old_list = ['5-10 years', 'Under 1 year',
       'I do not use machine learning methods', '10-20 years',
       '2-3 years', '1-2 years', '4-5 years', '3-4 years',
       '20 or more years', '< 1 years', '10-15 years', '20+ years',
       'I have never studied machine learning but plan to learn in the future',
       '< 1 year',
       'I have never studied machine learning and I do not plan to']

new_list = ["7. 5-10 years", "2. <1 year",
            "1. Never", "8. 10+ years", 
            "4. 2-3 years", "3. 1-2 years", "6. 4-5 years", "5. 3-4 years",
            "8. 10+ years", "2. <1 year", "8. 10+ years", "8. 10+ years",
            "1. Never",
            "2. <1 year",
            "1. Never"]

ml_yrs["MachineLearning"] = ml_yrs["MachineLearning"].replace(old_list, new_list)
ml_yrs = ml_yrs.value_counts().reset_index()

ml_yrs = pd.merge(ml_yrs, total, on=["Year", "Gender"])
ml_yrs["perc"] = ml_yrs[0] / ml_yrs["Count"]
ml_yrs.drop(columns=[0, "Count"], inplace=True)

ml_yrs = pd.pivot(ml_yrs, index=["Year", "MachineLearning"],
               columns="Gender", values="perc").reset_index()
ml_yrs.columns = ["year", "ml", "male", "female"]

ml_yrs.to_csv("ml_yrs.csv", index=False)

# ===== ML algorithms =====
cols = list(df.columns[84:93])
new_names = ["TreesOrForests", "GradientBoostingMachines", "BayesianApproaches", 
             "EvolutionaryApproaches", "DenseNeuralNets", "CNNs",
             "GANs", "RNNs", "Transformers"]

algorithm = get_data_multiple_answers(cols, new_names)
algorithm = algorithm[(algorithm["Year"]!=2017) & (algorithm["Year"]!=2018)]

algorithm_man = algorithm[algorithm["Gender"] == "Man"].drop(columns=["Gender"])
algorithm_man = algorithm_man.sort_values(algorithm_man.last_valid_index(), axis=1,
                    ascending=False)
algorithm_woman = algorithm[algorithm["Gender"] == "Woman"].drop(columns=["Gender"])
algorithm_woman = algorithm_woman.sort_values(algorithm_woman.last_valid_index(), axis=1,
                    ascending=False)

algorithm_man.to_csv("algorithm_man.csv", index=False)
algorithm_woman.to_csv("algorithm_woman.csv", index=False)

# Save usa.png img as well
im = imageio.imread('../input/kaggle-data-science-survey-20172021/images/man14.png')
Image.fromarray(im).save("man14.png")

im = imageio.imread('../input/kaggle-data-science-survey-20172021/images/lady14.png')
Image.fromarray(im).save("lady14.png")

htmlt = '''
<!-- Bootstrap -->
<link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.4.1/css/bootstrap.min.css" integrity="sha384-Vkoo8x4CGsO3+Hhxv8T/Q5PaXtkKtu6ug5TOeNV6gBiFeWPGFN9MuhOf23Q9Ifjh" crossorigin="anonymous">
<!-- Style -->
<style>
	.all14 {
		font-family: "Times New Roman", Times, serif;
		overflow-x: hidden;
		width: 98%;
		background-color:#e0c9a6;
		/* background-image : url("bkg.jpg"); changed via js */
	}
	.svg-container {
		display: inline-block;
		position: relative;
		width: 100%;
		padding-bottom: 100%;
		vertical-align: top;
		overflow: hidden;
	}
	.svg-content {
		display: inline-block;
		position: absolute;
		top: 0;
		left: 0;
	}

	.axisHidden path{
		stroke: #e0c9a6;
	}

	.axisHidden line{
		stroke: #e0c9a6;
	}

	.annotation.color text {
		fill: #000000;
	}

	@font-face {
		font-family: parklane;
		src: url(https://h4ks.net/ParkLaneNF.otf);
		}

	div.tooltip-donut {
		position: absolute;
		text-align: center;
		padding: .3rem;
		background: #FFFFFF;
		color: #000000;
		border: 1px solid #000000;
		border-radius: 8px;
		pointer-events: none;
		font-size: 2rem;
		z-index: 99999;
	}

</style>

<img id="baseimg" src="man.png" style="display:none" />
<div class="all14">
	<center><h1 style="font-family: parklane">Machine Learning: Hystoric</h1></center>
	<label for="filter14" style="font-size: 25px">Select Year Here:</label>
	<select id="filter14" style="font-size: 20px">
		<option value="2021">2021</option>
		<option value="2020">2020</option>
		<option value="2019">2019</option>
		<option value="2018">2018</option>
	  </select>
	<div class="row">
		<div id="map14" class="svg-container"></div>
	</div>
</div>
'''

js_t = '''
require.config({
  paths: {
    d3src: "https://d3js.org/",
  },
  map: {
    '*': {
      'd3v6': 'd3src/d3.v6.min',
      'd3-selection': 'd3src/d3-selection.v1.min',
      'd3-drag': 'd3src/d3-drag.v1.min',
      'd3-shape': 'd3src/d3-shape.v1.min',
      'd3-path': 'd3src/d3-path.v1.min',
      'd3-dispatch': 'd3src/d3-dispatch.v1.min',
      'd3-annotation': 'https://cdnjs.cloudflare.com/ajax/libs/d3-annotation/2.5.1/d3-annotation.min.js',
    }
  }
});

require(["d3v6", "d3-annotation"], function(d3, d3Annotation) {

// Inject d3Annotation methods to d3
for (var key in d3Annotation) {
  d3[key] = d3Annotation[key];
}

// Get Assets URL set by Kaggle
const baseAssetsUrl = document.getElementById('baseimg').src.replace(/man.png.*$/, '');

const $all14 = document.getElementsByClassName('all14')[0];
$all14.style.backgroundImage = `url("${baseAssetsUrl}bkg.jpg")`;

// Set the SVG area
const margin = { LEFT: 20, RIGHT: 40, TOP: 15, BOTTOM: 0 }
const width = 950 - margin.LEFT - margin.RIGHT
const height = 900 - margin.TOP - margin.BOTTOM
const centreSpacing = 100

const svg = d3.select("#map14").append("svg")
  .attr("preserveAspectRatio", "xMinYMin meet")
  .attr("viewBox", "0 0 950 900")
  .classed("svg-content", true)
  .attr("transform",
   `translate(${margin.LEFT}, ${margin.TOP})`)

// Gentleman
svg.append("image")
.attr("xlink:href", baseAssetsUrl+"man14.png")
    .attr("x", "-12%")
    .attr("y", "-3%")
    .attr("width", 440)
    .attr("height", 440)
    .style("opacity", 1)

// Lady
svg.append("image")
.attr("xlink:href", baseAssetsUrl+"lady14.png")
.attr("x", "74%")
.attr("y", "1%")
.attr("width", 440)
.attr("height", 440)
.style("opacity", 1)

// Gradient
const defs = svg.append('defs');

const bgGradient1 = defs
  .append('linearGradient')
  .attr('id', 'bg-gradient14')

bgGradient1
  .append('stop')
  .attr('stop-color', '#927366')
  .attr('offset', '0%');
bgGradient1
  .append('stop')
  .attr('stop-color', '#633725')
  .attr('offset', '100%');

const bgGradient2 = defs
  .append('linearGradient')
  .attr('id', 'bg-gradient15')

bgGradient2
  .append('stop')
  .attr('stop-color', '#728551')
  .attr('offset', '0%');
bgGradient2
  .append('stop')
  .attr('stop-color', '#aab697')
  .attr('offset', '100%');

// ------ DEFAULTS ------
// Initialize scales
const y = d3.scaleBand()
.range([height/2.7, 0])
.padding(0.1)

const x = d3.scaleLinear()
.range([0, (width - centreSpacing) / 2-150])
const xAxis = svg.append("g")
  .attr('transform', 'translate(165,' + (height-530) + ')')

const xReverse = d3.scaleLinear()
.range([0, (width - centreSpacing) / 2-150])
const xAxisReverse = svg.append("g")
  .attr('transform', 'translate(515,' + (height-530) + ')')


// Male + Female + Labels objects
var gM = svg.append("g")
  .attr("transform", 
  "translate(" + 20 + "," + margin.TOP + ")");

var gF = svg.append("g")
  .attr('transform',
    'translate(' +
      (margin.LEFT + (width - centreSpacing) / 2 + centreSpacing) +
      "," +
      margin.TOP +
      ")");

var gLabels = svg
.append('g')
.attr(
  'transform',
  'translate(' +
    (margin.LEFT + (width - centreSpacing) / 2 + 
    ',' + margin.TOP + ')'));


// === UPDATE ===
function update(data) {

  data.forEach(d => {
    d.female = Number(d.female)
    d.male = Number(d.male)
  })

  // Scales
  y.domain(data.map(d => d.ml))

  const maxVal = d3.max(data, d => d3.max([d.male, d.female]))

  x.domain([0, maxVal])
  xReverse.domain([0, maxVal])

  

  // Male
  gM.selectAll('rect')
    .data(data)
    .join('rect')
    .transition()
    .duration(1000)
    .attr('x', d => (width - centreSpacing) / 2 - x(d.male))
    .attr('y', d => y(d.ml))
    .attr('rx', 10)
    .attr('ry', 10)
    .attr('height', y.bandwidth())
    .attr('width', d => x(d.male))
    .style('fill', 'url(#bg-gradient14)');


  // Female
  gF.selectAll('rect')
    .data(data)
    .join('rect')
    .transition()
    .duration(1000)
    .attr('x', 0)
    .attr('y', d => y(d.ml))
    .attr('rx', 10)
    .attr('ry', 10)
    .attr('height', y.bandwidth())
    .attr('width', d => x(d.female))
    .style('fill', 'url(#bg-gradient15)');
  

  //Labels
  gLabels.selectAll('text')
    .data(data)
    .join('text')
    .attr('x', centreSpacing / 2-45)
    .attr('y', d => y(d.ml) + y.bandwidth() / 2+5)
    .style('font-weight', 600)
    .text((d, i) => d.ml);

  gLabels
    .append('text')
    .text('Years doing Machine Learning')
    .attr('x', centreSpacing / 2-135)
    .attr('y', -1)
    .style('font-size', '20px')
    .style('font-weight', 600);


  // Axis Update
  xAxis
    .transition()
    .duration(1000)
    .call(d3.axisBottom(x).ticks(3).tickFormat(d3.format(".0%")))
    .selectAll("text")
      .attr("font-size", "15px");

  xAxisReverse
    .transition()
    .duration(1000)
    .call(d3.axisBottom(xReverse).ticks(3).tickFormat(d3.format(".0%")))
    .selectAll("text")
      .attr("font-size", "15px");
}


// === Years Programming ===
d3.csv(baseAssetsUrl + "ml_yrs.csv").then(function(dataload){

  // default view
  var data = dataload.filter(function(d) { 
    var sq = d3.select("#filter14").property("value");
    return d.year === sq;
  });

  update(data)


  // on change
  d3.select("#filter14").on("change", function() {

    var data = dataload.filter(function(d) { 
      var sq = d3.select("#filter14").property("value");
      return d.year === sq;
    });

    update(data);
  });

});


function create_stacked_bar(data, subgroups, x_global, y_range, color) {

  const groups = data.map(d => d.Year)

  // Add X axis
  const x = d3.scaleBand()
      .domain(groups)
      .range([0, width/2.5])
      .padding([0.2])
  svg.append("g")
    .attr("transform", `translate(${x_global}, ${height/1.01})`)
    .call(d3.axisBottom(x).tickSizeOuter(0))
    .selectAll("text")
      .attr("y", "10")
      .attr("x", "0")
      .attr("font-size", "15px")
      .attr("font-weight", 300)
      .attr("text-anchor", "middle")

  // Add Y axis
  const y = d3.scaleLinear()
    .domain([0, y_range])
    .range([ height/2.3, 0]);
  svg.append("g")
  .attr("class", "axisHidden")
  .attr("transform", `translate(${x_global}, ${height/1.85})`)
    .call(d3.axisLeft(y).ticks(0))
    .selectAll("text")
      .attr("font-size", "11px")
      .attr("font-weight", 300);

  //s Stack data
  const stackedData = d3.stack()
    .keys(subgroups)
    (data)

  // Show the bars
  const bars = svg.append("g")
    .selectAll("g")
    .data(stackedData)
    .join("g")
    .attr("transform", `translate(${x_global}, ${height/1.85})`)
      .attr("fill", d => color(d.key))
      .attr("stroke", "black")
      .attr("stroke-width", 0.2)
      .attr("class", d => "myRect " + d.key ) 
      .selectAll("rect")
      .data(d => d)
      .join("rect")
        .attr("x", d => x(d.data.Year))
        .attr("y", d => y(d[1]))
        .attr("height", d => y(d[0]) - y(d[1]))
        .attr("width",x.bandwidth())

  // Interactive
  var div = d3.select("body").append("div")
  .attr("class", "tooltip-donut")
  .style("opacity", 0);

  bars
    .on('mouseenter', function (event, dt) {
    // MOUSE ON
    const subGroupName = d3.select(this.parentNode).datum().key 
          
    d3.selectAll(".myRect").style("opacity", 0.2)  
          
    d3.selectAll("."+subGroupName).style("opacity",1)


    // Makes the new div appear
    div.transition()
        .duration(50)
        .style("opacity", 1);

    const subgroupName = d3.select(this.parentNode).datum().key;
    const subgroupValue = dt.data[subgroupName];

    div.html("Alg: " + subgroupName + "<br>" + "Perc: " + subgroupValue +'%')
        .style("left", (event.pageX) + "px")
        .style("top", (event.pageY-30) + "px");


  })
    // MOUSE LEAVE
    .on('mouseleave', function () {
      d3.selectAll(".myRect")
          .style("opacity",1) 

      // New div dissapears
      div.transition()
          .duration('50')
          .style("opacity", 0);
      svg.selectAll('.divergence').remove()
    })

}


// === Preferences ===
d3.csv(baseAssetsUrl + "algorithm_man.csv").then(function(data){

  const x_global = 50
  const y_range = 230
  const subgroups = data.columns.slice(1)
  const color = d3.scaleOrdinal()
    .domain(subgroups)
    .range(["#633725", "#734b3b", "#825f51",
            "#927366", "#a1877c", "#b19b92",
            "#c1afa8", "#d0c3be", "#e0d7d3"]);

  svg.append("text")
  .attr("y", "55%")
  .attr("x", "25%")
  .attr("font-size", "22px")
  .attr("text-anchor", "middle")
  .attr("font-weight", 600)
  .text("Gentlemen: ML Algorithms Preferences")

  create_stacked_bar(data, subgroups, x_global, y_range, color)
  
});

d3.csv(baseAssetsUrl + "algorithm_woman.csv").then(function(data){

  const x_global = 500
  const y_range = 230
  const subgroups = data.columns.slice(1)
  const color = d3.scaleOrdinal()
    .domain(subgroups)
    .range(["#728551", "#809162", "#8e9d74",
            "#9caa85", "#aab697", "#b9c2a8",
            "#c7ceb9", "#d5dacb", "#e3e7dc"]);

  svg.append("text")
  .attr("y", "55%")
  .attr("x", "70%")
  .attr("font-size", "22px")
  .attr("text-anchor", "middle")
  .attr("font-weight", 600)
  .text("Ladies: ML Algorithms Preferences")

  create_stacked_bar(data, subgroups, x_global, y_range, color)
  
});

});
'''


h = display(HTML(htmlt))
j = py_display.Javascript(js_t)
py_display.display_javascript(j)

<h3 style="font-family: times-new-roman; font-size: 1.5em">II. The Frameworks</h3>

* <p style="font-family: times-new-roman"><b>Scikit-learn, Tensorflow and Keras</b> are the most used ML frameworks for both genders. However, 💃<b>ladies do tend to have less interest</b> within them than 🎩men do, as it shows within the smaller numbers of usage for each framework.</p>
* <p style="font-family: times-new-roman">Most frameworks have a steady trend of usage, with no apparent substantial increase or decrease in popularity. I believe that <b>the most "apparent" improvement in the last 4 years was for PyTorch</b>, which increased 8% popularity points for 🎩gents and 6% popularity points for 💃ladies in 2021 vs 2018.</p>

<p style="font-family: times-new-roman">Note: As the number of respondents that answered they used Other Frameworks or None at all was very small (less than 3% for both genders), I excluded them from the graphs.</p>

In [ ]:
cols = list(df.columns[67:79])
new_names = ["Scikit-learn", "TensorFlow", "Keras", "PyTorch", "Fast.ai",
             "MXNet", "XGBoost", "LightGBM", "CatBoost", "Prophet", "H2O",
             "Caret"]

ml = get_data_multiple_answers(cols, new_names)
ml = ml[ml["Year"]!=2017]

ml_man = ml[ml["Gender"] == "Man"].drop(columns=["Gender"])
ml_woman = ml[ml["Gender"] == "Woman"].drop(columns=["Gender"])

ml_man.to_csv("ml_man.csv", index=False)
ml_woman.to_csv("ml_woman.csv", index=False)

# Save usa.png img as well
im = imageio.imread('../input/kaggle-data-science-survey-20172021/images/man15.png')
Image.fromarray(im).save("man15.png")

im = imageio.imread('../input/kaggle-data-science-survey-20172021/images/lady15.png')
Image.fromarray(im).save("lady15.png")

htmlt = '''
<!-- Bootstrap -->
<link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.4.1/css/bootstrap.min.css" integrity="sha384-Vkoo8x4CGsO3+Hhxv8T/Q5PaXtkKtu6ug5TOeNV6gBiFeWPGFN9MuhOf23Q9Ifjh" crossorigin="anonymous">
<!-- Style -->
<style>
	.all15 {
		font-family: "Times New Roman", Times, serif;
		overflow-x: hidden;
		width: 98%;
		background-color:#e0c9a6;
		/* background-image : url("bkg.jpg"); changed via js */
	}
	.all15 .svg-container {
		display: inline-block;
		position: relative;
		width: 100%;
		padding-bottom: 54%;
		vertical-align: top;
		overflow: hidden;
	}
	.svg-content {
		display: inline-block;
		position: absolute;
		top: 0;
		left: 0;
	}

	.axisHidden path{
		stroke: #e0c9a6;
	}

	.axisHidden line{
		stroke: #e0c9a6;
	}

	.annotation.color text {
		fill: #000000;
	}

	@font-face {
		font-family: parklane;
		src: url(https://h4ks.net/ParkLaneNF.otf);
		}

	div.tooltip-donut {
		position: absolute;
		text-align: center;
		padding: .3rem;
		background: #FFFFFF;
		color: #000000;
		border: 1px solid #000000;
		border-radius: 8px;
		pointer-events: none;
		font-size: 2rem;
		z-index: 99999;
	}

</style>

<img id="baseimg" src="man.png" style="display:none" />
<div class="all15">
	<center><h1 style="font-family: parklane">Machine Learning Blues</h1></center>
	<center><h3><i>- ML Frameworks -</i></h3></center>
	<div class="row">
		<div id="map15-gents" class="svg-container"></div>
	</div>

	<div class="row">
		<div id="map15-ladies" class="svg-container"></div>
	</div>
</div>
'''

js_t = '''
require.config({
  paths: {
    d3src: "https://d3js.org/",
  },
  map: {
    '*': {
      'd3v6': 'd3src/d3.v6.min',
      'd3-selection': 'd3src/d3-selection.v1.min',
      'd3-drag': 'd3src/d3-drag.v1.min',
      'd3-shape': 'd3src/d3-shape.v1.min',
      'd3-path': 'd3src/d3-path.v1.min',
      'd3-dispatch': 'd3src/d3-dispatch.v1.min',
      'd3-annotation': 'https://cdnjs.cloudflare.com/ajax/libs/d3-annotation/2.5.1/d3-annotation.min.js',
    }
  }
});

require(["d3v6", "d3-annotation"], function(d3, d3Annotation) {

// Inject d3Annotation methods to d3
for (var key in d3Annotation) {
  d3[key] = d3Annotation[key];
}

// Get Assets URL set by Kaggle
const baseAssetsUrl = document.getElementById('baseimg').src.replace(/man.png.*$/, '');

const $all15 = document.getElementsByClassName('all15')[0];
$all15.style.backgroundImage = `url("${baseAssetsUrl}bkg.jpg")`;

// Set the SVG area
const margin = { LEFT: 20, RIGHT: 40, TOP: 15, BOTTOM: 0 }
const width = 950 - margin.LEFT - margin.RIGHT
const height = 500 - margin.TOP - margin.BOTTOM
const chart_size = 120
const x_global = 150

function create_graph(data, svg, name){

  const XY_axis = height/2-200

  svg.append("text")
  .attr("transform", `translate(${x_global-60}, ${XY_axis-10})`)
  .attr("font-size", "24px")
  .attr("text-anchor", "middle")
  .attr("font-weight", 600)
  .text(name)

  data.forEach(d => {
    d.Year = Number(d.Year)
  })

  // List of groups
  var keys = data.columns.slice(1)

  // Stacking
  var series = d3.stack()
    .keys(keys)
    .offset(d3.stackOffsetSilhouette)
    (data)

  // X Axis
  var x = d3.scaleLinear()
    .domain(d3.extent(data, d => d.Year))
    .range([0, width/1.5])

  var xAxis = g => g
    .attr("transform", `translate(${x_global}, ${XY_axis})`)
    .call(d3.axisBottom(x)
            .tickSize(height/1.2).ticks(4).tickFormat(d3.format("d")))
    .call(g => g.select(".domain").remove())
    .call(g => g.selectAll(".tick line")
                .attr("stroke", "#3A3A3A")
                .attr('stroke-dasharray', '5 5'))
    .call(g => g.selectAll(".tick")
                .select('text')
                .attr('fill', "#3A3A3A")
                .style("font-size", 15))

  // Y Axis
  var y = d3.scaleLinear()
  .domain([-chart_size, chart_size])
  .range([height/1.2, 0])
  svg.append("g")
  .attr("class", "axisHidden")
  .attr("transform", `translate(${x_global}, ${XY_axis})`)
    .call(d3.axisLeft(y).ticks(0))

  // Area Chart
  var area = d3.area()
  .x(d => x(d.data.Year))
  .y0(d => y(d[0]))
  .y1(d => y(d[1]))

  // Color
  const color = d3.scaleOrdinal()
    .domain(keys)
     .range(["#D67A83", "#D05257", "#D87F58","#CE8E27",
     "#8FB79C", "#83AF92", "#588968", "#0B656F", "#74A5D2",
      "#226BBF", "#184E8B", "#BA3B8B"])

  // Show
  const path = svg.append("g")
    .selectAll("path")
    .data(series)
    .join("path")
    .attr("transform", `translate(${x_global}, ${XY_axis})`)
      .attr("data-label", d => d.key)
      .attr("fill", ({key}) => color(key))
      .attr("stroke", "black")
      .attr("stroke-width", 0.2)
      .attr("d", area)
      .attr("opacity", 0.8)

  svg.append("g")
      .call(xAxis)

  // ==== Interactive ====
  const hover = (svg, path) => {
  
    const line = svg.append("g")
         .attr("display", "none");
  
    line.append("g")
        .selectAll("line")
        .data(series)
        .join("line")
        .attr("class", "cursor-line")
        .attr("fill", "#fff")
        .attr("stroke-width", 0.8)
        .attr("x1", 10)
        .attr("y1", height)
        .attr("x2", 10)
        .attr("y2", 10);

    line.append("text")
      .attr("class", "text-year")
      .attr("font-size", 14)
      .attr("x", 0)
      .attr("y", 26)
      .attr("transform", "rotate(-90 20 20) translate(0, -20)");
    
    line.append("g")
        .selectAll("text")
        .data(series)
        .join("text")
        .attr("class", "text-label")
        .attr("font-weight", 900)
        .attr("font-size", 17)
        .attr("y", height-10)
        .attr("x", -8)
        .attr("transform", 
        "rotate(-90 10 460)");

    line.append("g")
        .selectAll("text")
        .data(data)
        .join("text")
        .attr("class", "text-value")
        .attr("font-weight", 700)
        .attr("font-size", 14)
        .attr("y", height-(height*0.94))
        .attr("x", 14);

    const mousemove = (event) => {
  
      event?.preventDefault();
      const pointer = d3.pointer(event);

      const xm = x.invert(pointer[0]);
      const label = d3.select(event.target).attr("data-label");
  
      line.attr("transform", `translate(${pointer[0]}, 0)`)
          .style("visibility", "inherit");
    
          line.select(".text-year")
          .text(parseInt(xm));
      
      line.selectAll(".text-value")
        .text(d => d.Year === parseInt(xm) ? d[label]+"%" : "");

      line.selectAll(".text-label")
        .style("visibility", "hidden")
        .filter(d => d.key === label)
        .style("visibility", "inherit")
        .attr("fill", d => d.key === label ? color(d.key) : "#000")
        .text(label);
      
      line.selectAll(".cursor-line")
        .attr("stroke", "fff0")
        .filter(d => d.key === label)
        .attr("stroke", d => d.key === label ? color(d.key) : "#fff0")
      
      path.attr("opacity", d => d.key === label ? 0.8 : 0.3);
      
      if (label === null) {
        path.attr("opacity", 0.8);
        line.style("visibility", "hidden");
      }
    }

    const mouseenter = () => {
      line.attr("display", null);
    }
  
    const mouseleave = () => {
      line.attr("display", "none");
    }

    svg
        .on("mousemove", mousemove)
        .on("mouseenter", mouseenter)
        .on("mouseleave", mouseleave)

  }

  svg.call(hover, path);
}

// ===== MEN =====
const svg1 = d3.select("#map15-gents").append("svg")
// .attr("style", "outline: thin solid red;")
  .attr("preserveAspectRatio", "xMinYMin meet")
  .attr("viewBox", "0 0 950 500")
  .classed("svg-content", true)
  .attr("transform",
   `translate(${margin.LEFT}, ${margin.TOP})`)

// Gentleman
svg1.append("image")
.attr("xlink:href", baseAssetsUrl+"man15.png")
    .attr("x", "-20%")
    .attr("y", "13%")
    .attr("width", 390)
    .attr("height", 390)
    .style("opacity", 1)

// Comment
const annot1 = [
  {
  note: { 
    label: "There is no substantial increase nor decrease in any of the frameworks for both men & women.",
    title: "Steady Trend",
    wrap: 100, 
    padding: 5, 
  },
  className: "color",
  // connector: { end: "arrow" },
  color: ["#ffffff"],
  x: 780,
  y: 350,
  dy: -1,
  dx: +1
}]

svg1.append("g")
  .style('font-size', 17)
  .call(d3.annotation()
          .annotations(annot1))

d3.csv(baseAssetsUrl + "ml_man.csv").then(function(data){

  const name = "Gentlemen"
  create_graph(data, svg1, name)

});

// ===== WOMEN =====
const svg2 = d3.select("#map15-ladies").append("svg")
// .attr("style", "outline: thin solid red;")
.attr("preserveAspectRatio", "xMinYMin meet")
.attr("viewBox", "0 0 950 500")
.classed("svg-content", true)
.attr("transform",
  `translate(${margin.LEFT}, ${margin.TOP})`)

// Lady
svg2.append("image")
.attr("xlink:href", baseAssetsUrl+"lady15.png")
.attr("x", "-17%")
.attr("y", "12%")
.attr("width", 450)
.attr("height", 450)
.style("opacity", 1)

// Comment
const annot2 = [
  {
  note: { 
    label: "Overall, ladies have smaller interest for ml frameworks than the gents do.",
    title: "Less Usage",
    wrap: 100, 
    padding: 5, 
  },
  className: "color",
  // connector: { end: "arrow" },
  color: ["#ffffff"],
  x: 780,
  y: 350,
  dy: -1,
  dx: +1
}]

svg2.append("g")
  .style('font-size', 17)
  .call(d3.annotation()
          .annotations(annot2))

d3.csv(baseAssetsUrl + "ml_woman.csv").then(function(data){

  const name = "Ladies"
  create_graph(data, svg2, name)

});

});
'''


h = display(HTML(htmlt))
j = py_display.Javascript(js_t)
py_display.display_javascript(j)

## 3.3 Deep Learning "Swing"

* <p style="font-family: times-new-roman">In regards to Computer Vision, <b>Image Classification (e.g.: ResNet, EffNet etc.) is the most used method for both genders</b>. However, the second most used method differs for the 2: for 🎩gents is Object Detection (e.g.: RetinaNet), while for the 💃Ladies is Image Segmentation (e.g.: U-Net), although the 2 are very close in terms of popularity from one to another.</p>
* <p style="font-family: times-new-roman">In terms of NLP (Natural Language Processing), we can see a <b>very strong increase in popularity for Transformer Language Models (e.g.: BERT)</b> for both genders. It even surpassed in popularity of the EncoderDecorderModels (e.g.: seq2seq), which were much more popular in 2019.</p>

In [ ]:
# ===== Computer Vision =====
cols = list(df.columns[96:100])
new_names = ["Image segmentation (U-Net)", "Object detection (RetinaNet)", "Image classification (ResNet)",
             "Generative Networks (GAN)"]

cv = get_data_multiple_answers(cols, new_names)
cv = cv[(cv["Year"]!=2017) & (cv["Year"]!=2018)]
cv.columns = ['Year', 'Gender', 'ImageSegmentation', 'ObjectDetection',
              'ImageClassification', 'GenerativeNetworks']

cv_man = cv[cv["Gender"] == "Man"].drop(columns=["Gender"])
cv_man = cv_man.sort_values(cv_man.last_valid_index(), axis=1,
                    ascending=False)
cv_woman = cv[cv["Gender"] == "Woman"].drop(columns=["Gender"])
cv_woman = cv_woman.sort_values(cv_woman.last_valid_index(), axis=1,
                    ascending=False)

cv_man.to_csv("cv_man.csv", index=False)
cv_woman.to_csv("cv_woman.csv", index=False)

# ===== NLP =====
cols = list(df.columns[103:106])
new_names = ["Encoder-Decorder models (seq2seq)", "Contextualized embeddings (ELMo)",
             "Transformer Language Models (BERT)"]

nlp = get_data_multiple_answers(cols, new_names)
nlp = nlp[(nlp["Year"]!=2017) & (nlp["Year"]!=2018)]
nlp.columns = ['Year', 'Gender', 'EncoderDecorderModels',
               'ContextualizedEmbeddings', 'TransformerLanguageModels']

nlp_man = nlp[nlp["Gender"] == "Man"].drop(columns=["Gender"])
nlp_man = nlp_man.sort_values(nlp_man.last_valid_index(), axis=1,
                    ascending=False)
nlp_woman = nlp[nlp["Gender"] == "Woman"].drop(columns=["Gender"])
nlp_woman = nlp_woman.sort_values(nlp_woman.last_valid_index(), axis=1,
                    ascending=False)

nlp_man.to_csv("nlp_man.csv", index=False)
nlp_woman.to_csv("nlp_woman.csv", index=False)

# Save usa.png img as well
im = imageio.imread('../input/kaggle-data-science-survey-20172021/images/man16.png')
Image.fromarray(im).save("man16.png")

im = imageio.imread('../input/kaggle-data-science-survey-20172021/images/lady16.png')
Image.fromarray(im).save("lady16.png")

htmlt = '''
<!-- Bootstrap -->
<link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.4.1/css/bootstrap.min.css" integrity="sha384-Vkoo8x4CGsO3+Hhxv8T/Q5PaXtkKtu6ug5TOeNV6gBiFeWPGFN9MuhOf23Q9Ifjh" crossorigin="anonymous">
<!-- Style -->
<style>
	.all16 {
		font-family: "Times New Roman", Times, serif;
		overflow-x: hidden;
		width: 98%;
		background-color:#e0c9a6;
		/* background-image : url("bkg.jpg"); changed via js */
	}
	.all16 .svg-container {
		display: inline-block;
		position: relative;
		width: 100%;
		padding-bottom: 54%;
		vertical-align: top;
		overflow: hidden;
	}
	.svg-content {
		display: inline-block;
		position: absolute;
		top: 0;
		left: 0;
	}

	.axisHidden path{
		stroke: #e0c9a6;
	}

	.axisHidden line{
		stroke: #e0c9a6;
	}

	.annotation.color text {
		fill: #000000;
	}

	@font-face {
		font-family: parklane;
		src: url(https://h4ks.net/ParkLaneNF.otf);
		}

	div.tooltip-donut {
		position: absolute;
		text-align: center;
		padding: .3rem;
		background: #FFFFFF;
		color: #000000;
		border: 1px solid #000000;
		border-radius: 8px;
		pointer-events: none;
		font-size: 2rem;
		z-index: 99999;
	}

</style>

<img id="baseimg" src="man.png" style="display:none" />
<div class="all16">
	<center><h1 style="font-family: parklane">DL: The Visual & The Talk</h1></center>
	<center><h3><i>- Computer Vision & NLP Frameworks -</i></h3></center>
	<div class="row">
		<div id="map16-cv" class="svg-container"></div>
	</div>

	<div class="row">
		<div id="map16-nlp" class="svg-container"></div>
	</div>
</div>
'''

js_t = '''
require.config({
  paths: {
    d3src: "https://d3js.org/",
  },
  map: {
    '*': {
      'd3v6': 'd3src/d3.v6.min',
      'd3-selection': 'd3src/d3-selection.v1.min',
      'd3-drag': 'd3src/d3-drag.v1.min',
      'd3-shape': 'd3src/d3-shape.v1.min',
      'd3-path': 'd3src/d3-path.v1.min',
      'd3-dispatch': 'd3src/d3-dispatch.v1.min',
      'd3-annotation': 'https://cdnjs.cloudflare.com/ajax/libs/d3-annotation/2.5.1/d3-annotation.min.js',
    }
  }
});

require(["d3v6", "d3-annotation"], function(d3, d3Annotation) {

// Inject d3Annotation methods to d3
for (var key in d3Annotation) {
  d3[key] = d3Annotation[key];
}

// Get Assets URL set by Kaggle
const baseAssetsUrl = document.getElementById('baseimg').src.replace(/man.png.*$/, '');

const $all16 = document.getElementsByClassName('all16')[0];
$all16.style.backgroundImage = `url("${baseAssetsUrl}bkg.jpg")`;

// Set the SVG area
const margin = { LEFT: 20, RIGHT: 40, TOP: 15, BOTTOM: 0 }
const width = 950 - margin.LEFT - margin.RIGHT
const height = 500 - margin.TOP - margin.BOTTOM


// STACKED CHART
function create_stacked_bar(svg, data, subgroups, x_global,
   y_range, color, flag) {

  const groups = data.map(d => d.Year)

  // Add X axis
  const x = d3.scaleBand()
      .domain(groups)
      .range([0, width/2.3])
      .padding([0.2])
  svg.append("g")
    .attr("transform", `translate(${x_global}, ${height/1.05})`)
    .call(d3.axisBottom(x).tickSizeOuter(0))
    .selectAll("text")
      .attr("y", "10")
      .attr("x", "0")
      .attr("font-size", "15px")
      .attr("font-weight", 300)
      .attr("text-anchor", "middle")

  // Add Y axis
  const y = d3.scaleLinear()
    .domain([0, y_range])
    .range([ height/1.05, 0]);
  svg.append("g")
  .attr("class", "axisHidden")
  .attr("transform", `translate(${x_global}, ${height-500})`)
    .call(d3.axisLeft(y).ticks(0))
    .selectAll("text")
      .attr("font-size", "11px")
      .attr("font-weight", 300);

  //s Stack data
  const stackedData = d3.stack()
    .keys(subgroups)
    (data)

  // Show the bars
  const bars = svg.append("g")
    .selectAll("g")
    .data(stackedData)
    .join("g")
    .attr("transform", `translate(${x_global}, ${height-500})`)
      .attr("fill", d => color(d.key))
      .attr("stroke", "black")
      .attr("stroke-width", 0.2)
      .attr("class", d => "myRect " + d.key ) 
      .selectAll("rect")
      .data(d => d)
      .join("rect")
        .attr("x", d => x(d.data.Year))
        .attr("y", d => y(d[1]))
        .attr("height", d => y(d[0]) - y(d[1]))
        .attr("width",x.bandwidth())

  // Interactive
  var div = d3.select("body").append("div")
  .attr("class", "tooltip-donut")
  .style("opacity", 0);

  bars
    .on('mouseenter', function (event, dt) {
    // MOUSE ON
    const subGroupName = d3.select(this.parentNode).datum().key 
          
    d3.selectAll(".myRect").style("opacity", 0.2)  
          
    d3.selectAll("."+subGroupName).style("opacity",1)


    // Makes the new div appear
    div.transition()
        .duration(50)
        .style("opacity", 1);

    const subgroupName = d3.select(this.parentNode).datum().key;
    const subgroupValue = dt.data[subgroupName];

    div.html(flag + ": " + subgroupName + "<br>" + "Perc: " + subgroupValue +'%')
        .style("left", (event.pageX) + "px")
        .style("top", (event.pageY-30) + "px");


  })
    // MOUSE LEAVE
    .on('mouseleave', function () {
      d3.selectAll(".myRect")
          .style("opacity",1) 

      // New div dissapears
      div.transition()
          .duration('50')
          .style("opacity", 0);
      svg.selectAll('.divergence').remove()
    })

}

// ===== Computer Vision =====
const svg1 = d3.select("#map16-cv").append("svg")
// .attr("style", "outline: thin solid red;")
  .attr("preserveAspectRatio", "xMinYMin meet")
  .attr("viewBox", "0 0 950 500")
  .classed("svg-content", true)
  .attr("transform",
   `translate(${margin.LEFT}, ${margin.TOP})`)

svg1.append("text")
.attr("y", "8%")
.attr("x", "-27%")
.attr("font-size", "30px")
.attr("text-anchor", "middle")
.attr("transform", "rotate(-90)")
.attr("font-family", "parklane")
.text("Computer Vision")

// Ladies
svg1.append("image")
.attr("xlink:href", baseAssetsUrl+"lady16.png")
    .attr("x", "68%")
    .attr("y", "13%")
    .attr("width", 210)
    .attr("height", 210)
    .style("opacity", 1)


d3.csv(baseAssetsUrl + "cv_man.csv").then(function(data){

  const x_global = 80
  const y_range = 80
  const subgroups = data.columns.slice(1)
  const color = d3.scaleOrdinal()
    .domain(subgroups)
    .range(["#284a5d", "#5f7384", "#90a6b3",
            "#595052"]);
  const flag = "CV"

  svg1.append("text")
  .attr("y", "10%")
  .attr("x", "28%")
  .attr("font-size", "22px")
  .attr("text-anchor", "middle")
  .attr("font-weight", 600)
  .text("Gentlemen")

  create_stacked_bar(svg1, data, subgroups,
     x_global, y_range, color, flag)

});

d3.csv(baseAssetsUrl + "cv_woman.csv").then(function(data){

  const x_global = 500
  const y_range = 80
  const subgroups = data.columns.slice(1)
  const color = d3.scaleOrdinal()
    .domain(subgroups)
    .range(["#8a3223", "#aa5549", "#bf897a",
            "#c26e4c"]);
  const flag = "CV"

  svg1.append("text")
  .attr("y", "10%")
  .attr("x", "73%")
  .attr("font-size", "22px")
  .attr("text-anchor", "middle")
  .attr("font-weight", 600)
  .text("Ladies")

  create_stacked_bar(svg1, data, subgroups,
     x_global, y_range, color, flag)

});



// ===== NLP =====
const svg2 = d3.select("#map16-nlp").append("svg")
// .attr("style", "outline: thin solid red;")
.attr("preserveAspectRatio", "xMinYMin meet")
.attr("viewBox", "0 0 950 500")
.classed("svg-content", true)
.attr("transform",
  `translate(${margin.LEFT}, ${margin.TOP})`)

// Gents
svg2.append("image")
.attr("xlink:href", baseAssetsUrl+"man16.png")
.attr("x", "6%")
.attr("y", "12%")
.attr("width", 220)
.attr("height", 220)
.style("opacity", 1)

svg2.append("text")
.attr("y", "8%")
.attr("x", "-27%")
.attr("font-size", "30px")
.attr("text-anchor", "middle")
.attr("transform", "rotate(-90)")
.attr("font-family", "parklane")
.text("Natural Lang. Processing")

d3.csv(baseAssetsUrl + "nlp_man.csv").then(function(data){

  const x_global = 80
  const y_range = 50
  const subgroups = data.columns.slice(1)
  const color = d3.scaleOrdinal()
    .domain(subgroups)
    .range(["#284a5d", "#5f7384", "#90a6b3"]);
  const flag = "NLP"

  svg2.append("text")
  .attr("y", "10%")
  .attr("x", "28%")
  .attr("font-size", "22px")
  .attr("text-anchor", "middle")
  .attr("font-weight", 600)
  .text("Gentlemen")

  create_stacked_bar(svg2, data, subgroups,
     x_global, y_range, color, flag)

});

d3.csv(baseAssetsUrl + "nlp_woman.csv").then(function(data){

  const x_global = 500
  const y_range = 50
  const subgroups = data.columns.slice(1)
  const color = d3.scaleOrdinal()
    .domain(subgroups)
    .range(["#8a3223", "#aa5549", "#bf897a"]);
  const flag = "NLP"

  svg2.append("text")
  .attr("y", "10%")
  .attr("x", "73%")
  .attr("font-size", "22px")
  .attr("text-anchor", "middle")
  .attr("font-weight", 600)
  .text("Ladies")

  create_stacked_bar(svg2, data, subgroups,
     x_global, y_range, color, flag)

});

});
'''


h = display(HTML(htmlt))
j = py_display.Javascript(js_t)
py_display.display_javascript(j)

## 3.4 Excuse me, do you experiment?

<p style="font-family: times-new-roman">Machine Learning experiment tracking tools have been a new area within the Data Science discipline. I know I have discovered and started using ML tracking this year, and we can see it is a new concept from the fact that a question regarding this area has appeared in the Data Science Survey only last year. However, it looks like it is starting to gain a little bit more popularity within our community.</p>

* <p style="font-family: times-new-roman">25% of 🎩men say they use no Experiment Tracking - however the percentage of 💃women that affirm they don't use any of these tools increased ~3% to around 20% in 2021.</p>
* <p style="font-family: times-new-roman"><b>Tensorboard and Weights&Biases</b> are the most used tools within experiment tracking.</p>

In [ ]:
cols = list(df.columns[183:192])
new_names = ["NeptuneAi", "WeightsAndBiases", "CometMl",
             "SacredAndOmniboard", "TensorBoard", "GuildAi",
             "Polyaxon", "DominoModelMonitor", "None"]

exp = get_data_multiple_answers(cols, new_names)
exp = exp[(exp["Year"]==2020) | (exp["Year"]==2021)]

exp_man = exp[exp["Gender"] == "Man"].drop(columns=["Gender"])
exp_man = exp_man.sort_values(exp_man.last_valid_index(), axis=1,
                    ascending=False)
exp_woman = exp[exp["Gender"] == "Woman"].drop(columns=["Gender"])
exp_woman = exp_woman.sort_values(exp_woman.last_valid_index(), axis=1,
                    ascending=False)

exp_man.to_csv("exp_man.csv", index=False)
exp_woman.to_csv("exp_woman.csv", index=False)

# Save usa.png img as well
im = imageio.imread('../input/kaggle-data-science-survey-20172021/images/man17.png')
Image.fromarray(im).save("man17.png")

im = imageio.imread('../input/kaggle-data-science-survey-20172021/images/lady17.png')
Image.fromarray(im).save("lady17.png")

htmlt = '''
<!-- Bootstrap -->
<link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.4.1/css/bootstrap.min.css" integrity="sha384-Vkoo8x4CGsO3+Hhxv8T/Q5PaXtkKtu6ug5TOeNV6gBiFeWPGFN9MuhOf23Q9Ifjh" crossorigin="anonymous">
<!-- Style -->
<style>
	.all17 {
		font-family: "Times New Roman", Times, serif;
		overflow-x: hidden;
		width: 98%;
		background-color:#e0c9a6;
		/* background-image : url("bkg.jpg"); changed via js */
	}
	.all17 .svg-container {
		display: inline-block;
		position: relative;
		width: 100%;
		padding-bottom: 80%;
		vertical-align: top;
		overflow: hidden;
	}
	.svg-content {
		display: inline-block;
		position: absolute;
		top: 0;
		left: 0;
	}

	.axisHidden path{
		stroke: #e0c9a6;
	}

	.axisHidden line{
		stroke: #e0c9a6;
	}

	.annotation.color text {
		fill: #000000;
	}

	@font-face {
		font-family: parklane;
		src: url(https://h4ks.net/ParkLaneNF.otf);
		}

	div.tooltip-donut {
		position: absolute;
		text-align: center;
		padding: .3rem;
		background: #FFFFFF;
		color: #000000;
		border: 1px solid #000000;
		border-radius: 8px;
		pointer-events: none;
		font-size: 2rem;
		z-index: 99999;
	}

</style>

<img id="baseimg" src="man.png" style="display:none" />
<div class="all17">
	<center><h1 style="font-family: parklane">Excuse me, do you experiment?</h1></center>
	<center><h3><i>- Tools used for ML Experiment Tracking -</i></h3></center>
	<div class="row">
		<div id="map17" class="svg-container"></div>
	</div>
</div>
'''

js_t = '''
require.config({
  paths: {
    d3src: "https://d3js.org/",
  },
  map: {
    '*': {
      'd3v6': 'd3src/d3.v6.min',
      'd3-selection': 'd3src/d3-selection.v1.min',
      'd3-drag': 'd3src/d3-drag.v1.min',
      'd3-shape': 'd3src/d3-shape.v1.min',
      'd3-path': 'd3src/d3-path.v1.min',
      'd3-dispatch': 'd3src/d3-dispatch.v1.min',
      'd3-annotation': 'https://cdnjs.cloudflare.com/ajax/libs/d3-annotation/2.5.1/d3-annotation.min.js',
    }
  }
});

require(["d3v6", "d3-annotation"], function(d3, d3Annotation) {

// Inject d3Annotation methods to d3
for (var key in d3Annotation) {
  d3[key] = d3Annotation[key];
}

// Get Assets URL set by Kaggle
const baseAssetsUrl = document.getElementById('baseimg').src.replace(/man.png.*$/, '');

const $all17 = document.getElementsByClassName('all17')[0];
$all17.style.backgroundImage = `url("${baseAssetsUrl}bkg.jpg")`;


// Set the SVG area
const margin = { LEFT: 20, RIGHT: 40, TOP: 15, BOTTOM: 0 }
const width = 950 - margin.LEFT - margin.RIGHT
const height = 700 - margin.TOP - margin.BOTTOM
const centreSpacing = 100

const svg = d3.select("#map17").append("svg")
// .attr("style", "outline: thin solid red;")
  .attr("preserveAspectRatio", "xMinYMin meet")
  .attr("viewBox", "0 0 950 700")
  .classed("svg-content", true)
  .attr("transform",
   `translate(${margin.LEFT}, ${margin.TOP})`)

// Gentleman
svg.append("image")
.attr("xlink:href", baseAssetsUrl+"man17.png")
    .attr("x", "23%")
    .attr("y", "7%")
    .attr("width", 230)
    .attr("height", 230)
    .style("opacity", 1)

// Lady
svg.append("image")
.attr("xlink:href", baseAssetsUrl+"lady17.png")
.attr("x", "52%")
.attr("y", "18%")
.attr("width", 270)
.attr("height", 270)
.style("opacity", 1)


function create_stacked_bar(data, subgroups, x_global, y_range, color) {

  const groups = data.map(d => d.Year)

  // Add X axis
  const x = d3.scaleBand()
      .domain(groups)
      .range([0, width/2.5])
      .padding([0.2])
  svg.append("g")
    .attr("transform", `translate(${x_global}, ${height/1.25})`)
    .call(d3.axisBottom(x).tickSizeOuter(0))
    .selectAll("text")
      .attr("y", "10")
      .attr("x", "0")
      .attr("font-size", "15px")
      .attr("font-weight", 300)
      .attr("text-anchor", "middle")

  // Add Y axis
  const y = d3.scaleLinear()
    .domain([0, y_range])
    .range([ height/2.3, 0]);
  svg.append("g")
  .attr("class", "axisHidden")
  .attr("transform", `translate(${x_global}, ${height/2-100})`)
    .call(d3.axisLeft(y).ticks(0))
    .selectAll("text")
      .attr("font-size", "11px")
      .attr("font-weight", 300);

  //s Stack data
  const stackedData = d3.stack()
    .keys(subgroups)
    (data)

  // Show the bars
  const bars = svg.append("g")
    .selectAll("g")
    .data(stackedData)
    .join("g")
    .attr("transform", `translate(${x_global}, ${height/2-100})`)
      .attr("fill", d => color(d.key))
      .attr("stroke", "black")
      .attr("stroke-width", 0.2)
      .attr("class", d => "myRect " + d.key ) 
      .selectAll("rect")
      .data(d => d)
      .join("rect")
        .attr("x", d => x(d.data.Year))
        .attr("y", d => y(d[1]))
        .attr("height", d => y(d[0]) - y(d[1]))
        .attr("width",x.bandwidth())

  // Interactive
  var div = d3.select("body").append("div")
  .attr("class", "tooltip-donut")
  .style("opacity", 0);

  bars
    .on('mouseenter', function (event, dt) {
    // MOUSE ON
    const subGroupName = d3.select(this.parentNode).datum().key 
          
    d3.selectAll(".myRect").style("opacity", 0.2)  
          
    d3.selectAll("."+subGroupName).style("opacity",1)


    // Makes the new div appear
    div.transition()
        .duration(50)
        .style("opacity", 1);

    const subgroupName = d3.select(this.parentNode).datum().key;
    const subgroupValue = dt.data[subgroupName];

    div.html("Exp: " + subgroupName + "<br>" + "Perc: " + subgroupValue +'%')
        .style("left", (event.pageX) + "px")
        .style("top", (event.pageY-30) + "px");


  })
    // MOUSE LEAVE
    .on('mouseleave', function () {
      d3.selectAll(".myRect")
          .style("opacity",1) 

      // New div dissapears
      div.transition()
          .duration('50')
          .style("opacity", 0);
      svg.selectAll('.divergence').remove()
    })

}


// === Preferences ===
d3.csv(baseAssetsUrl + "exp_man.csv").then(function(data){

  const x_global = 50
  const y_range = 42
  const subgroups = data.columns.slice(1)
  const color = d3.scaleOrdinal()
    .domain(subgroups)
    .range(["#233345", "#394758", "#4f5c6a",
            "#65707d", "#7b858f", "#9199a2",
            "#a7adb5", "#bdc2c7", "#d3d6da"]);

  svg.append("text")
  .attr("y", "5%")
  .attr("x", "25%")
  .attr("font-size", "25px")
  .attr("text-anchor", "middle")
  .attr("font-weight", 600)
  .text("Gentlemen: Experiment Tracking")

  create_stacked_bar(data, subgroups, x_global, y_range, color)
  
});

d3.csv(baseAssetsUrl + "exp_woman.csv").then(function(data){

  const x_global = 500
  const y_range = 42
  const subgroups = data.columns.slice(1)
  const color = d3.scaleOrdinal()
    .domain(subgroups)
    .range(["#8c2c69", "#984178", "#a35687",
            "#af6b96", "#ba80a5", "#c696b4",
            "#d1abc3", "#ddc0d2", "#e8d5e1"]);

  svg.append("text")
  .attr("y", "5%")
  .attr("x", "70%")
  .attr("font-size", "25px")
  .attr("text-anchor", "middle")
  .attr("font-weight", 600)
  .text("Ladies: Experiment Tracking")

  create_stacked_bar(data, subgroups, x_global, y_range, color)
  
});

const annot2 = [
  {
  note: { 
    label: "Most respondents choose to use no tools for ML Experiment Tracking.",
    title: "Overall No Usage",
    wrap: 270, 
    padding: 5, 
  },
  className: "color",
  // connector: { end: "arrow" },
  color: ["#ffffff"],
  x: 100,
  y: 610,
  dy: +0,
  dx: +0
},
{
  note: { 
    label: "There is no substantial change for any tool between 2020 and 2021 for any of the genders.",
    title: "General Constant Evolution",
    wrap: 270, 
    padding: 5, 
  },
  className: "color",
  // connector: { end: "arrow" },
  color: ["#ffffff"],
  x: 550,
  y: 610,
  dy: +0,
  dx: +0
}]

svg.append("g")
  .style('font-size', 18)
  .call(d3.annotation()
          .annotations(annot2))


});
'''


h = display(HTML(htmlt))
j = py_display.Javascript(js_t)
py_display.display_javascript(j)

<div class="alert simple-alert" style="font-family: times-new-roman">
  <p style="font-family: times-new-roman"><center>👀 We know that Matplotlib, Seaborn and Plotly are the overall "rulers" in terms of data visualization, with Seaborn increasing in popularity in 2021. We know that there is a decrease in specialized ML coders due to youngsters and that 🎩gents tend to have more experience than the 💃ladies do in this department. Scikit-learn, Tensorflow and Keras remain the most used frameworks, with PyTorch catching up each year. We know that Trees, Ensembles, CNNs and Image Classification are and remain very popular, with a surge in the Transformer Language Models department. And we can say that the Experiment Tracking tools are starting to gain more interest, however there is room to grow and shine in the coming years.</center></p>
</div>

# 4. Can I give you a ride over the estate?

> <p style="font-family: times-new-roman">🏛️ Historically, an estate comprises the houses, outbuildings, supporting farmland, and woods that surround the gardens and grounds of a very large property, such as a country house or mansion. It is an "estate" because the profits from its produce and rents are sufficient to support the household in the house at its center, formerly known as the manor house (like in the image below). Thus, "the estate" may refer to all other cottages and villages in the same ownership as the mansion itself, covering more than one former manor (many similarities to ... a business). (<a href="https://en.wikipedia.org/wiki/Estate_(land)">source can be found here</a>)</p>

<center><img src="https://i.imgur.com/yM2X6Xx.gif" width=800></center>

## 4.1 Estate's Industry & Size

<h3 style="font-family: times-new-roman; font-size: 1.5em">I. The Industry of the Business</h3>

* <p style="font-family: times-new-roman">🎩 In 2021, <b>gents can be found the most within the Technology Industry</b> (I am thinking Engineering, as many of them have this job position) <b>and within Academics</b> (teachers/students). They also prefer the <i>Manufacturing</i> Industry slightly better than ladies do.</p>
* <p style="font-family: times-new-roman">💃 As for <b>ladies, they can be found the most within Academia and Technology Industries</b> (reversed order vs men), although the percentages are extremely close to each other. Ladies also better prefer the <i>Medical/Pharmaceutical</i> Industry.</p>

> <p style="font-family: times-new-roman">📌 Note: this question has data available only for the years 2018 and 2021. Hence, I could not show a reliable ranking evolution through time, but I can say that there are no significant changes between 2018 and 2021.</p>

<p style="font-family: times-new-roman">Full Legend of Industries below:</p>


<center>
<table>
<thead>
<tr>
<th>Mapping</th>
<th>Full Name</th>
</tr>
</thead>
<tbody><tr>
<td>Technology</td>
<td>Computers/Technology</td>
</tr>
<tr>
<td>Academics</td>
<td>Academics/Education</td>
</tr>
<tr>
<td>Finance</td>
<td>Accounting/Finance</td>
</tr>
<tr>
<td>Manufacturing</td>
<td>Manufacturing/Fabrication</td>
</tr>
<tr>
<td>Medical</td>
<td>Medical/Pharmaceutical</td>
</tr>
<tr>
<td>Government</td>
<td>Government/Public Service</td>
</tr>
<tr>
<td>Energy</td>
<td>Energy/Mining</td>
</tr>
<tr>
<td>Internet Service</td>
<td>Online Service/Internet-based Services</td>
</tr>
<tr>
<td>Sales</td>
<td>Retail/Sales</td>
</tr>
<tr>
<td>Insurance</td>
<td>Insurance/Risk Assessment</td>
</tr>
<tr>
<td>Marketing</td>
<td>Marketing/CRM</td>
</tr>
<tr>
<td>Communications</td>
<td>Broadcasting/Communications</td>
</tr>
<tr>
<td>Shipping</td>
<td>Shipping/Transportation</td>
</tr>
<tr>
<td>Non-profit</td>
<td>Non-profit/Service</td>
</tr>
<tr>
<td>Security</td>
<td>Military/Security/Defense</td>
</tr>
<tr>
<td>Entertainment</td>
<td>Hospitality/Entertainment/Sports</td>
</tr>
</tbody></table>
</center>

In [ ]:
cols = ["Year", "What is your gender? - Selected Choice",
       df.columns[108]]

industry = df[cols]
industry.columns = ["Year", "Gender", "Industry"]
industry = industry[industry["Gender"].isin(["Man", "Woman"])]

old_list = ['Computers/Technology', 'Academics/Education',
       'Accounting/Finance', 'Other', 'Manufacturing/Fabrication',
       'Medical/Pharmaceutical', 'Government/Public Service',
       'Energy/Mining', 'Online Service/Internet-based Services',
       'Retail/Sales', 'Insurance/Risk Assessment', 'Marketing/CRM',
       'Broadcasting/Communications', 'Shipping/Transportation',
       'Non-profit/Service', 'Online Business/Internet-based Sales',
       'Military/Security/Defense', 'Hospitality/Entertainment/Sports']

new_list = ["Technology", "Academics", 
            "Finance", "Other", "Manufacturing",
            "Medical", "Government", 
            "Energy", "Internet Service",
            "Sales", "Insurance", "Marketing",
            "Communications", "Shipping", 
            "Non-profit", "Internet Service", 
            "Security", "Entertainment"]

industry["Industry"] = industry["Industry"].replace(old_list, new_list)
industry = industry.value_counts().reset_index()
industry.columns = ["Year", "Gender", "Industry", "Count"]
industry = industry[industry["Year"]==2021]   ### YEAR 2021

industry_man = industry[industry["Gender"]=="Man"]
industry_woman = industry[industry["Gender"]=="Woman"]

industry_man.to_csv("industry_man.csv", index=False)
industry_woman.to_csv("industry_woman.csv", index=False)

# Save usa.png img as well
im = imageio.imread('../input/kaggle-data-science-survey-20172021/images/man18.png')
Image.fromarray(im).save("man18.png")

im = imageio.imread('../input/kaggle-data-science-survey-20172021/images/lady18.png')
Image.fromarray(im).save("lady18.png")

htmlt = '''
<!-- Bootstrap -->
<link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.4.1/css/bootstrap.min.css" integrity="sha384-Vkoo8x4CGsO3+Hhxv8T/Q5PaXtkKtu6ug5TOeNV6gBiFeWPGFN9MuhOf23Q9Ifjh" crossorigin="anonymous">
<!-- Style -->
<style>
	.all18 {
		font-family: "Times New Roman", Times, serif;
		overflow-x: hidden;
		width: 98%;
		background-color:#e0c9a6;
		/* background-image : url("bkg.jpg"); changed via js */
	}
	.svg-container {
		display: inline-block;
		position: relative;
		width: 100%;
		padding-bottom: 100%;
		vertical-align: top;
		overflow: hidden;
	}
	.svg-content {
		display: inline-block;
		position: absolute;
		top: 0;
		left: 0;
	}

	.axisHidden path{
		stroke: #e0c9a6;
	}

	.axisHidden line{
		stroke: #e0c9a6;
	}

	.all18 .annotation.gents text {
		fill: #374e5c;
	}

	.all18 .annotation.ladies text {
		fill: #af3021;
	}

	@font-face {
		font-family: parklane;
		src: url(https://h4ks.net/ParkLaneNF.otf);
		}

	div.tooltip-donut {
		position: absolute;
		text-align: center;
		padding: .3rem;
		background: #FFFFFF;
		color: #000000;
		border: 1px solid #000000;
		border-radius: 8px;
		pointer-events: none;
		font-size: 1.3rem;
		z-index: 99999;
	}

</style>

<img id="baseimg" src="man.png" style="display:none" />
<div class="all18">
	<center><h1 style="font-family: parklane">Estate's Industry</h1></center>
	<center><h2>2021: In which industry do the respondents work?</h2></center>
	<div class="row">
		<div id="map18" class="svg-container"></div>
	</div>
</div>
'''

js_t = '''
require.config({
  paths: {
    d3src: "https://d3js.org/",
  },
  map: {
    '*': {
      'd3v6': 'd3src/d3.v6.min',
      'd3-selection': 'd3src/d3-selection.v1.min',
      'd3-drag': 'd3src/d3-drag.v1.min',
      'd3-shape': 'd3src/d3-shape.v1.min',
      'd3-path': 'd3src/d3-path.v1.min',
      'd3-dispatch': 'd3src/d3-dispatch.v1.min',
      'd3-annotation': 'https://cdnjs.cloudflare.com/ajax/libs/d3-annotation/2.5.1/d3-annotation.min.js',
    }
  }
});

require(["d3v6", "d3-annotation"], function(d3, d3Annotation) {

// Inject d3Annotation methods to d3
for (var key in d3Annotation) {
  d3[key] = d3Annotation[key];
}

// Get Assets URL set by Kaggle
const baseAssetsUrl = document.getElementById('baseimg').src.replace(/man.png.*$/, '');

const $all18 = document.getElementsByClassName('all18')[0];
$all18.style.backgroundImage = `url("${baseAssetsUrl}bkg.jpg")`;

// Set the SVG area
const margin = { LEFT: 10, RIGHT: 10, TOP: 10, BOTTOM: 0 }
const width = 950 - margin.LEFT - margin.RIGHT
const height = 900 - margin.TOP - margin.BOTTOM
const innerRadius = 100
const outerRadius = Math.min(width, height) / 5

const svg = d3.select("#map18").append("svg")
  .attr("preserveAspectRatio", "xMinYMin meet")
  .attr("viewBox", "0 0 950 900")
  .classed("svg-content", true)
  .attr("transform",
   `translate(${margin.LEFT}, ${margin.TOP})`)


// Men
svg.append("image")
.attr("xlink:href", baseAssetsUrl+"man18.png")
    .attr("x", "9.5%")
    .attr("y", "47%")
    .attr("width", 280)
    .attr("height", 280)
    .style("opacity", 1)

// Women
svg.append("image")
.attr("xlink:href", baseAssetsUrl+"lady18.png")
    .attr("x", "59%")
    .attr("y", "47%")
    .attr("width", 270)
    .attr("height", 270)
    .style("opacity", 1)

svg.append('line')
.style("stroke", "black")
.style("stroke-width", 1)
    .attr("x1", 480)
    .attr("y1", 60)
    .attr("x2", 480)
    .attr("y2", 900)


// === Men ===
d3.csv(baseAssetsUrl + "industry_man.csv").then(function(data){

  const x_global = 230

  svg.append("text")
  .attr("transform", `translate(${x_global}, ${width/3-200})`)
  .attr("font-size", "28px")
  .attr("text-anchor", "middle")
  .attr("font-weight", 600)
  .text("Gentlemen")

  data.forEach(d => {
    d.Count = Number(d.Count)
  })

  // Color
  var myColor = d3.scaleLinear().domain([1,18])
                  .range(["#374e5c", "#c3cace"])

  // Scales
  const x = d3.scaleBand()
      .range([0, 2 * Math.PI]) 
      .align(0)        
      .domain(data.map(d => d.Industry));
  const y = d3.scaleRadial()
      .range([innerRadius, outerRadius])  
      .domain([0, d3.max(data, d => d.Count)]); 

  // Add the bars
  svg.append("g")
    .selectAll("path")
    .data(data)
    .join("path")
    .attr("transform", `translate(${x_global}, ${width/1.8})`)
      .attr("fill", (d, i) => myColor(i))
      .attr("d", d3.arc()   
          .innerRadius(innerRadius)
          .outerRadius(d => y(d['Count']))
          .startAngle(d => x(d.Industry))
          .endAngle(d => x(d.Industry) + x.bandwidth())
          .padAngle(0.01)
          .padRadius(innerRadius))

  // Add the labels
  svg.append("g")
  .attr("transform", `translate(${x_global}, ${width/1.8})`)
      .selectAll("g")
      .data(data)
      .join("g")
        .attr("text-anchor", function(d) { return (x(d.Industry) + x.bandwidth() / 2 + Math.PI) % (2 * Math.PI) < Math.PI ? "end" : "start"; })
        .attr("transform", function(d) { return "rotate(" + ((x(d.Industry) + x.bandwidth() / 2) * 180 / Math.PI - 90) + ")"+"translate(" + (y(d['Count'])+10) + ",0)"; })
      .append("text")
        .text(function(d){return(d.Industry)})
        .attr("transform", function(d) { return (x(d.Industry) + x.bandwidth() / 2 + Math.PI) % (2 * Math.PI) < Math.PI ? "rotate(180)" : "rotate(0)"; })
        .style("font-size", "14px")
        .attr("font-weight", 600)
        .attr("alignment-baseline", "middle")

  });

// === Women ===
d3.csv(baseAssetsUrl + "industry_woman.csv").then(function(data){

  const x_global = 695

  svg.append("text")
  .attr("transform", `translate(${x_global}, ${width/3-200})`)
  .attr("font-size", "28px")
  .attr("text-anchor", "middle")
  .attr("font-weight", 600)
  .text("Ladies")

  data.forEach(d => {
    d.Count = Number(d.Count)
  })

  // Color
  var myColor = d3.scaleLinear().domain([1,18])
                  .range(["#af3021", "#e7c1bc"])

  // Scales
  const x = d3.scaleBand()
      .range([0, 2 * Math.PI]) 
      .align(0)        
      .domain(data.map(d => d.Industry));
  const y = d3.scaleRadial()
      .range([innerRadius, outerRadius])  
      .domain([0, d3.max(data, d => d.Count)]); 

  // Add the bars
  svg.append("g")
    .selectAll("path")
    .data(data)
    .join("path")
    .attr("transform", `translate(${x_global}, ${width/1.8})`)
    .attr("fill", (d, i) => myColor(i))
      .attr("d", d3.arc()   
          .innerRadius(innerRadius)
          .outerRadius(d => y(d['Count']))
          .startAngle(d => x(d.Industry))
          .endAngle(d => x(d.Industry) + x.bandwidth())
          .padAngle(0.01)
          .padRadius(innerRadius))

  // Add the labels
  svg.append("g")
  .attr("transform", `translate(${x_global}, ${width/1.8})`)
      .selectAll("g")
      .data(data)
      .join("g")
        .attr("text-anchor", function(d) { return (x(d.Industry) + x.bandwidth() / 2 + Math.PI) % (2 * Math.PI) < Math.PI ? "end" : "start"; })
        .attr("transform", function(d) { return "rotate(" + ((x(d.Industry) + x.bandwidth() / 2) * 180 / Math.PI - 90) + ")"+"translate(" + (y(d['Count'])+10) + ",0)"; })
      .append("text")
        .text(function(d){return(d.Industry)})
        .attr("transform", function(d) { return (x(d.Industry) + x.bandwidth() / 2 + Math.PI) % (2 * Math.PI) < Math.PI ? "rotate(180)" : "rotate(0)"; })
        .style("font-size", "14px")
        .attr("font-weight", 600)
        .attr("alignment-baseline", "middle")

  });


// Annot
const annot1 = [
  {
  note: { 
    label: "These are top 3 most frequent industries for both genders, with a slight difference in order.",
    title: "Tech/Academics/Finance",
    wrap: 300, 
    padding: 5, 
  },
  className: "gents",
  connector: { end: "arrow" },
  color: ["#ffffff"],
  x: 310,
  y: 350,
  dy: -100,
  dx: -1
},
{
  note: { 
    label: "In top 6 places we see that the Medical/Pharmaceutical fields are more common for ladies vs the Manufacturing/Fabrication fields for gents.",
    title: "Medical vs Manufacturing",
    wrap: 340, 
    padding: 5, 
  },
  className: "ladies",
  connector: { end: "arrow" },
  color: ["#ffffff"],
  x: 845,
  y: 610,
  dy: +150,
  dx: -1
}]

svg.append("g")
  .style('font-size', 19)
  .call(d3.annotation()
          .annotations(annot1))

});
'''


h = display(HTML(htmlt))
j = py_display.Javascript(js_t)
py_display.display_javascript(j)

<h3 style="font-family: times-new-roman; font-size: 1.5em">II. The Size of the Business</h3>

* <p style="font-family: times-new-roman"> Around <b>20% of respondents</b> (same pattern in both genders) say that they are <b>employed in very small companies</b> (between 1 and 49 employees). This trend remained quite constant from 2019 through 2021.</p>
* <p style="font-family: times-new-roman"> Another <b>20%-25% of respondents</b> are at the other end of the spectrum, saying that they <b>work within large and very large companies</b> of more than 1,000 employees (shown for both ladies and gents).</p>
* <p style="font-family: times-new-roman"> There was a <i>smaller percentage of respondents that answered this question</i> (be aware, not in absolute numbers - but in percentage on year and gender) in 2020 vs 2019. In 2021 it stabilized a little, but I am wondering if the decrease in 2020 was marked by some elderly more <i>experienced people on the platform that preferred to keep these pieces of information private</i>.</p>

In [ ]:
cols = ["Year", "What is your gender? - Selected Choice",
       df.columns[109]]

size = df[cols]
size.columns = ["Year", "Gender", "Size"]
size = size[size["Gender"].isin(["Man", "Woman"])]

old_list = ['0-49 employees', '> 10,000 employees', '1000-9,999 employees',
            '50-249 employees', '250-999 employees', '10,000 or more employees']
new_list = ['1. 0-49 employees', '5. >10,000 employees', '4. 1000-9,999 employees',
            '2. 50-249 employees', '3. 250-999 employees', '5. >10,000 employees']

size["Size"] = size["Size"].replace(old_list, new_list)
size = size.value_counts().reset_index()
size = pd.merge(size, total, on=["Year", "Gender"])

size["Perc"] = size[0]/size["Count"]*100
size["Perc"] = size["Perc"].apply(lambda x: round(x, 1))
size.drop(columns=[0, "Count"], axis=1, inplace=True)
size = pd.pivot(size, index=["Year", "Gender"], columns="Size",
                values="Perc").reset_index()

size_man = size[size["Gender"] == "Man"].drop(columns=["Gender"])
size_woman = size[size["Gender"] == "Woman"].drop(columns=["Gender"])

size_man.to_csv("size_man.csv", index=False)
size_woman.to_csv("size_woman.csv", index=False)

# Save usa.png img as well
im = imageio.imread('../input/kaggle-data-science-survey-20172021/images/man19.png')
Image.fromarray(im).save("man19.png")

im = imageio.imread('../input/kaggle-data-science-survey-20172021/images/lady19.png')
Image.fromarray(im).save("lady19.png")

htmlt = '''
<!-- Bootstrap -->
<link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.4.1/css/bootstrap.min.css" integrity="sha384-Vkoo8x4CGsO3+Hhxv8T/Q5PaXtkKtu6ug5TOeNV6gBiFeWPGFN9MuhOf23Q9Ifjh" crossorigin="anonymous">
<!-- Style -->
<style>
	.all19 {
		font-family: "Times New Roman", Times, serif;
		overflow-x: hidden;
		width: 98%;
		background-color:#e0c9a6;
		/* background-image : url("bkg.jpg"); changed via js */
	}
	.all19 .svg-container {
		display: inline-block;
		position: relative;
		width: 100%;
		padding-bottom: 54%;
		vertical-align: top;
		overflow: hidden;
	}
	.svg-content {
		display: inline-block;
		position: absolute;
		top: 0;
		left: 0;
	}

	.axisHidden path{
		stroke: #e0c9a6;
	}

	.axisHidden line{
		stroke: #e0c9a6;
	}

	.annotation.color text {
		fill: #000000;
	}

	@font-face {
		font-family: parklane;
		src: url(https://h4ks.net/ParkLaneNF.otf);
		}

	div.tooltip-donut {
		position: absolute;
		text-align: center;
		padding: .3rem;
		background: #FFFFFF;
		color: #000000;
		border: 1px solid #000000;
		border-radius: 8px;
		pointer-events: none;
		font-size: 2rem;
		z-index: 99999;
	}

</style>

<img id="baseimg" src="man.png" style="display:none" />
<div class="all19">
	<center><h1 style="font-family: parklane">Estate's Size</h1></center>
	<center><h3><i>- Percent of respondents on company size -</i></h3></center>
	<div class="row">
		<div id="map19-gents" class="svg-container"></div>
	</div>

	<div class="row">
		<div id="map19-ladies" class="svg-container"></div>
	</div>
</div>
'''

js_t = '''
require.config({
  paths: {
    d3src: "https://d3js.org/",
  },
  map: {
    '*': {
      'd3v6': 'd3src/d3.v6.min',
      'd3-selection': 'd3src/d3-selection.v1.min',
      'd3-drag': 'd3src/d3-drag.v1.min',
      'd3-shape': 'd3src/d3-shape.v1.min',
      'd3-path': 'd3src/d3-path.v1.min',
      'd3-dispatch': 'd3src/d3-dispatch.v1.min',
      'd3-annotation': 'https://cdnjs.cloudflare.com/ajax/libs/d3-annotation/2.5.1/d3-annotation.min.js',
    }
  }
});

require(["d3v6", "d3-annotation"], function(d3, d3Annotation) {

// Inject d3Annotation methods to d3
for (var key in d3Annotation) {
  d3[key] = d3Annotation[key];
}

// Get Assets URL set by Kaggle
const baseAssetsUrl = document.getElementById('baseimg').src.replace(/man.png.*$/, '');

const $all19 = document.getElementsByClassName('all19')[0];
$all19.style.backgroundImage = `url("${baseAssetsUrl}bkg.jpg")`;

// Set the SVG area
const margin = { LEFT: 20, RIGHT: 40, TOP: 15, BOTTOM: 0 }
const width = 950 - margin.LEFT - margin.RIGHT
const height = 500 - margin.TOP - margin.BOTTOM
const chart_size = 40
const x_global = 150

// STREAMGRAPH
function create_graph(data, svg, name){

  const XY_axis = height/2-200

  svg.append("text")
  .attr("transform", `translate(${x_global-60}, ${XY_axis-10})`)
  .attr("font-size", "24px")
  .attr("text-anchor", "middle")
  .attr("font-weight", 600)
  .text(name)

  data.forEach(d => {
    d.Year = Number(d.Year)
  })

  // List of groups
  var keys = data.columns.slice(1)

  // Stacking
  var series = d3.stack()
    .keys(keys)
    .offset(d3.stackOffsetSilhouette)
    (data)

  // X Axis
  var x = d3.scaleLinear()
    .domain(d3.extent(data, d => d.Year))
    .range([0, width/1.5])

  var xAxis = g => g
    .attr("transform", `translate(${x_global}, ${XY_axis})`)
    .call(d3.axisBottom(x)
            .tickSize(height/1.2).ticks(2).tickFormat(d3.format("d")))
    .call(g => g.select(".domain").remove())
    .call(g => g.selectAll(".tick line")
                .attr("stroke", "#3A3A3A")
                .attr('stroke-dasharray', '5 5'))
    .call(g => g.selectAll(".tick")
                .select('text')
                .attr('fill', "#3A3A3A")
                .style("font-size", 15))

  // Y Axis
  var y = d3.scaleLinear()
  .domain([-chart_size, chart_size])
  .range([height/1.2, 0])
  svg.append("g")
  .attr("class", "axisHidden")
  .attr("transform", `translate(${x_global}, ${XY_axis})`)
    .call(d3.axisLeft(y).ticks(0))

  // Area Chart
  var area = d3.area()
  .x(d => x(d.data.Year))
  .y0(d => y(d[0]))
  .y1(d => y(d[1]))

  // Color
  const color = d3.scaleOrdinal()
    .domain(keys)
     .range(["#D05257", "#D87F58", "#CE8E27","#79A988",
     "#507C5D"])

  // Show
  const path = svg.append("g")
    .selectAll("path")
    .data(series)
    .join("path")
    .attr("transform", `translate(${x_global}, ${XY_axis})`)
      .attr("data-label", d => d.key)
      .attr("fill", ({key}) => color(key))
      .attr("stroke", "black")
      .attr("stroke-width", 0.2)
      .attr("d", area)
      .attr("opacity", 0.8)

  svg.append("g")
      .call(xAxis)

  // ==== Interactive ====
  const hover = (svg, path) => {
  
    const line = svg.append("g")
         .attr("display", "none");
  
    line.append("g")
        .selectAll("line")
        .data(series)
        .join("line")
        .attr("class", "cursor-line")
        .attr("fill", "#fff")
        .attr("stroke-width", 0.8)
        .attr("x1", 10)
        .attr("y1", height)
        .attr("x2", 10)
        .attr("y2", 10);

    line.append("text")
      .attr("class", "text-year")
      .attr("font-size", 14)
      .attr("x", 0)
      .attr("y", 26)
      .attr("transform", "rotate(-90 20 20) translate(0, -20)");
    
    line.append("g")
        .selectAll("text")
        .data(series)
        .join("text")
        .attr("class", "text-label")
        .attr("font-weight", 900)
        .attr("font-size", 17)
        .attr("y", height-10)
        .attr("x", -8)
        .attr("transform", 
        "rotate(-90 10 460)");

    line.append("g")
        .selectAll("text")
        .data(data)
        .join("text")
        .attr("class", "text-value")
        .attr("font-weight", 700)
        .attr("font-size", 14)
        .attr("y", height-(height*0.94))
        .attr("x", 14);

    const mousemove = (event) => {
  
      event?.preventDefault();
      const pointer = d3.pointer(event);

      const xm = x.invert(pointer[0]);
      const label = d3.select(event.target).attr("data-label");
  
      line.attr("transform", `translate(${pointer[0]}, 0)`)
          .style("visibility", "inherit");
    
          line.select(".text-year")
          .text(parseInt(xm));
      
      line.selectAll(".text-value")
        .text(d => d.Year === parseInt(xm) ? d[label]+"%" : "");

      line.selectAll(".text-label")
        .style("visibility", "hidden")
        .filter(d => d.key === label)
        .style("visibility", "inherit")
        .attr("fill", d => d.key === label ? color(d.key) : "#000")
        .text(label);
      
      line.selectAll(".cursor-line")
        .attr("stroke", "fff0")
        .filter(d => d.key === label)
        .attr("stroke", d => d.key === label ? color(d.key) : "#fff0")
      
      path.attr("opacity", d => d.key === label ? 0.8 : 0.3);
      
      if (label === null) {
        path.attr("opacity", 0.8);
        line.style("visibility", "hidden");
      }
    }

    const mouseenter = () => {
      line.attr("display", null);
    }
  
    const mouseleave = () => {
      line.attr("display", "none");
    }

    svg
        .on("mousemove", mousemove)
        .on("mouseenter", mouseenter)
        .on("mouseleave", mouseleave)

  }

  svg.call(hover, path);
}

// ===== MEN =====
const svg1 = d3.select("#map19-gents").append("svg")
// .attr("style", "outline: thin solid red;")
  .attr("preserveAspectRatio", "xMinYMin meet")
  .attr("viewBox", "0 0 950 500")
  .classed("svg-content", true)
  .attr("transform",
   `translate(${margin.LEFT}, ${margin.TOP})`)

// Gentleman
svg1.append("image")
.attr("xlink:href", baseAssetsUrl+"man19.png")
    .attr("x", "-13%")
    .attr("y", "11%")
    .attr("width", 430)
    .attr("height", 430)
    .style("opacity", 1)

// Comment
const annot1 = [
  {
  note: { 
    label: "~20% of respondents (men & women) say that they work in small companies (0-49 employees).",
    title: "Small Companies",
    wrap: 100, 
    padding: 5, 
  },
  className: "color",
  connector: { end: "arrow" },
  color: ["#ffffff"],
  x: 755,
  y: 350,
  dy: -35,
  dx: +30
}]

svg1.append("g")
  .style('font-size', 17)
  .call(d3.annotation()
          .annotations(annot1))

d3.csv(baseAssetsUrl + "size_man.csv").then(function(data){

  const name = "Gentlemen"
  create_graph(data, svg1, name)

});

// ===== WOMEN =====
const svg2 = d3.select("#map19-ladies").append("svg")
// .attr("style", "outline: thin solid red;")
.attr("preserveAspectRatio", "xMinYMin meet")
.attr("viewBox", "0 0 950 500")
.classed("svg-content", true)
.attr("transform",
  `translate(${margin.LEFT}, ${margin.TOP})`)

// Lady
svg2.append("image")
.attr("xlink:href", baseAssetsUrl+"lady19.png")
.attr("x", "-19%")
.attr("y", "8%")
.attr("width", 490)
.attr("height", 490)
.style("opacity", 1)

// Comment
const annot2 = [
  {
  note: { 
    label: "At the other end of the spectrum, another ~20%-25% of respondents (both genders) confirm that they work in large companies (1k+ employees).",
    title: "Large Companies",
    wrap: 120, 
    padding: 5, 
  },
  className: "color",
  connector: { end: "arrow" },
  color: ["#ffffff"],
  x: 755,
  y: 170,
  dy: +35,
  dx: +30
}]

svg2.append("g")
  .style('font-size', 17)
  .call(d3.annotation()
          .annotations(annot2))

d3.csv(baseAssetsUrl + "size_woman.csv").then(function(data){

  const name = "Ladies"
  create_graph(data, svg2, name)

});

});
'''


h = display(HTML(htmlt))
j = py_display.Javascript(js_t)
py_display.display_javascript(j)

## 4.2 Estate's Profitability

<h3 style="font-family: times-new-roman; font-size: 1.5em">I. The Salary</h3>

> <p style="font-family: times-new-roman"> 📌 Note: The salaries were adjusted on respondent's country, meaning that the pay (in US dollars) was balanced to be showing the power of purchase within the place of residence, rather than just the overall brute amount. The adjustment was done as I did last year, meaning that I have converted the salary to "number of McMeal units a person can buy in their country". The more units, the bigger the purchasing power. <i>Unfortunately, I could not adjust the pay on years too as I could not find data for all 4 years - hence there might be some inflation bias within the numbers.</i></p>

* <p style="font-family: times-new-roman"> 🎩 For gentlemen, the most prominent change between 2018 and 2021 is that there are many <b>more respondents that have lower salaries</b>. This is also evident in the general mean of the amount, which increased in 2019 but dropped at the lowest in 2021.</p>
* <p style="font-family: times-new-roman"> 💃 This trend is the <b>exact same for ladies</b> too. In general, there are many more respondents with lower income in 2021 vs 2018. The average pay dropped suddenly for women in 2020 vs 2019 and remained at the same value in 2021. </p>
* <p style="font-family: times-new-roman"> I strongly believe this decrease in overall pay is <i>not due to general lower income within the Data Science/ Machine Learning Industry</i>. These industries actually have some of the biggest pays around the world. I would rather think this trend shows because <b>there are many more young people that took the survey, hence they naturally have lower income vs somebody with 5+ years of experience</b>. I am confident we will see a surge in higher pays in the following years.</p>

In [ ]:
# Combine salary with job title
cols = ["Year", "What is your gender? - Selected Choice",
        "In which country do you currently reside?",
       df.columns[120]]

salary = df[cols]
salary.columns = ["Year", "Gender", "Country", "Salary"]
salary = salary[salary["Gender"].isin(["Man", "Woman"])]

old_list = ['25,000-29,999', '60,000-69,999', '$0-999', '30,000-39,999',
       '15,000-19,999', '70,000-79,999', '2,000-2,999', '10,000-14,999',
       '5,000-7,499', '20,000-24,999', '1,000-1,999', '100,000-124,999',
       '7,500-9,999', '4,000-4,999', '40,000-49,999', '50,000-59,999',
       '3,000-3,999', '300,000-499,999', '200,000-249,999',
       '125,000-149,999', '250,000-299,999', '80,000-89,999',
       '90,000-99,999', '150,000-199,999', '>$1,000,000',
       '$500,000-999,999', '> $500,000', '300,000-500,000', '10-20,000',
       '0-10,000',
       'I do not wish to disclose my approximate yearly compensation',
       '20-30,000', '125-150,000', '30-40,000', '50-60,000',
       '100-125,000', '90-100,000', '70-80,000', '80-90,000', '60-70,000',
       '400-500,000', '40-50,000', '150-200,000', '300-400,000',
       '200-250,000', '500,000+', '250-300,000']
new_list = ['25,000-29,999', '60,000-69,999', '0-999', '30,000-39,999',
       '15,000-19,999', '70,000-79,999', '2,000-2,999', '10,000-14,999',
       '5,000-7,499', '20,000-24,999', '1,000-1,999', '100,000-124,999',
       '7,500-9,999', '4,000-4,999', '40,000-49,999', '50,000-59,999',
       '3,000-3,999', '300,000-499,999', '200,000-249,999',
       '125,000-149,999', '250,000-299,999', '80,000-89,999',
       '90,000-99,999', '150,000-199,999', '500,000',
       '500,000', '500,000', '300,000-499,999', '10,000-19,999',
       '0-9,999',
       'Undisclosed',
       '20,000-29,999', '125,000-149,999', '30,000-39,999', '50,000-59,999',
       '100,000-124,999', '90,000-99,999', '70,000-79,999', '80,000-89,999', '60,000-69,999',
       '400,000-499,999', '40,000-49,999', '150,000-199,999', '300,000-399,999',
       '200,000-249,999', '500,000', '250-299,999']

salary["Salary"] = salary["Salary"].replace(old_list, new_list)
salary = salary[salary["Salary"]!="Undisclosed"].dropna().reset_index(drop=True)

# Adjust Salary
salary = adjust_amount(data=salary, amount_column="Salary")

# Sepparate genders
salary_man = salary[salary["Gender"]=="Man"].drop(columns="Gender", axis=1)
salary_woman = salary[salary["Gender"]=="Woman"].drop(columns="Gender", axis=1)

salary_man.to_csv("salary_man.csv", index=False)
salary_woman.to_csv("salary_woman.csv", index=False)

# Save usa.png img as well
im = imageio.imread('../input/kaggle-data-science-survey-20172021/images/man20.png')
Image.fromarray(im).save("man20.png")

im = imageio.imread('../input/kaggle-data-science-survey-20172021/images/lady20.png')
Image.fromarray(im).save("lady20.png")

htmlt = '''
<!-- Bootstrap -->
<link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.4.1/css/bootstrap.min.css" integrity="sha384-Vkoo8x4CGsO3+Hhxv8T/Q5PaXtkKtu6ug5TOeNV6gBiFeWPGFN9MuhOf23Q9Ifjh" crossorigin="anonymous">
<!-- Style -->
<style>
	.all20 {
		font-family: "Times New Roman", Times, serif;
		overflow-x: hidden;
		width: 98%;
		background-color:#e0c9a6;
		/* background-image : url("bkg.jpg"); changed via js */
	}
	.all20 .svg-container {
		display: inline-block;
		position: relative;
		width: 100%;
		padding-bottom: 54%;
		vertical-align: top;
		overflow: hidden;
	}
	.svg-content {
		display: inline-block;
		position: absolute;
		top: 0;
		left: 0;
	}

	.all20 .axisHidden path{
		stroke-opacity: 0;
	}

	.all20 .axisHidden line{
		stroke-opacity: 0;
	}

	.all20 .annotation.gents text {
		fill: #33555e;
	}

	.all20 .annotation.ladies text {
		fill: #9C4254;
	}

	.all20 .annotation.neutral text {
		fill: #000000;
	}

	@font-face {
		font-family: parklane;
		src: url(https://h4ks.net/ParkLaneNF.otf);
		}

	div.tooltip-donut {
		position: absolute;
		text-align: center;
		padding: .3rem;
		background: #FFFFFF;
		color: #000000;
		border: 1px solid #000000;
		border-radius: 8px;
		pointer-events: none;
		font-size: 2rem;
		z-index: 99999;
	}

</style>

<img id="baseimg" src="man.png" style="display:none" />
<div class="all20">
	<center><h1 style="font-family: parklane">Respondent's Payment</h1></center>
	<center><h3><i>- Salary was adjusted on respondent's rezidence -</i></h3></center>
	<div class="row">
		<div id="map20-gents" class="svg-container"></div>
	</div>

	<div class="row">
		<div id="map20-ladies" class="svg-container"></div>
	</div>
</div>
'''

js_t = '''
require.config({
  paths: {
    d3src: "https://d3js.org/",
  },
  map: {
    '*': {
      'd3v4': 'd3src/d3.v4.min',
      'd3-selection': 'd3src/d3-selection.v1.min',
      'd3-drag': 'd3src/d3-drag.v1.min',
      'd3-shape': 'd3src/d3-shape.v1.min',
      'd3-path': 'd3src/d3-path.v1.min',
      'd3-dispatch': 'd3src/d3-dispatch.v1.min',
      'd3-annotation': 'https://cdnjs.cloudflare.com/ajax/libs/d3-annotation/2.5.1/d3-annotation.min.js',
      }
  }
});

require(["d3v4", "d3-annotation"], function(d3, d3Annotation) {

// Inject d3Annotation methods to d3
for (var key in d3Annotation) {
  d3[key] = d3Annotation[key];
}

// Get Assets URL set by Kaggle
const baseAssetsUrl = document.getElementById('baseimg').src.replace(/man.png.*$/, '');

const $all20 = document.getElementsByClassName('all20')[0];
$all20.style.backgroundImage = `url("${baseAssetsUrl}bkg.jpg")`;

// Set the SVG area
const margin = { LEFT: 20, RIGHT: 40, TOP: 15, BOTTOM: 0 }
const width = 950 - margin.LEFT - margin.RIGHT
const height = 500 - margin.TOP - margin.BOTTOM
const x_global = 180

// VIOLIN
function create_violin(data, svg, name, color1, color2,
              grad_name, data2){
  // Title
  svg.append("text")
  .attr("transform", `translate(${x_global-110}, ${height-450})`)
  .attr("font-size", "24px")
  .attr("text-anchor", "middle")
  .attr("font-weight", 600)
  .text(name)

  // small - high
  svg.append("text")
  .attr("transform", `translate(${x_global+30}, ${height-450})`)
  .attr("font-size", "15px")
  .attr("text-anchor", "middle")
  .attr("font-weight", 600)
  .text("Highest")

  svg.append("text")
  .attr("transform", `translate(${x_global-25}, ${height-30})`)
  .attr("font-size", "15px")
  .attr("text-anchor", "middle")
  .attr("font-weight", 600)
  .text("Lowest")

  // Y Scale
  var y = d3.scaleLinear()
    .domain([d3.min(data, d => d.Amount),
             d3.max(data, d => d.Amount)])          
    .range([height/1.2, 0])
  svg.append("g")
  .attr("transform", `translate(${x_global}, ${height-450})`)
  .call(d3.axisLeft(y).ticks(7))
  .selectAll("text")
      .attr("font-size", "13px")
      .attr("font-weight", 300)

  // X Scale
  var x = d3.scaleBand()
    .range([ width/1.5, 0 ])
    .domain(data.map(d => d.Year))
    .padding(0.05)     
  svg.append("g")
  .attr("transform", `translate(${x_global}, ${height-35})`)
    .call(d3.axisBottom(x))
    .selectAll("text")
      .attr("font-size", "15px")
      .attr("font-weight", 300)

  // Violin (set thresh)
  var histogram = d3.histogram()
        .domain(y.domain())
        .thresholds(y.ticks(20))
        .value(d => d)

  // Compute the binning for each group of the dataset
  var sumstat = d3.nest()  
    .key(function(d) { return d.Year;})
    .rollup(function(d) { 
      var input = d.map(function(g) { return g.Amount;}) 
      var bins = histogram(input)
      return(bins)
    })
    .entries(data)

  // Biggest value in a bin
  var maxNum = 0
  for ( var i in sumstat ){
    var allBins = sumstat[i].value
    var lengths = allBins.map(function(a){return a.length;})
    var longuest = d3.max(lengths)
    if (longuest > maxNum) { maxNum = longuest }
  }

  // Violin Max Length
  var xNum = d3.scaleLinear()
    .range([0, x.bandwidth()])
    .domain([-maxNum,maxNum])

  // Gradient
  const defs = svg.append('defs');

  const bgGradient = defs
    .append('linearGradient')
    .attr('id', grad_name)
    .attr('gradientTransform', 'rotate(90)');
  bgGradient
    .append('stop')
    .attr('stop-color', color1)
    .attr('offset', '0%');
  bgGradient
    .append('stop')
    .attr('stop-color', color2)
    .attr('offset', '100%');

  // Add the shape to this svg!
  svg
    .selectAll("myViolin")
    .data(sumstat)
    .enter()      
    .append("g")
      .attr("transform", function(d){ return("translate(" + x(d.key) +" ,0)") } )
    .append("path")
    .attr("transform", `translate(${x_global}, ${height-450})`)
        .datum(function(d){ return(d.value)}) 
        .style("stroke", `url(#${grad_name})`)
        .style("stroke-width", 12)
        .style("fill", `url(#${grad_name})`)
        .attr("d", d3.area()
            .x0(function(d){ return(xNum(-d.length)) } )
            .x1(function(d){ return(xNum(d.length)) } )
            .y(function(d){ return(y(d.x0)) } )
            .curve(d3.curveCatmullRom)
        )

  // === POINTS ===
  
  var x2 = d3.scaleTime()
      .domain(d3.extent(data2, function(d) { return d.Year; }))
      .range([ 0, width/1.5 ]);
    svg.append("g")
      .attr("transform", `translate(${x_global}, ${height-35})`)
      .call(d3.axisBottom(x2).tickValues([]));

  // Add Y axis
  var y2 = d3.scaleLinear()
    .domain([d3.min(data2, d => d.Amount),
      d3.max(data2, d => d.Amount)])
    .range([ height/1.2, 0 ]);
  svg.append("g")
    .attr("class", "axisHidden")
    .attr("transform", `translate(${x_global+70}, ${height-450})`)
    .call(d3.axisLeft(y2).tickValues([]));

  // Add the line
  svg.append("path")
    .datum(data2)
    .attr("fill", "none")
    .attr("stroke", "#a46400")
    .attr("stroke-width", 2)
    .style("stroke-dasharray", ("10, 5"))
    .attr("transform", `translate(${x_global+70}, ${height-450})`)
    .attr("d", d3.line()
      .x(function(d) { return x(d.Year) })
      .y(function(d) { return y(d.Amount) })
      )

  // Add the points
  svg
    .append("g")
    .selectAll("dot")
    .data(data2)
    .enter()
    .append("circle")
    .attr("transform", `translate(${x_global+70}, ${height-450})`)
      .attr("cx", function(d) { return x(d.Year) } )
      .attr("cy", function(d) { return y(d.Amount) } )
      .attr("r", 10)
      .attr("stroke", "black")
      .attr("stroke-width", 0.5)
      .attr("fill", "#Deaf5a")
}


// ===== MEN =====
const svg1 = d3.select("#map20-gents").append("svg")
// .attr("style", "outline: thin solid red;")
  .attr("preserveAspectRatio", "xMinYMin meet")
  .attr("viewBox", "0 0 950 500")
  .classed("svg-content", true)
  .attr("transform",
   `translate(${margin.LEFT}, ${margin.TOP})`)

// Gentleman
svg1.append("image")
.attr("xlink:href", baseAssetsUrl+"man20.png")
    .attr("x", "-35%")
    .attr("y", "-10%")
    .attr("width", 540)
    .attr("height", 540)
    .style("opacity", 1)


// Comment
const annot1 = [
  {
  note: { 
    label: "There has been a decrease in higher salaries (amounts) in 2021 vs 2018 (seen in both genders).",
    title: "2021 vs 2018",
    wrap: 100, 
    padding: 5, 
  },
  className: "gents",
  connector: { end: "arrow" },
  color: ["#ffffff"],
  x: 720,
  y: 100,
  dy: +20,
  dx: +60
}]

svg1.append("g")
  .style('font-size', 17)
  .call(d3.annotation()
          .annotations(annot1))

d3.csv(baseAssetsUrl + "salary_man.csv", function(data){

  const data2 = [{"Year": 2018, "Amount":	7549},
  {"Year": 2019, "Amount":	8349},
  {"Year": 2020, "Amount":	7281},
  {"Year": 2021, "Amount":	6999}]

   const name = "Gentlemen"
   const color1 = "#657d83"
   const color2 = "#33555e"
   const grad_name = 'bg-gradient20'

   create_violin(data, svg1, name, color1, color2, grad_name,
     data2)
  
});


// ===== WOMEN =====
const svg2 = d3.select("#map20-ladies").append("svg")
// .attr("style", "outline: thin solid red;")
.attr("preserveAspectRatio", "xMinYMin meet")
.attr("viewBox", "0 0 950 500")
.classed("svg-content", true)
.attr("transform",
  `translate(${margin.LEFT}, ${margin.TOP})`)

// Lady
svg2.append("image")
.attr("xlink:href", baseAssetsUrl+"lady20.png")
.attr("x", "-23%")
.attr("y", "4%")
.attr("width", 490)
.attr("height", 490)
.style("opacity", 1)

// Comment
const annot2 = [
  {
  note: { 
    label: "Both genders have in 2021 vs 2018 a bigger pool of respondents with much lower income (<1k units).",
    title: "Low income",
    wrap: 100, 
    padding: 5, 
  },
  className: "ladies",
  connector: { end: "arrow" },
  color: ["#ffffff"],
  x: 720,
  y: 415,
  dy: -20,
  dx: +60
}]

svg2.append("g")
  .style('font-size', 17)
  .call(d3.annotation()
          .annotations(annot2))


d3.csv(baseAssetsUrl + "salary_woman.csv", function(data){

  const data2 = [{"Year": 2018, "Amount":	6572},
  {"Year": 2019, "Amount":	6168},
  {"Year": 2020, "Amount":	4867},
  {"Year": 2021, "Amount":	4808}]

  const name = "Ladies"
  const color1 = "#9C4254"
  const color2 = "#B5838d"
  const grad_name = 'bg-gradient21'

  create_violin(data, svg2, name, color2, color1, grad_name,
      data2)

});

// Comment
const annot3 = [
  {
  note: { 
    label: "",
    title: "Average Income",
    wrap: 100, 
    padding: 2, 
  },
  className: "neutral",
  connector: { end: "arrow" },
  color: ["#ffffff"],
  x: 420,
  y: 185,
  dy: -15,
  dx: +15
}]

svg2.append("g")
  .style('font-size', 13)
  .call(d3.annotation()
          .annotations(annot3))

});
'''


h = display(HTML(htmlt))
j = py_display.Javascript(js_t)
py_display.display_javascript(j)

<h3 style="font-family: times-new-roman; font-size: 1.5em">Curiosity Bonus: Salaries & Roles</h3>

<p style="font-family: times-new-roman">I know pay is a taboo subject, but I could not help myself to also look within the top 6 most frequent Roles and what is the average salary for each of them. I have come up with an extremely informative graph:</p>

* <p style="font-family: times-new-roman">For 🎩gents, the highest paying jobs are <b>Project Manager</b> (1st place) and <b>Data Scientist</b> (2nd place). Coming next, the 3rd best-paying salary is for <b>Software Engineering</b> - and we know men love this job, as it is also the 3rd most frequent role found within our respondents.</p>
* <p style="font-family: times-new-roman">For 💃ladies, the first 2 most high-paying roles are <b>Project Manager</b> (1st place) and <b>Data Scientist</b> (2nd place) as well! What I find the most interesting is that for them, on the 3rd place on the list of the best paying jobs is <b>Business Analyst</b> (very close to Data Analyst I might add), which are also extremely popular roles for women (as we saw in the chart in Chapter 2.1).</p>
* <p style="font-family: times-new-roman">What I find interesting is that <b>each gender has in the top 4 best paying jobs roles that they are really good at</b>. For women there is Data Science, Data Analysis and Business Analysis, while for men is Data Science, Software Engineering and Research.</p>

In [ ]:
# Combine salary with job title
cols = ["Year", "What is your gender? - Selected Choice",
        "In which country do you currently reside?",
        "Select the title most similar to your current role (or most recent title if retired): - Selected Choice",
       df.columns[120]]

salary2 = df[cols]
salary2.columns = ["Year", "Gender", "Country", "Role", "Salary"]
salary2 = salary2[salary2["Gender"].isin(["Man", "Woman"])]

old_list = ['25,000-29,999', '60,000-69,999', '$0-999', '30,000-39,999',
       '15,000-19,999', '70,000-79,999', '2,000-2,999', '10,000-14,999',
       '5,000-7,499', '20,000-24,999', '1,000-1,999', '100,000-124,999',
       '7,500-9,999', '4,000-4,999', '40,000-49,999', '50,000-59,999',
       '3,000-3,999', '300,000-499,999', '200,000-249,999',
       '125,000-149,999', '250,000-299,999', '80,000-89,999',
       '90,000-99,999', '150,000-199,999', '>$1,000,000',
       '$500,000-999,999', '> $500,000', '300,000-500,000', '10-20,000',
       '0-10,000',
       'I do not wish to disclose my approximate yearly compensation',
       '20-30,000', '125-150,000', '30-40,000', '50-60,000',
       '100-125,000', '90-100,000', '70-80,000', '80-90,000', '60-70,000',
       '400-500,000', '40-50,000', '150-200,000', '300-400,000',
       '200-250,000', '500,000+', '250-300,000']
new_list = ['25,000-29,999', '60,000-69,999', '0-999', '30,000-39,999',
       '15,000-19,999', '70,000-79,999', '2,000-2,999', '10,000-14,999',
       '5,000-7,499', '20,000-24,999', '1,000-1,999', '100,000-124,999',
       '7,500-9,999', '4,000-4,999', '40,000-49,999', '50,000-59,999',
       '3,000-3,999', '300,000-499,999', '200,000-249,999',
       '125,000-149,999', '250,000-299,999', '80,000-89,999',
       '90,000-99,999', '150,000-199,999', '500,000',
       '500,000', '500,000', '300,000-499,999', '10,000-19,999',
       '0-9,999',
       'Undisclosed',
       '20,000-29,999', '125,000-149,999', '30,000-39,999', '50,000-59,999',
       '100,000-124,999', '90,000-99,999', '70,000-79,999', '80,000-89,999', '60,000-69,999',
       '400,000-499,999', '40,000-49,999', '150,000-199,999', '300,000-399,999',
       '200,000-249,999', '500,000', '250-299,999']

salary2["Salary"] = salary2["Salary"].replace(old_list, new_list)
salary2 = salary2[salary2["Salary"]!="Undisclosed"].dropna().reset_index(drop=True)

old_role = ['Currently not employed', 'Product Manager',
            'DBA/Database Engineer', 'Product/Project Manager',
            'Not employed', 'Manager', 
            'Software Developer/Software Engineer', 'Scientist/Researcher',
            'Research Assistant', 'Predictive Modeler', 'Data Miner',
            'Programmer', 'Operations Research Practitioner', 'Developer Advocate',
            'Machine Learning Engineer', 'Researcher', 'Program/Project Manager']
new_role = ['Unemployed', 'Project Manager',
            'DBA/Database Engineer', 'Project Manager',
            'Unemployed', 'Project Manager',
            'Software Engineer', 'Research Scientist',
            'Research Scientist', 'Data Scientist', 'Data Engineer',
            'Computer Scientist', 'Research Scientist', 'Software Engineer',
            'Data Scientist', 'Research Scientist', 'Project Manager']
salary2["Role"] = salary2["Role"].replace(old_role, new_role)

# Adjust Salary
salary2 = adjust_amount(data=salary2, amount_column="Salary")

keep = ["Data Scientist", "Data Analyst", "Software Engineer", 
        "Research Scientist", "Business Analyst", "Project Manager"]
salary2 = salary2.groupby(["Year", "Gender", "Role"])["Amount"]\
            .mean().reset_index()
salary2 = salary2[salary2["Role"].isin(keep)]
salary2["Amount"] = salary2["Amount"].apply(lambda x: round(x))

salary2 = salary2.pivot(index=["Year", "Gender"], 
                        columns="Role", values="Amount").reset_index()
salary2.columns = ["Year", "Gender", "BusinessAnalyst", "DataAnalyst",
                   "DataScientist", "ProjectManager", "ResearchScientist",
                   "SoftwareEngineer"]

# Sepparate genders
salary2_man = salary2[salary2["Gender"]=="Man"].drop(columns="Gender")
salary2_man = salary2_man.sort_values(salary2_man.last_valid_index(),
                                      axis=1, ascending=False)
cols = list(salary2_man.columns)
cols = [cols[-1]] + cols[:-1]
salary2_man = salary2_man[cols]


salary2_woman = salary2[salary2["Gender"]=="Woman"].drop(columns="Gender")
salary2_woman = salary2_woman.sort_values(salary2_woman.last_valid_index(), axis=1,
                    ascending=False)
cols = list(salary2_woman.columns)
cols = [cols[-1]] + cols[:-1]
salary2_woman = salary2_woman[cols]

salary2_man.to_csv("salary2_man.csv", index=False)
salary2_woman.to_csv("salary2_woman.csv", index=False)

# Save usa.png img as well
im = imageio.imread('../input/kaggle-data-science-survey-20172021/images/man22.png')
Image.fromarray(im).save("man22.png")

im = imageio.imread('../input/kaggle-data-science-survey-20172021/images/lady22.png')
Image.fromarray(im).save("lady22.png")

htmlt = '''
<!-- Bootstrap -->
<link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.4.1/css/bootstrap.min.css" integrity="sha384-Vkoo8x4CGsO3+Hhxv8T/Q5PaXtkKtu6ug5TOeNV6gBiFeWPGFN9MuhOf23Q9Ifjh" crossorigin="anonymous">
<!-- Style -->
<style>
	.all22 {
		font-family: "Times New Roman", Times, serif;
		overflow-x: hidden;
		width: 98%;
		background-color:#e0c9a6;
		/* background-image : url("bkg.jpg"); changed via js */
	}
	.all22 .svg-container {
		display: inline-block;
		position: relative;
		width: 100%;
		padding-bottom: 80%;
		vertical-align: top;
		overflow: hidden;
	}
	.svg-content {
		display: inline-block;
		position: absolute;
		top: 0;
		left: 0;
	}

	.axisHidden path{
		stroke: #e0c9a6;
	}

	.axisHidden line{
		stroke: #e0c9a6;
	}

	.annotation.color text {
		fill: #000000;
	}

	@font-face {
		font-family: parklane;
		src: url(https://h4ks.net/ParkLaneNF.otf);
		}

	div.tooltip-donut {
		position: absolute;
		text-align: center;
		padding: .3rem;
		background: #FFFFFF;
		color: #000000;
		border: 1px solid #000000;
		border-radius: 8px;
		pointer-events: none;
		font-size: 2rem;
		z-index: 99999;
	}

</style>

<img id="baseimg" src="man.png" style="display:none" />
<div class="all22">
	<center><h1 style="font-family: parklane">Bonus: Average Salaries on Role & Gender</h1></center>
	<center><h3><i>- Salary was adjusted on respondent's rezidence -</i></h3></center>
	<div class="row">
		<div id="map22" class="svg-container"></div>
	</div>
</div>
'''

js_t = '''
require.config({
  paths: {
    d3src: "https://d3js.org/",
  },
  map: {
    '*': {
      'd3v6': 'd3src/d3.v6.min',
      'd3-selection': 'd3src/d3-selection.v1.min',
      'd3-drag': 'd3src/d3-drag.v1.min',
      'd3-shape': 'd3src/d3-shape.v1.min',
      'd3-path': 'd3src/d3-path.v1.min',
      'd3-dispatch': 'd3src/d3-dispatch.v1.min',
      'd3-annotation': 'https://cdnjs.cloudflare.com/ajax/libs/d3-annotation/2.5.1/d3-annotation.min.js',
    }
  }
});

require(["d3v6", "d3-annotation"], function(d3, d3Annotation) {

// Inject d3Annotation methods to d3
for (var key in d3Annotation) {
  d3[key] = d3Annotation[key];
}

// Get Assets URL set by Kaggle
const baseAssetsUrl = document.getElementById('baseimg').src.replace(/man.png.*$/, '');

const $all22 = document.getElementsByClassName('all22')[0];
$all22.style.backgroundImage = `url("${baseAssetsUrl}bkg.jpg")`;


// Set the SVG area
const margin = { LEFT: 20, RIGHT: 40, TOP: 15, BOTTOM: 0 }
const width = 950 - margin.LEFT - margin.RIGHT
const height = 700 - margin.TOP - margin.BOTTOM
const centreSpacing = 100

const svg = d3.select("#map22").append("svg")
// .attr("style", "outline: thin solid red;")
  .attr("preserveAspectRatio", "xMinYMin meet")
  .attr("viewBox", "0 0 950 700")
  .classed("svg-content", true)
  .attr("transform",
   `translate(${margin.LEFT}, ${margin.TOP})`)

// Gentleman
svg.append("image")
.attr("xlink:href", baseAssetsUrl+"man22.png")
    .attr("x", "21%")
    .attr("y", "8.5%")
    .attr("width", 230)
    .attr("height", 230)
    .style("opacity", 1)

// Lady
svg.append("image")
.attr("xlink:href", baseAssetsUrl+"lady22.png")
.attr("x", "65%")
.attr("y", "18%")
.attr("width", 260)
.attr("height", 260)
.style("opacity", 1)


function create_stacked_bar(data, subgroups, x_global, y_range, color) {

  const groups = data.map(d => d.Year)

  // Add X axis
  const x = d3.scaleBand()
      .domain(groups)
      .range([0, width/2.5])
      .padding([0.2])
  svg.append("g")
    .attr("transform", `translate(${x_global}, ${height/1.15})`)
    .call(d3.axisBottom(x).tickSizeOuter(0))
    .selectAll("text")
      .attr("y", "10")
      .attr("x", "0")
      .attr("font-size", "15px")
      .attr("font-weight", 300)
      .attr("text-anchor", "middle")

  // Add Y axis
  const y = d3.scaleLinear()
    .domain([0, y_range])
    .range([ height/2.3, 0]);
  svg.append("g")
  .attr("class", "axisHidden")
  .attr("transform", `translate(${x_global}, ${height/2-50})`)
    .call(d3.axisLeft(y).ticks(0))
    .selectAll("text")
      .attr("font-size", "11px")
      .attr("font-weight", 300);

  //s Stack data
  const stackedData = d3.stack()
    .keys(subgroups)
    (data)

  // Show the bars
  const bars = svg.append("g")
    .selectAll("g")
    .data(stackedData)
    .join("g")
    .attr("transform", `translate(${x_global}, ${height/2-50})`)
      .attr("fill", d => color(d.key))
      .attr("stroke", "black")
      .attr("stroke-width", 0.2)
      .attr("class", d => "myRect " + d.key ) 
      .selectAll("rect")
      .data(d => d)
      .join("rect")
        .attr("x", d => x(d.data.Year))
        .attr("y", d => y(d[1]))
        .attr("height", d => y(d[0]) - y(d[1]))
        .attr("width",x.bandwidth())

  // Interactive
  var div = d3.select("body").append("div")
  .attr("class", "tooltip-donut")
  .style("opacity", 0);

  bars
    .on('mouseenter', function (event, dt) {
    // MOUSE ON
    const subGroupName = d3.select(this.parentNode).datum().key 
          
    d3.selectAll(".myRect").style("opacity", 0.2)  
          
    d3.selectAll("."+subGroupName).style("opacity",1)


    // Makes the new div appear
    div.transition()
        .duration(50)
        .style("opacity", 1);

    const subgroupName = d3.select(this.parentNode).datum().key;
    const subgroupValue = dt.data[subgroupName];

    div.html(subgroupName + "<br>" + "Avg: " + subgroupValue)
        .style("left", (event.pageX) + "px")
        .style("top", (event.pageY-30) + "px");


  })
    // MOUSE LEAVE
    .on('mouseleave', function () {
      d3.selectAll(".myRect")
          .style("opacity",1) 

      // New div dissapears
      div.transition()
          .duration('50')
          .style("opacity", 0);
      svg.selectAll('.divergence').remove()
    })

}


// === Preferences ===
d3.csv(baseAssetsUrl + "salary2_man.csv").then(function(data){

  const x_global = 50
  const y_range = 40000
  const subgroups = ['ProjectManager', 'DataScientist', 
  'SoftwareEngineer', 'ResearchScientist', 'BusinessAnalyst',
   'DataAnalyst']
  const color = d3.scaleOrdinal()
    .domain(subgroups)
    .range(["#6AA07B", "#1198A7", "#5691C8",
    "#226BBF", "#184E8B", "#593d96"]);

  svg.append("text")
  .attr("y", "5%")
  .attr("x", "25%")
  .attr("font-size", "25px")
  .attr("text-anchor", "middle")
  .attr("font-weight", 600)
  .text("Gentlemen: Avg. Amount on Roles")

  create_stacked_bar(data, subgroups, x_global, y_range, color)
  
});

d3.csv(baseAssetsUrl + "salary2_woman.csv").then(function(data){

  const x_global = 500
  const y_range = 40000
  const subgroups = ['ProjectManager', 'DataScientist', 
  'BusinessAnalyst', 'DataAnalyst', 'SoftwareEngineer',
   'ResearchScientist']
  const color = d3.scaleOrdinal()
    .domain(subgroups)
    .range(["#6AA07B", "#1198A7", "#184E8B",
    "#593d96", "#5691C8", "#226BBF"]);

  svg.append("text")
  .attr("y", "5%")
  .attr("x", "70%")
  .attr("font-size", "25px")
  .attr("text-anchor", "middle")
  .attr("font-weight", 600)
  .text("Ladies: Avg. Amount on Roles")

  create_stacked_bar(data, subgroups, x_global, y_range, color)
  
});

const annot2 = [
  {
  note: { 
    label: "Best paid salaries are PM and DS, followed by Software Eng. (which decreased in avg since 2018).",
    title: "Gents",
    wrap: 400, 
    padding: 5, 
  },
  className: "color",
  // connector: { end: "arrow" },
  color: ["#ffffff"],
  x: 50,
  y: 630,
  dy: +0,
  dx: +0
},
{
  note: { 
    label: "Top 2 coincide with gents (PM & DS), however the 3rd best paid is Business Analyst.",
    title: "Ladies",
    wrap: 400, 
    padding: 5, 
  },
  className: "color",
  // connector: { end: "arrow" },
  color: ["#ffffff"],
  x: 500,
  y: 630,
  dy: +0,
  dx: +0
}]

svg.append("g")
  .style('font-size', 18)
  .call(d3.annotation()
          .annotations(annot2))


});
'''


h = display(HTML(htmlt))
j = py_display.Javascript(js_t)
py_display.display_javascript(j)

<h3 style="font-family: times-new-roman; font-size: 1.5em">II. The (Machine Learning) Investment</h3>

<p style="font-family: times-new-roman">Are companies interested in ML? How much are they willing to pay for it?</p>

<p style="font-family: times-new-roman">Something that I found rather odd is that there is some sort of <b>decrease in Machine Learning interest from companies</b>, at least from the respondent's perspective. I would have actually expected an increase in companies that have started using ML or have been implementing it for a while. However, not only the percentage of interest decrease but also <b>the overall number of respondents that answered this question decreased too</b>. I would put a part of this decrease on the lack of administrative knowledge from respondents (if they don't know for sure they might choose to hold back from answering).</p>

In [ ]:
cols = ["Year", "What is your gender? - Selected Choice",
       df.columns[111]]

incorp = df[cols]
incorp.columns = ["Year", "Gender", "Incorporate_ML"]
incorp = incorp[incorp["Gender"].isin(["Man", "Woman"])]

old_list = ['No (we do not use ML methods)',
       'We are exploring ML methods (and may one day put a model into production)',
       'I do not know',
       'We use ML methods for generating insights (but do not put working models into production)',
       'We have well established ML methods (i.e., models in production for more than 2 years)',
       'We recently started using ML methods (i.e., models in production for less than 2 years)']
new_list = ['2. Never',
       '3. Exploring Possibility',
       '1. No Idea',
       '5. Somehow Yes',
       '6. Definitely Yes',
       '4. Recently']

incorp["Incorporate_ML"] = incorp["Incorporate_ML"].replace(old_list, new_list)
incorp = incorp.value_counts().reset_index()
incorp = pd.merge(incorp, total, on=["Year", "Gender"])

incorp["Perc"] = incorp[0]/incorp["Count"]*100
incorp["Perc"] = incorp["Perc"].apply(lambda x: round(x, 1))
incorp.drop(columns=[0, "Count"], axis=1, inplace=True)
incorp = pd.pivot(incorp, index=["Year", "Gender"], columns="Incorporate_ML",
                   values="Perc").reset_index()

incorp_man = incorp[incorp["Gender"] == "Man"].drop(columns=["Gender"])
incorp_woman = incorp[incorp["Gender"] == "Woman"].drop(columns=["Gender"])

incorp_man.to_csv("incorp_man.csv", index=False)
incorp_woman.to_csv("incorp_woman.csv", index=False)

# Save usa.png img as well
im = imageio.imread('../input/kaggle-data-science-survey-20172021/images/man23.png')
Image.fromarray(im).save("man23.png")

im = imageio.imread('../input/kaggle-data-science-survey-20172021/images/lady23.png')
Image.fromarray(im).save("lady23.png")

htmlt = '''
<!-- Bootstrap -->
<link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.4.1/css/bootstrap.min.css" integrity="sha384-Vkoo8x4CGsO3+Hhxv8T/Q5PaXtkKtu6ug5TOeNV6gBiFeWPGFN9MuhOf23Q9Ifjh" crossorigin="anonymous">
<!-- Style -->
<style>
	.all23 {
		font-family: "Times New Roman", Times, serif;
		overflow-x: hidden;
		width: 98%;
		background-color:#e0c9a6;
		/* background-image : url("bkg.jpg"); changed via js */
	}
	.all23 .svg-container {
		display: inline-block;
		position: relative;
		width: 100%;
		padding-bottom: 54%;
		vertical-align: top;
		overflow: hidden;
	}
	.svg-content {
		display: inline-block;
		position: absolute;
		top: 0;
		left: 0;
	}

	.axisHidden path{
		stroke: #e0c9a6;
	}

	.axisHidden line{
		stroke: #e0c9a6;
	}

	.annotation.color text {
		fill: #000000;
	}

	@font-face {
		font-family: parklane;
		src: url(https://h4ks.net/ParkLaneNF.otf);
		}

	div.tooltip-donut {
		position: absolute;
		text-align: center;
		padding: .3rem;
		background: #FFFFFF;
		color: #000000;
		border: 1px solid #000000;
		border-radius: 8px;
		pointer-events: none;
		font-size: 2rem;
		z-index: 99999;
	}

</style>

<img id="baseimg" src="man.png" style="display:none" />
<div class="all23">
	<center><h1 style="font-family: parklane">Is your company using ML Techniques?</h1></center>
	<center><h3><i>- Are companies incorporating more ML into their business? -</i></h3></center>
	<div class="row">
		<div id="map23-gents" class="svg-container"></div>
	</div>

	<div class="row">
		<div id="map23-ladies" class="svg-container"></div>
	</div>
</div>
'''

js_t = '''
require.config({
  paths: {
    d3src: "https://d3js.org/",
  },
  map: {
    '*': {
      'd3v6': 'd3src/d3.v6.min',
      'd3-selection': 'd3src/d3-selection.v1.min',
      'd3-drag': 'd3src/d3-drag.v1.min',
      'd3-shape': 'd3src/d3-shape.v1.min',
      'd3-path': 'd3src/d3-path.v1.min',
      'd3-dispatch': 'd3src/d3-dispatch.v1.min',
      'd3-annotation': 'https://cdnjs.cloudflare.com/ajax/libs/d3-annotation/2.5.1/d3-annotation.min.js',
    }
  }
});

require(["d3v6", "d3-annotation"], function(d3, d3Annotation) {

// Inject d3Annotation methods to d3
for (var key in d3Annotation) {
  d3[key] = d3Annotation[key];
}

// Get Assets URL set by Kaggle
const baseAssetsUrl = document.getElementById('baseimg').src.replace(/man.png.*$/, '');

const $all23 = document.getElementsByClassName('all23')[0];
$all23.style.backgroundImage = `url("${baseAssetsUrl}bkg.jpg")`;

// Set the SVG area
const margin = { LEFT: 20, RIGHT: 40, TOP: 15, BOTTOM: 0 }
const width = 950 - margin.LEFT - margin.RIGHT
const height = 500 - margin.TOP - margin.BOTTOM
const chart_size = 50
const x_global = 150

function create_graph(data, svg, name){

  const XY_axis = height/2-200

  svg.append("text")
  .attr("transform", `translate(${x_global-60}, ${XY_axis-10})`)
  .attr("font-size", "24px")
  .attr("text-anchor", "middle")
  .attr("font-weight", 600)
  .text(name)

  data.forEach(d => {
    d.Year = Number(d.Year)
  })

  // List of groups
  var keys = data.columns.slice(1)

  // Stacking
  var series = d3.stack()
    .keys(keys)
    .offset(d3.stackOffsetSilhouette)
    (data)

  // X Axis
  var x = d3.scaleLinear()
    .domain(d3.extent(data, d => d.Year))
    .range([0, width/1.5])

  var xAxis = g => g
    .attr("transform", `translate(${x_global}, ${XY_axis})`)
    .call(d3.axisBottom(x)
            .tickSize(height/1.2).ticks(4).tickFormat(d3.format("d")))
    .call(g => g.select(".domain").remove())
    .call(g => g.selectAll(".tick line")
                .attr("stroke", "#3A3A3A")
                .attr('stroke-dasharray', '5 5'))
    .call(g => g.selectAll(".tick")
                .select('text')
                .attr('fill', "#3A3A3A")
                .style("font-size", 15))

  // Y Axis
  var y = d3.scaleLinear()
  .domain([-chart_size, chart_size])
  .range([height/1.2, 0])
  svg.append("g")
  .attr("class", "axisHidden")
  .attr("transform", `translate(${x_global}, ${XY_axis})`)
    .call(d3.axisLeft(y).ticks(0))

  // Area Chart
  var area = d3.area()
  .x(d => x(d.data.Year))
  .y0(d => y(d[0]))
  .y1(d => y(d[1]))

  // Color
  const color = d3.scaleOrdinal()
    .domain(keys)
     .range(["#6AA07B", "#12A6B6", "#4687C3",
     "#1F61AD", "#AB3680", "#8C2C69"])

  // Show
  const path = svg.append("g")
    .selectAll("path")
    .data(series)
    .join("path")
    .attr("transform", `translate(${x_global}, ${XY_axis})`)
      .attr("data-label", d => d.key)
      .attr("fill", ({key}) => color(key))
      .attr("stroke", "black")
      .attr("stroke-width", 0.2)
      .attr("d", area)
      .attr("opacity", 0.8)

  svg.append("g")
      .call(xAxis)

  // ==== Interactive ====
  const hover = (svg, path) => {
  
    const line = svg.append("g")
         .attr("display", "none");
  
    line.append("g")
        .selectAll("line")
        .data(series)
        .join("line")
        .attr("class", "cursor-line")
        .attr("fill", "#fff")
        .attr("stroke-width", 0.8)
        .attr("x1", 10)
        .attr("y1", height)
        .attr("x2", 10)
        .attr("y2", 10);

    line.append("text")
      .attr("class", "text-year")
      .attr("font-size", 14)
      .attr("x", 0)
      .attr("y", 26)
      .attr("transform", "rotate(-90 20 20) translate(0, -20)");
    
    line.append("g")
        .selectAll("text")
        .data(series)
        .join("text")
        .attr("class", "text-label")
        .attr("font-weight", 900)
        .attr("font-size", 17)
        .attr("y", height-10)
        .attr("x", -8)
        .attr("transform", 
        "rotate(-90 10 460)");

    line.append("g")
        .selectAll("text")
        .data(data)
        .join("text")
        .attr("class", "text-value")
        .attr("font-weight", 700)
        .attr("font-size", 14)
        .attr("y", height-(height*0.94))
        .attr("x", 14);

    const mousemove = (event) => {
  
      event?.preventDefault();
      const pointer = d3.pointer(event);

      const xm = x.invert(pointer[0]);
      const label = d3.select(event.target).attr("data-label");
  
      line.attr("transform", `translate(${pointer[0]}, 0)`)
          .style("visibility", "inherit");
    
          line.select(".text-year")
          .text(parseInt(xm));
      
      line.selectAll(".text-value")
        .text(d => d.Year === parseInt(xm) ? d[label]+"%" : "");

      line.selectAll(".text-label")
        .style("visibility", "hidden")
        .filter(d => d.key === label)
        .style("visibility", "inherit")
        .attr("fill", d => d.key === label ? color(d.key) : "#000")
        .text(label);
      
      line.selectAll(".cursor-line")
        .attr("stroke", "fff0")
        .filter(d => d.key === label)
        .attr("stroke", d => d.key === label ? color(d.key) : "#fff0")
      
      path.attr("opacity", d => d.key === label ? 0.8 : 0.3);
      
      if (label === null) {
        path.attr("opacity", 0.8);
        line.style("visibility", "hidden");
      }
    }

    const mouseenter = () => {
      line.attr("display", null);
    }
  
    const mouseleave = () => {
      line.attr("display", "none");
    }

    svg
        .on("mousemove", mousemove)
        .on("mouseenter", mouseenter)
        .on("mouseleave", mouseleave)

  }

  svg.call(hover, path);
}

// ===== MEN =====
const svg1 = d3.select("#map23-gents").append("svg")
// .attr("style", "outline: thin solid red;")
  .attr("preserveAspectRatio", "xMinYMin meet")
  .attr("viewBox", "0 0 950 500")
  .classed("svg-content", true)
  .attr("transform",
   `translate(${margin.LEFT}, ${margin.TOP})`)

// Gentleman
svg1.append("image")
.attr("xlink:href", baseAssetsUrl+"man23.png")
    .attr("x", "-6%")
    .attr("y", "-11%")
    .attr("width", 530)
    .attr("height", 530)
    .style("opacity", 1)

// Comment
const annot1 = [
  {
  note: { 
    label: "The no. of respondents that say that their company is exploring the possibility/ have recently implemented ML has decreased since 2018.",
    title: "Decrease in ML Interest",
    wrap: 120, 
    padding: 5, 
  },
  className: "color",
  connector: { end: "arrow" },
  color: ["#ffffff"],
  x: 750,
  y: 210,
  dy: +20,
  dx: +25
}]

svg1.append("g")
  .style('font-size', 17)
  .call(d3.annotation()
          .annotations(annot1))

d3.csv(baseAssetsUrl + "incorp_man.csv").then(function(data){

  const name = "Gentlemen"
  create_graph(data, svg1, name)

});

// ===== WOMEN =====
const svg2 = d3.select("#map23-ladies").append("svg")
// .attr("style", "outline: thin solid red;")
.attr("preserveAspectRatio", "xMinYMin meet")
.attr("viewBox", "0 0 950 500")
.classed("svg-content", true)
.attr("transform",
  `translate(${margin.LEFT}, ${margin.TOP})`)

// Lady
svg2.append("image")
.attr("xlink:href", baseAssetsUrl+"lady23.png")
.attr("x", "-19%")
.attr("y", "10%")
.attr("width", 420)
.attr("height", 420)
.style("opacity", 1)

// Comment
const annot2 = [
  {
  note: { 
    label: "There is a much smaller percentage or people that answered this question in 2021 vs 2018 (for both genders).",
    title: "Overall decrease",
    wrap: 120, 
    padding: 5, 
  },
  className: "color",
  // connector: { end: "arrow" },
  color: ["#ffffff"],
  x: 770,
  y: 350,
  dy: -1,
  dx: +1
}]

svg2.append("g")
  .style('font-size', 17)
  .call(d3.annotation()
          .annotations(annot2))

d3.csv(baseAssetsUrl + "incorp_woman.csv").then(function(data){

  const name = "Ladies"
  create_graph(data, svg2, name)

});

});
'''


h = display(HTML(htmlt))
j = py_display.Javascript(js_t)
py_display.display_javascript(j)

<p style="font-family: times-new-roman"> The <b>investing activity is quite correlated to the interest in Machine Learning</b>. For both genders, we can see that <b>~30% of respondents say that their company either doesn't invest in ML at all, or it invests a very small amount of money</b> (less than 100 dollars yearly). On the other end of the spectrum, very few respondents (less than 5% for 💃ladies and less than 🎩10% for men) say that their company invests more than 100K dollars in ML.</p>

<p style="font-family: times-new-roman"> Again, I am reluctant to put this behavior on the lack of interest or lack of administrative knowledge. As we know from previous graphs, there were quite a few people that said they work in big companies, however the percentages don't match with the investment - meaning that <b>large companies don't seem to necessarily invest a lot in Machine Learning</b>.</p>

In [ ]:
cols = ["Year", "What is your gender? - Selected Choice",
       df.columns[121]]

investm = df[cols]
investm.columns = ["Year", "Gender", "Investment"]
investm = investm[investm["Gender"].isin(["Man", "Woman"])]

old_list = ['$100-$999', '$0 ($USD)', '$1000-$9,999', '$1-$99',
       '$100,000 or more ($USD)', '$10,000-$99,999', '$0 (USD)',
       '> $100,000 ($USD)']
new_list = ['3. $100-$999', '1. $0', '4. $1000-$9,999', '2. $1-$99',
       '6. > $100,000', '5. $10,000-$99,999', '1. $0',
       '6. > $100,000']

investm["Investment"] = investm["Investment"].replace(old_list, new_list)
investm = investm.value_counts().reset_index()
investm = pd.merge(investm, total, on=["Year", "Gender"])

investm["Perc"] = investm[0]/investm["Count"]*100
investm["Perc"] = investm["Perc"].apply(lambda x: round(x, 1))
investm.drop(columns=[0, "Count"], axis=1, inplace=True)
investm = pd.pivot(investm, index=["Year", "Gender"], columns="Investment",
                   values="Perc").reset_index()

investm_man = investm[investm["Gender"] == "Man"].drop(columns=["Gender"])
investm_woman = investm[investm["Gender"] == "Woman"].drop(columns=["Gender"])

investm_man.to_csv("investm_man.csv", index=False)
investm_woman.to_csv("investm_woman.csv", index=False)

# Save usa.png img as well
im = imageio.imread('../input/kaggle-data-science-survey-20172021/images/man21.png')
Image.fromarray(im).save("man21.png")

im = imageio.imread('../input/kaggle-data-science-survey-20172021/images/lady21.png')
Image.fromarray(im).save("lady21.png")

htmlt = '''
<!-- Bootstrap -->
<link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.4.1/css/bootstrap.min.css" integrity="sha384-Vkoo8x4CGsO3+Hhxv8T/Q5PaXtkKtu6ug5TOeNV6gBiFeWPGFN9MuhOf23Q9Ifjh" crossorigin="anonymous">
<!-- Style -->
<style>
	.all21 {
		font-family: "Times New Roman", Times, serif;
		overflow-x: hidden;
		width: 98%;
		background-color:#e0c9a6;
		/* background-image : url("bkg.jpg"); changed via js */
	}
	.all21 .svg-container {
		display: inline-block;
		position: relative;
		width: 100%;
		padding-bottom: 54%;
		vertical-align: top;
		overflow: hidden;
	}
	.svg-content {
		display: inline-block;
		position: absolute;
		top: 0;
		left: 0;
	}

	.axisHidden path{
		stroke: #e0c9a6;
	}

	.axisHidden line{
		stroke: #e0c9a6;
	}

	.annotation.color text {
		fill: #000000;
	}

	@font-face {
		font-family: parklane;
		src: url(https://h4ks.net/ParkLaneNF.otf);
		}

	div.tooltip-donut {
		position: absolute;
		text-align: center;
		padding: .3rem;
		background: #FFFFFF;
		color: #000000;
		border: 1px solid #000000;
		border-radius: 8px;
		pointer-events: none;
		font-size: 2rem;
		z-index: 99999;
	}

</style>

<img id="baseimg" src="man.png" style="display:none" />
<div class="all21">
	<center><h1 style="font-family: parklane">Machine Learning Investment</h1></center>
	<center><h3><i>- How much do companies invest in Data Science? -</i></h3></center>
	<div class="row">
		<div id="map21-gents" class="svg-container"></div>
	</div>

	<div class="row">
		<div id="map21-ladies" class="svg-container"></div>
	</div>
</div>
'''

js_t = '''
require.config({
  paths: {
    d3src: "https://d3js.org/",
  },
  map: {
    '*': {
      'd3v6': 'd3src/d3.v6.min',
      'd3-selection': 'd3src/d3-selection.v1.min',
      'd3-drag': 'd3src/d3-drag.v1.min',
      'd3-shape': 'd3src/d3-shape.v1.min',
      'd3-path': 'd3src/d3-path.v1.min',
      'd3-dispatch': 'd3src/d3-dispatch.v1.min',
      'd3-annotation': 'https://cdnjs.cloudflare.com/ajax/libs/d3-annotation/2.5.1/d3-annotation.min.js',
    }
  }
});

require(["d3v6", "d3-annotation"], function(d3, d3Annotation) {

// Inject d3Annotation methods to d3
for (var key in d3Annotation) {
  d3[key] = d3Annotation[key];
}

// Get Assets URL set by Kaggle
const baseAssetsUrl = document.getElementById('baseimg').src.replace(/man.png.*$/, '');

const $all21 = document.getElementsByClassName('all21')[0];
$all21.style.backgroundImage = `url("${baseAssetsUrl}bkg.jpg")`;

// Set the SVG area
const margin = { LEFT: 20, RIGHT: 40, TOP: 15, BOTTOM: 0 }
const width = 950 - margin.LEFT - margin.RIGHT
const height = 500 - margin.TOP - margin.BOTTOM
const chart_size = 40
const x_global = 150

// STREAMGRAPH
function create_graph(data, svg, name){

  const XY_axis = height/2-200

  svg.append("text")
  .attr("transform", `translate(${x_global-60}, ${XY_axis-10})`)
  .attr("font-size", "24px")
  .attr("text-anchor", "middle")
  .attr("font-weight", 600)
  .text(name)

  data.forEach(d => {
    d.Year = Number(d.Year)
  })

  // List of groups
  var keys = data.columns.slice(1)

  // Stacking
  var series = d3.stack()
    .keys(keys)
    .offset(d3.stackOffsetSilhouette)
    (data)

  // X Axis
  var x = d3.scaleLinear()
    .domain(d3.extent(data, d => d.Year))
    .range([0, width/1.5])

  var xAxis = g => g
    .attr("transform", `translate(${x_global}, ${XY_axis})`)
    .call(d3.axisBottom(x)
            .tickSize(height/1.2).ticks(2).tickFormat(d3.format("d")))
    .call(g => g.select(".domain").remove())
    .call(g => g.selectAll(".tick line")
                .attr("stroke", "#3A3A3A")
                .attr('stroke-dasharray', '5 5'))
    .call(g => g.selectAll(".tick")
                .select('text')
                .attr('fill', "#3A3A3A")
                .style("font-size", 15))

  // Y Axis
  var y = d3.scaleLinear()
  .domain([-chart_size, chart_size])
  .range([height/1.2, 0])
  svg.append("g")
  .attr("class", "axisHidden")
  .attr("transform", `translate(${x_global}, ${XY_axis})`)
    .call(d3.axisLeft(y).ticks(0))

  // Area Chart
  var area = d3.area()
  .x(d => x(d.data.Year))
  .y0(d => y(d[0]))
  .y1(d => y(d[1]))

  // Color
  const color = d3.scaleOrdinal()
    .domain(keys)
     .range(["#DFAC59", "#79A988", "#12A6B6","#5993C9",
     "#236EC5", "#9C3175"])

  // Show
  const path = svg.append("g")
    .selectAll("path")
    .data(series)
    .join("path")
    .attr("transform", `translate(${x_global}, ${XY_axis})`)
      .attr("data-label", d => d.key)
      .attr("fill", ({key}) => color(key))
      .attr("stroke", "black")
      .attr("stroke-width", 0.2)
      .attr("d", area)
      .attr("opacity", 0.8)

  svg.append("g")
      .call(xAxis)

  // ==== Interactive ====
  const hover = (svg, path) => {
  
    const line = svg.append("g")
         .attr("display", "none");
  
    line.append("g")
        .selectAll("line")
        .data(series)
        .join("line")
        .attr("class", "cursor-line")
        .attr("fill", "#fff")
        .attr("stroke-width", 0.8)
        .attr("x1", 10)
        .attr("y1", height)
        .attr("x2", 10)
        .attr("y2", 10);

    line.append("text")
      .attr("class", "text-year")
      .attr("font-size", 14)
      .attr("x", 0)
      .attr("y", 26)
      .attr("transform", "rotate(-90 20 20) translate(0, -20)");
    
    line.append("g")
        .selectAll("text")
        .data(series)
        .join("text")
        .attr("class", "text-label")
        .attr("font-weight", 900)
        .attr("font-size", 17)
        .attr("y", height-10)
        .attr("x", -8)
        .attr("transform", 
        "rotate(-90 10 460)");

    line.append("g")
        .selectAll("text")
        .data(data)
        .join("text")
        .attr("class", "text-value")
        .attr("font-weight", 700)
        .attr("font-size", 14)
        .attr("y", height-(height*0.94))
        .attr("x", 14);

    const mousemove = (event) => {
  
      event?.preventDefault();
      const pointer = d3.pointer(event);

      const xm = x.invert(pointer[0]);
      const label = d3.select(event.target).attr("data-label");
  
      line.attr("transform", `translate(${pointer[0]}, 0)`)
          .style("visibility", "inherit");
    
          line.select(".text-year")
          .text(parseInt(xm));
      
      line.selectAll(".text-value")
        .text(d => d.Year === parseInt(xm) ? d[label]+"%" : "");

      line.selectAll(".text-label")
        .style("visibility", "hidden")
        .filter(d => d.key === label)
        .style("visibility", "inherit")
        .attr("fill", d => d.key === label ? color(d.key) : "#000")
        .text(label);
      
      line.selectAll(".cursor-line")
        .attr("stroke", "fff0")
        .filter(d => d.key === label)
        .attr("stroke", d => d.key === label ? color(d.key) : "#fff0")
      
      path.attr("opacity", d => d.key === label ? 0.8 : 0.3);
      
      if (label === null) {
        path.attr("opacity", 0.8);
        line.style("visibility", "hidden");
      }
    }

    const mouseenter = () => {
      line.attr("display", null);
    }
  
    const mouseleave = () => {
      line.attr("display", "none");
    }

    svg
        .on("mousemove", mousemove)
        .on("mouseenter", mouseenter)
        .on("mouseleave", mouseleave)

  }

  svg.call(hover, path);
}

// ===== MEN =====
const svg1 = d3.select("#map21-gents").append("svg")
// .attr("style", "outline: thin solid red;")
  .attr("preserveAspectRatio", "xMinYMin meet")
  .attr("viewBox", "0 0 950 500")
  .classed("svg-content", true)
  .attr("transform",
   `translate(${margin.LEFT}, ${margin.TOP})`)

// Gentleman
svg1.append("image")
.attr("xlink:href", baseAssetsUrl+"man21.png")
    .attr("x", "-12%")
    .attr("y", "15%")
    .attr("width", 370)
    .attr("height", 370)
    .style("opacity", 1)

// Comment
const annot1 = [
  {
  note: { 
    label: "~20% of respondents (men & women) say that their company doesn't invest in ML at all (no investment or not aware?).",
    title: "No Investment",
    wrap: 100, 
    padding: 5, 
  },
  className: "color",
  connector: { end: "arrow" },
  color: ["#ffffff"],
  x: 755,
  y: 350,
  dy: -35,
  dx: +30
}]

svg1.append("g")
  .style('font-size', 17)
  .call(d3.annotation()
          .annotations(annot1))

d3.csv(baseAssetsUrl + "investm_man.csv").then(function(data){

  const name = "Gentlemen"
  create_graph(data, svg1, name)

});

// ===== WOMEN =====
const svg2 = d3.select("#map21-ladies").append("svg")
// .attr("style", "outline: thin solid red;")
.attr("preserveAspectRatio", "xMinYMin meet")
.attr("viewBox", "0 0 950 500")
.classed("svg-content", true)
.attr("transform",
  `translate(${margin.LEFT}, ${margin.TOP})`)

// Lady
svg2.append("image")
.attr("xlink:href", baseAssetsUrl+"lady21.png")
.attr("x", "-18%")
.attr("y", "5%")
.attr("width", 520)
.attr("height", 520)
.style("opacity", 1)

// Comment
const annot2 = [
  {
  note: { 
    label: "Very few respondents confirm that their company invests more than $10k in Machine Learning.",
    title: "10k+ Investment",
    wrap: 120, 
    padding: 5, 
  },
  className: "color",
  connector: { end: "arrow" },
  color: ["#ffffff"],
  x: 755,
  y: 140,
  dy: +35,
  dx: +30
}]

svg2.append("g")
  .style('font-size', 17)
  .call(d3.annotation()
          .annotations(annot2))

d3.csv(baseAssetsUrl + "investm_woman.csv").then(function(data){

  const name = "Ladies"
  create_graph(data, svg2, name)

});

});
'''


h = display(HTML(htmlt))
j = py_display.Javascript(js_t)
py_display.display_javascript(j)

## 4.3 Manpower within the Estate

<p style="font-family: times-new-roman">A first note here would be that I have explored the <i>activities that make up an important role for a respondent and there are no major changes</i> throughout the years. The most frequent activities remain "building and running Machine Learning services", while the least frequent activity is "performing research to advance the state of the art of Machine Learning".</p>

<p style="font-family: times-new-roman">In regards to the number of people that make up the Data Science teams we can observe the following:</p>

* <p style="font-family: times-new-roman">For both 🎩men and 💃women the <b>team size correlates with the size of the company</b>. There are around 35% of respondents that affirm that the DS team within the company is quite small (less than 4 people).</p>
* <p style="font-family: times-new-roman">On the opposite side there are ~16% (💃ladies) to almost 20% (🎩gents) that state they are working in teams equal or greater than 10 members.</p>
* <p style="font-family: times-new-roman">There is <i>no significant shift within the percentages of categories</i> throughout the years.</p>

In [ ]:
# Activities that make up your job
# Just mention within comments
cols = list(df.columns[113:120])
new_names = ["Build/Run Data Infrastructure",
             "Build prototypes to apply ML",
             "Build/Run a ML service",
             "Experimentationto improve ML models",
             "Research that advances the ML state of art",
             "None", "Other"]

activities = get_data_multiple_answers(cols, new_names)
activities = activities[activities["Year"]!=2017]

activities_man = activities[activities["Gender"] == "Man"].drop(columns=["Gender"])
activities_woman = activities[activities["Gender"] == "Woman"].drop(columns=["Gender"])

activities_man.to_csv("activities_man.csv", index=False)
activities_woman.to_csv("activities_woman.csv", index=False)


# Individuals at work
cols = ["Year", "What is your gender? - Selected Choice",
       df.columns[110]]

teamsize = df[cols]
teamsize.columns = ["Year", "Gender", "TeamSize"]
teamsize = teamsize[teamsize["Gender"].isin(["Man", "Woman"])]

old_list = ['4-Mar', '2-Jan', '0', '9-May', '14-Oct', '20+', '15-19']
new_list = ['3. 3-4 individuals', '2. 1-2 individuals', '1. 0 individuals',
            '4. 5-9 individuals', '5. 10-14 individuals',
            '7. 20+ individuals', '6. 15-19 individuals']
teamsize["TeamSize"] = teamsize["TeamSize"].replace(old_list, new_list)
teamsize = teamsize.value_counts().reset_index()
teamsize = pd.merge(teamsize, total, on=["Year", "Gender"])

teamsize["Perc"] = teamsize[0]/teamsize["Count"]*100
teamsize["Perc"] = teamsize["Perc"].apply(lambda x: round(x, 1))
teamsize.drop(columns=[0, "Count"], axis=1, inplace=True)
teamsize = pd.pivot(teamsize, index=["Year", "Gender"], columns="TeamSize",
                    values="Perc").reset_index()

teamsize_man = teamsize[teamsize["Gender"] == "Man"].drop(columns=["Gender"])
teamsize_woman = teamsize[teamsize["Gender"] == "Woman"].drop(columns=["Gender"])

teamsize_man.to_csv("teamsize_man.csv", index=False)
teamsize_woman.to_csv("teamsize_woman.csv", index=False)

# Save usa.png img as well
im = imageio.imread('../input/kaggle-data-science-survey-20172021/images/man24.png')
Image.fromarray(im).save("man24.png")

im = imageio.imread('../input/kaggle-data-science-survey-20172021/images/lady24.png')
Image.fromarray(im).save("lady24.png")

htmlt = '''
<!-- Bootstrap -->
<link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.4.1/css/bootstrap.min.css" integrity="sha384-Vkoo8x4CGsO3+Hhxv8T/Q5PaXtkKtu6ug5TOeNV6gBiFeWPGFN9MuhOf23Q9Ifjh" crossorigin="anonymous">
<!-- Style -->
<style>
	.all24 {
		font-family: "Times New Roman", Times, serif;
		overflow-x: hidden;
		width: 98%;
		background-color:#e0c9a6;
		/* background-image : url("bkg.jpg"); changed via js */
	}
	.all24 .svg-container {
		display: inline-block;
		position: relative;
		width: 100%;
		padding-bottom: 54%;
		vertical-align: top;
		overflow: hidden;
	}
	.svg-content {
		display: inline-block;
		position: absolute;
		top: 0;
		left: 0;
	}

	.axisHidden path{
		stroke: #e0c9a6;
	}

	.axisHidden line{
		stroke: #e0c9a6;
	}

	.annotation.color text {
		fill: #000000;
	}

	@font-face {
		font-family: parklane;
		src: url(https://h4ks.net/ParkLaneNF.otf);
		}

	div.tooltip-donut {
		position: absolute;
		text-align: center;
		padding: .3rem;
		background: #FFFFFF;
		color: #000000;
		border: 1px solid #000000;
		border-radius: 8px;
		pointer-events: none;
		font-size: 2rem;
		z-index: 99999;
	}

</style>

<img id="baseimg" src="man.png" style="display:none" />
<div class="all24">
	<center><h1 style="font-family: parklane">Data Science Teamsize</h1></center>
	<center><h3><i>- How many individuals are responsible for data science work? -</i></h3></center>
	<div class="row">
		<div id="map24-gents" class="svg-container"></div>
	</div>

	<div class="row">
		<div id="map24-ladies" class="svg-container"></div>
	</div>
</div>

'''

js_t = '''
require.config({
  paths: {
    d3src: "https://d3js.org/",
  },
  map: {
    '*': {
      'd3v6': 'd3src/d3.v6.min',
      'd3-selection': 'd3src/d3-selection.v1.min',
      'd3-drag': 'd3src/d3-drag.v1.min',
      'd3-shape': 'd3src/d3-shape.v1.min',
      'd3-path': 'd3src/d3-path.v1.min',
      'd3-dispatch': 'd3src/d3-dispatch.v1.min',
      'd3-annotation': 'https://cdnjs.cloudflare.com/ajax/libs/d3-annotation/2.5.1/d3-annotation.min.js',
    }
  }
});

require(["d3v6", "d3-annotation"], function(d3, d3Annotation) {

// Inject d3Annotation methods to d3
for (var key in d3Annotation) {
  d3[key] = d3Annotation[key];
}

// Get Assets URL set by Kaggle
const baseAssetsUrl = document.getElementById('baseimg').src.replace(/man.png.*$/, '');

const $all24 = document.getElementsByClassName('all24')[0];
$all24.style.backgroundImage = `url("${baseAssetsUrl}bkg.jpg")`;

// Set the SVG area
const margin = { LEFT: 20, RIGHT: 40, TOP: 15, BOTTOM: 0 }
const width = 950 - margin.LEFT - margin.RIGHT
const height = 500 - margin.TOP - margin.BOTTOM
const chart_size = 40
const x_global = 150

// STREAMGRAPH
function create_graph(data, svg, name){

  const XY_axis = height/2-200

  svg.append("text")
  .attr("transform", `translate(${x_global-60}, ${XY_axis-10})`)
  .attr("font-size", "24px")
  .attr("text-anchor", "middle")
  .attr("font-weight", 600)
  .text(name)

  data.forEach(d => {
    d.Year = Number(d.Year)
  })

  // List of groups
  var keys = data.columns.slice(1)

  // Stacking
  var series = d3.stack()
    .keys(keys)
    .offset(d3.stackOffsetSilhouette)
    (data)

  // X Axis
  var x = d3.scaleLinear()
    .domain(d3.extent(data, d => d.Year))
    .range([0, width/1.5])

  var xAxis = g => g
    .attr("transform", `translate(${x_global}, ${XY_axis})`)
    .call(d3.axisBottom(x)
            .tickSize(height/1.2).ticks(2).tickFormat(d3.format("d")))
    .call(g => g.select(".domain").remove())
    .call(g => g.selectAll(".tick line")
                .attr("stroke", "#3A3A3A")
                .attr('stroke-dasharray', '5 5'))
    .call(g => g.selectAll(".tick")
                .select('text')
                .attr('fill', "#3A3A3A")
                .style("font-size", 15))

  // Y Axis
  var y = d3.scaleLinear()
  .domain([-chart_size, chart_size])
  .range([height/1.2, 0])
  svg.append("g")
  .attr("class", "axisHidden")
  .attr("transform", `translate(${x_global}, ${XY_axis})`)
    .call(d3.axisLeft(y).ticks(0))

  // Area Chart
  var area = d3.area()
  .x(d => x(d.data.Year))
  .y0(d => y(d[0]))
  .y1(d => y(d[1]))

  // Color
  const color = d3.scaleOrdinal()
    .domain(keys)
     .range(["#CB6E77", "#B4414C", "#993336",
     "#CC7550", "#BE5F37", "#CE963B",
     "#5B8668"])

  // Show
  const path = svg.append("g")
    .selectAll("path")
    .data(series)
    .join("path")
    .attr("transform", `translate(${x_global}, ${XY_axis})`)
      .attr("data-label", d => d.key)
      .attr("fill", ({key}) => color(key))
      .attr("stroke", "black")
      .attr("stroke-width", 0.2)
      .attr("d", area)
      .attr("opacity", 0.8)

  svg.append("g")
      .call(xAxis)

  // ==== Interactive ====
  const hover = (svg, path) => {
  
    const line = svg.append("g")
         .attr("display", "none");
  
    line.append("g")
        .selectAll("line")
        .data(series)
        .join("line")
        .attr("class", "cursor-line")
        .attr("fill", "#fff")
        .attr("stroke-width", 0.8)
        .attr("x1", 10)
        .attr("y1", height)
        .attr("x2", 10)
        .attr("y2", 10);

    line.append("text")
      .attr("class", "text-year")
      .attr("font-size", 14)
      .attr("x", 0)
      .attr("y", 26)
      .attr("transform", "rotate(-90 20 20) translate(0, -20)");
    
    line.append("g")
        .selectAll("text")
        .data(series)
        .join("text")
        .attr("class", "text-label")
        .attr("font-weight", 900)
        .attr("font-size", 17)
        .attr("y", height-10)
        .attr("x", -8)
        .attr("transform", 
        "rotate(-90 10 460)");

    line.append("g")
        .selectAll("text")
        .data(data)
        .join("text")
        .attr("class", "text-value")
        .attr("font-weight", 700)
        .attr("font-size", 14)
        .attr("y", height-(height*0.94))
        .attr("x", 14);

    const mousemove = (event) => {
  
      event?.preventDefault();
      const pointer = d3.pointer(event);

      const xm = x.invert(pointer[0]);
      const label = d3.select(event.target).attr("data-label");
  
      line.attr("transform", `translate(${pointer[0]}, 0)`)
          .style("visibility", "inherit");
    
          line.select(".text-year")
          .text(parseInt(xm));
      
      line.selectAll(".text-value")
        .text(d => d.Year === parseInt(xm) ? d[label]+"%" : "");

      line.selectAll(".text-label")
        .style("visibility", "hidden")
        .filter(d => d.key === label)
        .style("visibility", "inherit")
        .attr("fill", d => d.key === label ? color(d.key) : "#000")
        .text(label);
      
      line.selectAll(".cursor-line")
        .attr("stroke", "fff0")
        .filter(d => d.key === label)
        .attr("stroke", d => d.key === label ? color(d.key) : "#fff0")
      
      path.attr("opacity", d => d.key === label ? 0.8 : 0.3);
      
      if (label === null) {
        path.attr("opacity", 0.8);
        line.style("visibility", "hidden");
      }
    }

    const mouseenter = () => {
      line.attr("display", null);
    }
  
    const mouseleave = () => {
      line.attr("display", "none");
    }

    svg
        .on("mousemove", mousemove)
        .on("mouseenter", mouseenter)
        .on("mouseleave", mouseleave)

  }

  svg.call(hover, path);
}

// ===== MEN =====
const svg1 = d3.select("#map24-gents").append("svg")
// .attr("style", "outline: thin solid red;")
  .attr("preserveAspectRatio", "xMinYMin meet")
  .attr("viewBox", "0 0 950 500")
  .classed("svg-content", true)
  .attr("transform",
   `translate(${margin.LEFT}, ${margin.TOP})`)

// Gentleman
svg1.append("image")
.attr("xlink:href", baseAssetsUrl+"man24.png")
    .attr("x", "-6%")
    .attr("y", "13%")
    .attr("width", 380)
    .attr("height", 380)
    .style("opacity", 1)

// Comment
const annot1 = [
  {
  note: { 
    label: "~35% of respondents say that the size of the DS team is small (1-4 individuals) - correlated with business size.",
    title: "Small Teams",
    wrap: 100, 
    padding: 5, 
  },
  className: "color",
  connector: { end: "arrow" },
  color: ["#ffffff"],
  x: 755,
  y: 300,
  dy: -35,
  dx: +30
}]

svg1.append("g")
  .style('font-size', 17)
  .call(d3.annotation()
          .annotations(annot1))

d3.csv(baseAssetsUrl + "teamsize_man.csv").then(function(data){

  const name = "Gentlemen"
  create_graph(data, svg1, name)

});

// ===== WOMEN =====
const svg2 = d3.select("#map24-ladies").append("svg")
// .attr("style", "outline: thin solid red;")
.attr("preserveAspectRatio", "xMinYMin meet")
.attr("viewBox", "0 0 950 500")
.classed("svg-content", true)
.attr("transform",
  `translate(${margin.LEFT}, ${margin.TOP})`)

// Lady
svg2.append("image")
.attr("xlink:href", baseAssetsUrl+"lady24.png")
.attr("x", "-22%")
.attr("y", "3%")
.attr("width", 510)
.attr("height", 510)
.style("opacity", 1)

// Comment
const annot2 = [
  {
  note: { 
    label: "Only ~12% of respondents affirm that the size of the DS team is larger than 20 individuals.",
    title: "Large Teams",
    wrap: 120, 
    padding: 5, 
  },
  className: "color",
  connector: { end: "arrow" },
  color: ["#ffffff"],
  x: 755,
  y: 140,
  dy: +35,
  dx: +30
}]

svg2.append("g")
  .style('font-size', 17)
  .call(d3.annotation()
          .annotations(annot2))

d3.csv(baseAssetsUrl + "teamsize_woman.csv").then(function(data){

  const name = "Ladies"
  create_graph(data, svg2, name)

});

});
'''


h = display(HTML(htmlt))
j = py_display.Javascript(js_t)
py_display.display_javascript(j)

<div class="alert simple-alert" style="font-family: times-new-roman">
  <p style="font-family: times-new-roman"><center>👀 🎩Gents and 💃ladies are found the most within the Academics and Technology industries. The company and team size are split almost 50-50 (between the respondents that answered these questions), meaning that ~half of them are part of small companies and teams, while the other half are part of larger companies and Data Science teams size. These companies don't seem to pay greater attention to ML nor invest more within this discipline in 2021, however the respondent's bias and administrative knowledge might influence this answer. Lastly, there has been a decrease in pay due to an increase in young (fresh) respondents, with some of the best paying jobs (for both genders) being Project Manager and Data Scientist.</center></p>
</div>

# 5. Getting Social - Sharing is Caring

## 5.1 Acquiring Knowledge

<p style="font-family: times-new-roman">The evolution of learning platforms has been as follows:</p>

* <p style="font-family: times-new-roman"><b>Kaggle Learn had the biggest increase in popularity</b> since 2018, having 34% of the 🎩gents and 30% of the 💃ladies saying that they use Kaggle Learn on a regular basis. Another platform that has increased in popularity has been Udemy, but at a slower pace than Kaggle Learn.</p>
* <p style="font-family: times-new-roman">Besides these two, <b>all the other learning platforms have been decreasing</b> in usage in the last 4 years.</p>

In [ ]:
# DS Courses
cols = list(df.columns[203:213])
new_names = ["Coursera", "edX", "Kaggle Learn", "DataCamp",
             "Fast.ai", "Udacity", "Udemy", "LinkedIn Learning",
             "Cloud Certifications", "University"]

courses = get_data_multiple_answers(cols, new_names)
courses = courses[courses["Year"]!=2017]

courses_man = courses[courses["Gender"] == "Man"].drop(columns=["Gender"])
courses_woman = courses[courses["Gender"] == "Woman"].drop(columns=["Gender"])

courses_man.to_csv("courses_man.csv", index=False)
courses_woman.to_csv("courses_woman.csv", index=False)

# Save usa.png img as well
im = imageio.imread('../input/kaggle-data-science-survey-20172021/images/man25.png')
Image.fromarray(im).save("man25.png")

im = imageio.imread('../input/kaggle-data-science-survey-20172021/images/lady25.png')
Image.fromarray(im).save("lady25.png")

htmlt = '''
<!-- Bootstrap -->
<link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.4.1/css/bootstrap.min.css" integrity="sha384-Vkoo8x4CGsO3+Hhxv8T/Q5PaXtkKtu6ug5TOeNV6gBiFeWPGFN9MuhOf23Q9Ifjh" crossorigin="anonymous">
<!-- Style -->
<style>
	.all25 {
		font-family: "Times New Roman", Times, serif;
		overflow-x: hidden;
		width: 98%;
		background-color:#e0c9a6;
		/* background-image : url("bkg.jpg"); changed via js */
	}
	.all25 .svg-container {
		display: inline-block;
		position: relative;
		width: 100%;
		padding-bottom: 54%;
		vertical-align: top;
		overflow: hidden;
	}
	.svg-content {
		display: inline-block;
		position: absolute;
		top: 0;
		left: 0;
	}

	.axisHidden path{
		stroke: #e0c9a6;
	}

	.axisHidden line{
		stroke: #e0c9a6;
	}

	.annotation.color text {
		fill: #000000;
	}

	@font-face {
		font-family: parklane;
		src: url(https://h4ks.net/ParkLaneNF.otf);
		}

	div.tooltip-donut {
		position: absolute;
		text-align: center;
		padding: .3rem;
		background: #FFFFFF;
		color: #000000;
		border: 1px solid #000000;
		border-radius: 8px;
		pointer-events: none;
		font-size: 2rem;
		z-index: 99999;
	}

</style>

<img id="baseimg" src="man.png" style="display:none" />
<div class="all25">
	<center><h1 style="font-family: parklane">Acquiring Knowledge</h1></center>
	<center><h3><i>- What are the most used platforms for learning? -</i></h3></center>
	<div class="row">
		<div id="map25-gents" class="svg-container"></div>
	</div>

	<div class="row">
		<div id="map25-ladies" class="svg-container"></div>
	</div>
</div>
'''

js_t = '''
require.config({
  paths: {
    d3src: "https://d3js.org/",
  },
  map: {
    '*': {
      'd3v6': 'd3src/d3.v6.min',
      'd3-selection': 'd3src/d3-selection.v1.min',
      'd3-drag': 'd3src/d3-drag.v1.min',
      'd3-shape': 'd3src/d3-shape.v1.min',
      'd3-path': 'd3src/d3-path.v1.min',
      'd3-dispatch': 'd3src/d3-dispatch.v1.min',
      'd3-annotation': 'https://cdnjs.cloudflare.com/ajax/libs/d3-annotation/2.5.1/d3-annotation.min.js',
    }
  }
});

require(["d3v6", "d3-annotation"], function(d3, d3Annotation) {

// Inject d3Annotation methods to d3
for (var key in d3Annotation) {
  d3[key] = d3Annotation[key];
}

// Get Assets URL set by Kaggle
const baseAssetsUrl = document.getElementById('baseimg').src.replace(/man.png.*$/, '');

const $all25 = document.getElementsByClassName('all25')[0];
$all25.style.backgroundImage = `url("${baseAssetsUrl}bkg.jpg")`;

// Set the SVG area
const margin = { LEFT: 20, RIGHT: 40, TOP: 15, BOTTOM: 0 }
const width = 950 - margin.LEFT - margin.RIGHT
const height = 500 - margin.TOP - margin.BOTTOM
const chart_size = 130
const x_global = 150

function create_graph(data, svg, name){

  const XY_axis = height/2-200

  svg.append("text")
  .attr("transform", `translate(${x_global-60}, ${XY_axis-10})`)
  .attr("font-size", "24px")
  .attr("text-anchor", "middle")
  .attr("font-weight", 600)
  .text(name)

  data.forEach(d => {
    d.Year = Number(d.Year)
  })

  // List of groups
  var keys = data.columns.slice(1)

  // Stacking
  var series = d3.stack()
    .keys(keys)
    .offset(d3.stackOffsetSilhouette)
    (data)

  // X Axis
  var x = d3.scaleLinear()
    .domain(d3.extent(data, d => d.Year))
    .range([0, width/1.5])

  var xAxis = g => g
    .attr("transform", `translate(${x_global}, ${XY_axis})`)
    .call(d3.axisBottom(x)
            .tickSize(height/1.2).ticks(4).tickFormat(d3.format("d")))
    .call(g => g.select(".domain").remove())
    .call(g => g.selectAll(".tick line")
                .attr("stroke", "#3A3A3A")
                .attr('stroke-dasharray', '5 5'))
    .call(g => g.selectAll(".tick")
                .select('text')
                .attr('fill', "#3A3A3A")
                .style("font-size", 15))

  // Y Axis
  var y = d3.scaleLinear()
  .domain([-chart_size, chart_size])
  .range([height/1.2, 0])
  svg.append("g")
  .attr("class", "axisHidden")
  .attr("transform", `translate(${x_global}, ${XY_axis})`)
    .call(d3.axisLeft(y).ticks(0))

  // Area Chart
  var area = d3.area()
  .x(d => x(d.data.Year))
  .y0(d => y(d[0]))
  .y1(d => y(d[1]))

  // Color
  const color = d3.scaleOrdinal()
    .domain(keys)
     .range(["#C77C90", "#BD5A6E", "#C66E60",
              "#CB905F", "#799E7F", "#249B93",
              "#609CB7", "#357DA7", "#B14E9D",
              "#803871"])

  // Show
  const path = svg.append("g")
    .selectAll("path")
    .data(series)
    .join("path")
    .attr("transform", `translate(${x_global}, ${XY_axis})`)
      .attr("data-label", d => d.key)
      .attr("fill", ({key}) => color(key))
      .attr("stroke", "black")
      .attr("stroke-width", 0.2)
      .attr("d", area)
      .attr("opacity", 0.8)

  svg.append("g")
      .call(xAxis)

  // ==== Interactive ====
  const hover = (svg, path) => {
  
    const line = svg.append("g")
         .attr("display", "none");
  
    line.append("g")
        .selectAll("line")
        .data(series)
        .join("line")
        .attr("class", "cursor-line")
        .attr("fill", "#fff")
        .attr("stroke-width", 0.8)
        .attr("x1", 10)
        .attr("y1", height)
        .attr("x2", 10)
        .attr("y2", 10);

    line.append("text")
      .attr("class", "text-year")
      .attr("font-size", 14)
      .attr("x", 0)
      .attr("y", 26)
      .attr("transform", "rotate(-90 20 20) translate(0, -20)");
    
    line.append("g")
        .selectAll("text")
        .data(series)
        .join("text")
        .attr("class", "text-label")
        .attr("font-weight", 900)
        .attr("font-size", 17)
        .attr("y", height-10)
        .attr("x", -8)
        .attr("transform", 
        "rotate(-90 10 460)");

    line.append("g")
        .selectAll("text")
        .data(data)
        .join("text")
        .attr("class", "text-value")
        .attr("font-weight", 700)
        .attr("font-size", 14)
        .attr("y", height-(height*0.94))
        .attr("x", 14);

    const mousemove = (event) => {
  
      event?.preventDefault();
      const pointer = d3.pointer(event);

      const xm = x.invert(pointer[0]);
      const label = d3.select(event.target).attr("data-label");
  
      line.attr("transform", `translate(${pointer[0]}, 0)`)
          .style("visibility", "inherit");
    
          line.select(".text-year")
          .text(parseInt(xm));
      
      line.selectAll(".text-value")
        .text(d => d.Year === parseInt(xm) ? d[label]+"%" : "");

      line.selectAll(".text-label")
        .style("visibility", "hidden")
        .filter(d => d.key === label)
        .style("visibility", "inherit")
        .attr("fill", d => d.key === label ? color(d.key) : "#000")
        .text(label);
      
      line.selectAll(".cursor-line")
        .attr("stroke", "fff0")
        .filter(d => d.key === label)
        .attr("stroke", d => d.key === label ? color(d.key) : "#fff0")
      
      path.attr("opacity", d => d.key === label ? 0.8 : 0.3);
      
      if (label === null) {
        path.attr("opacity", 0.8);
        line.style("visibility", "hidden");
      }
    }

    const mouseenter = () => {
      line.attr("display", null);
    }
  
    const mouseleave = () => {
      line.attr("display", "none");
    }

    svg
        .on("mousemove", mousemove)
        .on("mouseenter", mouseenter)
        .on("mouseleave", mouseleave)

  }

  svg.call(hover, path);
}

// ===== MEN =====
const svg1 = d3.select("#map25-gents").append("svg")
// .attr("style", "outline: thin solid red;")
  .attr("preserveAspectRatio", "xMinYMin meet")
  .attr("viewBox", "0 0 950 500")
  .classed("svg-content", true)
  .attr("transform",
   `translate(${margin.LEFT}, ${margin.TOP})`)

// Gentleman
svg1.append("image")
.attr("xlink:href", baseAssetsUrl+"man25.png")
    .attr("x", "-11%")
    .attr("y", "12%")
    .attr("width", 380)
    .attr("height", 380)
    .style("opacity", 1)

// Comment
const annot1 = [
  {
  note: { 
    label: "Has gradually increased in popularity, from 17% usage in 2018 to ~34% in 2021.",
    title: "Kaggle Learn",
    wrap: 120, 
    padding: 5, 
  },
  className: "color",
  connector: { end: "arrow" },
  color: ["#ffffff"],
  x: 750,
  y: 280,
  dy: -20,
  dx: +25
}]

svg1.append("g")
  .style('font-size', 17)
  .call(d3.annotation()
          .annotations(annot1))

d3.csv(baseAssetsUrl + "courses_man.csv").then(function(data){

  const name = "Gentlemen"
  create_graph(data, svg1, name)

});

// ===== WOMEN =====
const svg2 = d3.select("#map25-ladies").append("svg")
// .attr("style", "outline: thin solid red;")
.attr("preserveAspectRatio", "xMinYMin meet")
.attr("viewBox", "0 0 950 500")
.classed("svg-content", true)
.attr("transform",
  `translate(${margin.LEFT}, ${margin.TOP})`)

// Lady
svg2.append("image")
.attr("xlink:href", baseAssetsUrl+"lady25.png")
.attr("x", "-16%")
.attr("y", "4%")
.attr("width", 500)
.attr("height", 500)
.style("opacity", 1)

// Comment
const annot2 = [
  {
  note: { 
    label: "The classic learning within Universities is not as popular as the other online platforms and seems to slightly decrease in usage in 2021.",
    title: "University Courses",
    wrap: 120, 
    padding: 5, 
  },
  className: "color",
  connector: { end: "arrow" },
  color: ["#ffffff"],
  x: 750,
  y: 130,
  dy: +20,
  dx: +25
}]

svg2.append("g")
  .style('font-size', 17)
  .call(d3.annotation()
          .annotations(annot2))

d3.csv(baseAssetsUrl + "courses_woman.csv").then(function(data){

  const name = "Ladies"
  create_graph(data, svg2, name)

});

});
'''


h = display(HTML(htmlt))
j = py_display.Javascript(js_t)
py_display.display_javascript(j)

## 5.2 Sharing and Contribution

* <p style="font-family: times-new-roman"><b>GitHub, Kaggle and Colab are the most popular spaces</b> out of all to share Data Science work (for both genders). All three mentioned have been increasing in usage in 2021 vs 2020.</p>
* <p style="font-family: times-new-roman">Worth mentioning is that there are <i>fewer women</i> (in percentage points, not in absolute numbers) that answered this question versus men.</p>

In [ ]:
# Community share
cols = list(df.columns[193:201])
new_names = ["Plotly Dash", "Streamlit", "NBViewer", "GitHub",
             "Personal Blog", "Kaggle", "Colab", "Shiny"]

share = get_data_multiple_answers(cols, new_names)
share = share[share["Year"]>2019]
share.columns = ['Year', 'Gender', 'PlotlyDash', 'Streamlit', 'NBViewer', 'GitHub',
       'PersonalBlog', 'Kaggle', 'Colab', 'Shiny']

share_man = share[share["Gender"] == "Man"].drop(columns=["Gender"])
share_man = share_man.sort_values(share_man.last_valid_index(),
                                      axis=1, ascending=False)
share_woman = share[share["Gender"] == "Woman"].drop(columns=["Gender"])
share_woman = share_woman.sort_values(share_woman.last_valid_index(),
                                      axis=1, ascending=False)

share_man.to_csv("share_man.csv", index=False)
share_woman.to_csv("share_woman.csv", index=False)

# Save usa.png img as well
im = imageio.imread('../input/kaggle-data-science-survey-20172021/images/man26.png')
Image.fromarray(im).save("man26.png")

im = imageio.imread('../input/kaggle-data-science-survey-20172021/images/lady26.png')
Image.fromarray(im).save("lady26.png")

htmlt = '''
<!-- Bootstrap -->
<link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.4.1/css/bootstrap.min.css" integrity="sha384-Vkoo8x4CGsO3+Hhxv8T/Q5PaXtkKtu6ug5TOeNV6gBiFeWPGFN9MuhOf23Q9Ifjh" crossorigin="anonymous">
<!-- Style -->
<style>
	.all26 {
		font-family: "Times New Roman", Times, serif;
		overflow-x: hidden;
		width: 98%;
		background-color:#e0c9a6;
		/* background-image : url("bkg.jpg"); changed via js */
	}
	.all26 .svg-container {
		display: inline-block;
		position: relative;
		width: 100%;
		padding-bottom: 80%;
		vertical-align: top;
		overflow: hidden;
	}
	.svg-content {
		display: inline-block;
		position: absolute;
		top: 0;
		left: 0;
	}

	.axisHidden path{
		stroke: #e0c9a6;
	}

	.axisHidden line{
		stroke: #e0c9a6;
	}

	.annotation.color text {
		fill: #000000;
	}

	@font-face {
		font-family: parklane;
		src: url(https://h4ks.net/ParkLaneNF.otf);
		}

	div.tooltip-donut {
		position: absolute;
		text-align: center;
		padding: .3rem;
		background: #FFFFFF;
		color: #000000;
		border: 1px solid #000000;
		border-radius: 8px;
		pointer-events: none;
		font-size: 2rem;
		z-index: 99999;
	}

</style>

<img id="baseimg" src="man.png" style="display:none" />
<div class="all26">
	<center><h1 style="font-family: parklane">Sharing is Caring</h1></center>
	<center><h3><i>- Where do you publicly share your work? -</i></h3></center>
	<div class="row">
		<div id="map26" class="svg-container"></div>
	</div>
</div>
'''

js_t = '''
require.config({
  paths: {
    d3src: "https://d3js.org/",
  },
  map: {
    '*': {
      'd3v6': 'd3src/d3.v6.min',
      'd3-selection': 'd3src/d3-selection.v1.min',
      'd3-drag': 'd3src/d3-drag.v1.min',
      'd3-shape': 'd3src/d3-shape.v1.min',
      'd3-path': 'd3src/d3-path.v1.min',
      'd3-dispatch': 'd3src/d3-dispatch.v1.min',
      'd3-annotation': 'https://cdnjs.cloudflare.com/ajax/libs/d3-annotation/2.5.1/d3-annotation.min.js',
    }
  }
});

require(["d3v6", "d3-annotation"], function(d3, d3Annotation) {

// Inject d3Annotation methods to d3
for (var key in d3Annotation) {
  d3[key] = d3Annotation[key];
}

// Get Assets URL set by Kaggle
const baseAssetsUrl = document.getElementById('baseimg').src.replace(/man.png.*$/, '');

const $all26 = document.getElementsByClassName('all26')[0];
$all26.style.backgroundImage = `url("${baseAssetsUrl}bkg.jpg")`;


// Set the SVG area
const margin = { LEFT: 20, RIGHT: 40, TOP: 15, BOTTOM: 0 }
const width = 950 - margin.LEFT - margin.RIGHT
const height = 700 - margin.TOP - margin.BOTTOM
const centreSpacing = 100

const svg = d3.select("#map26").append("svg")
// .attr("style", "outline: thin solid red;")
  .attr("preserveAspectRatio", "xMinYMin meet")
  .attr("viewBox", "0 0 950 700")
  .classed("svg-content", true)
  .attr("transform",
   `translate(${margin.LEFT}, ${margin.TOP})`)

// Gentleman
svg.append("image")
.attr("xlink:href", baseAssetsUrl+"man26.png")
    .attr("x", "19%")
    .attr("y", "8.5%")
    .attr("width", 250)
    .attr("height", 250)
    .style("opacity", 1)

// Lady
svg.append("image")
.attr("xlink:href", baseAssetsUrl+"lady26.png")
.attr("x", "48%")
.attr("y", "24%")
.attr("width", 310)
.attr("height", 310)
.style("opacity", 1)


function create_stacked_bar(data, subgroups, x_global, y_range, color) {

  const groups = data.map(d => d.Year)

  // Add X axis
  const x = d3.scaleBand()
      .domain(groups)
      .range([0, width/2.5])
      .padding([0.2])
  svg.append("g")
    .attr("transform", `translate(${x_global}, ${height/1.15})`)
    .call(d3.axisBottom(x).tickSizeOuter(0))
    .selectAll("text")
      .attr("y", "10")
      .attr("x", "0")
      .attr("font-size", "15px")
      .attr("font-weight", 300)
      .attr("text-anchor", "middle")

  // Add Y axis
  const y = d3.scaleLinear()
    .domain([0, y_range])
    .range([ height/2.3, 0]);
  svg.append("g")
  .attr("class", "axisHidden")
  .attr("transform", `translate(${x_global}, ${height/2-50})`)
    .call(d3.axisLeft(y).ticks(0))
    .selectAll("text")
      .attr("font-size", "11px")
      .attr("font-weight", 300);

  //s Stack data
  const stackedData = d3.stack()
    .keys(subgroups)
    (data)

  // Show the bars
  const bars = svg.append("g")
    .selectAll("g")
    .data(stackedData)
    .join("g")
    .attr("transform", `translate(${x_global}, ${height/2-50})`)
      .attr("fill", d => color(d.key))
      .attr("stroke", "black")
      .attr("stroke-width", 0.2)
      .attr("class", d => "myRect " + d.key ) 
      .selectAll("rect")
      .data(d => d)
      .join("rect")
        .attr("x", d => x(d.data.Year))
        .attr("y", d => y(d[1]))
        .attr("height", d => y(d[0]) - y(d[1]))
        .attr("width",x.bandwidth())

  // Interactive
  var div = d3.select("body").append("div")
  .attr("class", "tooltip-donut")
  .style("opacity", 0);

  bars
    .on('mouseenter', function (event, dt) {
    // MOUSE ON
    const subGroupName = d3.select(this.parentNode).datum().key 
          
    d3.selectAll(".myRect").style("opacity", 0.2)  
          
    d3.selectAll("."+subGroupName).style("opacity",1)


    // Makes the new div appear
    div.transition()
        .duration(50)
        .style("opacity", 1);

    const subgroupName = d3.select(this.parentNode).datum().key;
    const subgroupValue = dt.data[subgroupName];

    div.html(subgroupName + "<br>" + "Avg: " + subgroupValue +"%")
        .style("left", (event.pageX) + "px")
        .style("top", (event.pageY-30) + "px");


  })
    // MOUSE LEAVE
    .on('mouseleave', function () {
      d3.selectAll(".myRect")
          .style("opacity",1) 

      // New div dissapears
      div.transition()
          .duration('50')
          .style("opacity", 0);
      svg.selectAll('.divergence').remove()
    })

}


// === Preferences ===
d3.csv(baseAssetsUrl + "share_man.csv").then(function(data){

  const x_global = 50
  const y_range = 50
  const subgroups = ['GitHub', 'Kaggle', 'Colab',
   'PersonalBlog', 'Streamlit',
  'Shiny', 'PlotlyDash', 'NBViewer']
  const color = d3.scaleOrdinal()
    .domain(subgroups)
    .range(["#609CB7", "#249B93", "#799E7F", 
    "#CB905F", "#C66E60", "#BD5A6E", "#C77C90", "#E0B8C2"]);

  svg.append("text")
  .attr("y", "5%")
  .attr("x", "25%")
  .attr("font-size", "25px")
  .attr("text-anchor", "middle")
  .attr("font-weight", 600)
  .text("Gentlemen: Preferences Percentages")

  create_stacked_bar(data, subgroups, x_global, y_range, color)
  
});

d3.csv(baseAssetsUrl + "share_woman.csv").then(function(data){

  const x_global = 500
  const y_range = 50
  const subgroups = ['GitHub', 'Kaggle', 'Colab', 
  'PersonalBlog', 'PlotlyDash',
  'Streamlit', 'Shiny', 'NBViewer']
  const color = d3.scaleOrdinal()
    .domain(subgroups)
    .range(["#609CB7", "#249B93", "#799E7F", 
    "#CB905F", "#C77C90", "#C66E60", "#BD5A6E", "#E0B8C2"]);

  svg.append("text")
  .attr("y", "5%")
  .attr("x", "70%")
  .attr("font-size", "25px")
  .attr("text-anchor", "middle")
  .attr("font-weight", 600)
  .text("Ladies: Preferences Percentages")

  create_stacked_bar(data, subgroups, x_global, y_range, color)
  
});

const annot2 = [
  {
  note: { 
    label: "Biggest sharing spaces are GitHub, Kaggle and Colab, all of whom have increased in popularity in 2021.",
    title: "Gents",
    wrap: 400, 
    padding: 5, 
  },
  className: "color",
  // connector: { end: "arrow" },
  color: ["#ffffff"],
  x: 50,
  y: 630,
  dy: +0,
  dx: +0
},
{
  note: { 
    label: "Same trends as for gents, although there is a smaller perc. of overall women that answered this question.",
    title: "Ladies",
    wrap: 400, 
    padding: 5, 
  },
  className: "color",
  // connector: { end: "arrow" },
  color: ["#ffffff"],
  x: 500,
  y: 630,
  dy: +0,
  dx: +0
}]

svg.append("g")
  .style('font-size', 18)
  .call(d3.annotation()
          .annotations(annot2))


});
'''


h = display(HTML(htmlt))
j = py_display.Javascript(js_t)
py_display.display_javascript(j)

## 5.3 Where the gossip is at

* <p style="font-family: times-new-roman"><b>Kaggle and Youtube</b> remain in 2021 (as in the past years too) the <b>most used platforms</b> for gathering information, news and interesting events about Data Science. I personally would have expected Twitter to be very used as well, but it has only 15% usage for gents and ~10% for ladies.</p>
* <p style="font-family: times-new-roman">For both 💃ladies and 🎩gents there has been an <b>overall decrease within almost all media sources</b>, due to a decrease in the percentage of respondents that answered this question. However, Youtube and Newsletters are the only two platforms that had a slight increase in popularity in 2021 compared to the last years.</p>

In [ ]:
# Favorite Media Sources
cols = list(df.columns[216:226])
new_names = ["Twitter", "Newsletter", "Reddit", "Kaggle",
             "Course Forums", "Youtube", "Podcasts", "Blogs",
             "Journal Publications", "Slack"]

media = get_data_multiple_answers(cols, new_names)
media = media[media["Year"]!=2017]
media = media[media["Year"]!=2018]

media_man = media[media["Gender"] == "Man"].drop(columns=["Gender"])
media_woman = media[media["Gender"] == "Woman"].drop(columns=["Gender"])

media_man.to_csv("media_man.csv", index=False)
media_woman.to_csv("media_woman.csv", index=False)

# Save usa.png img as well
im = imageio.imread('../input/kaggle-data-science-survey-20172021/images/man27.png')
Image.fromarray(im).save("man27.png")

im = imageio.imread('../input/kaggle-data-science-survey-20172021/images/lady27.png')
Image.fromarray(im).save("lady27.png")

htmlt = '''
<!-- Bootstrap -->
<link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.4.1/css/bootstrap.min.css" integrity="sha384-Vkoo8x4CGsO3+Hhxv8T/Q5PaXtkKtu6ug5TOeNV6gBiFeWPGFN9MuhOf23Q9Ifjh" crossorigin="anonymous">
<!-- Style -->
<style>
	.all27 {
		font-family: "Times New Roman", Times, serif;
		overflow-x: hidden;
		width: 98%;
		background-color:#e0c9a6;
		/* background-image : url("bkg.jpg"); changed via js */
	}
	.all27 .svg-container {
		display: inline-block;
		position: relative;
		width: 100%;
		padding-bottom: 54%;
		vertical-align: top;
		overflow: hidden;
	}
	.svg-content {
		display: inline-block;
		position: absolute;
		top: 0;
		left: 0;
	}

	.axisHidden path{
		stroke: #e0c9a6;
	}

	.axisHidden line{
		stroke: #e0c9a6;
	}

	.annotation.color text {
		fill: #000000;
	}

	@font-face {
		font-family: parklane;
		src: url(https://h4ks.net/ParkLaneNF.otf);
		}

	div.tooltip-donut {
		position: absolute;
		text-align: center;
		padding: .3rem;
		background: #FFFFFF;
		color: #000000;
		border: 1px solid #000000;
		border-radius: 8px;
		pointer-events: none;
		font-size: 2rem;
		z-index: 99999;
	}

</style>

<img id="baseimg" src="man.png" style="display:none" />
<div class="all27">
	<center><h1 style="font-family: parklane">Keeping Conversations</h1></center>
	<center><h3><i>- What are your favorite media sources? -</i></h3></center>
	<div class="row">
		<div id="map27-gents" class="svg-container"></div>
	</div>

	<div class="row">
		<div id="map27-ladies" class="svg-container"></div>
	</div>
</div>
'''

js_t = '''
require.config({
  paths: {
    d3src: "https://d3js.org/",
  },
  map: {
    '*': {
      'd3v6': 'd3src/d3.v6.min',
      'd3-selection': 'd3src/d3-selection.v1.min',
      'd3-drag': 'd3src/d3-drag.v1.min',
      'd3-shape': 'd3src/d3-shape.v1.min',
      'd3-path': 'd3src/d3-path.v1.min',
      'd3-dispatch': 'd3src/d3-dispatch.v1.min',
      'd3-annotation': 'https://cdnjs.cloudflare.com/ajax/libs/d3-annotation/2.5.1/d3-annotation.min.js',
    }
  }
});

require(["d3v6", "d3-annotation"], function(d3, d3Annotation) {

// Inject d3Annotation methods to d3
for (var key in d3Annotation) {
  d3[key] = d3Annotation[key];
}

// Get Assets URL set by Kaggle
const baseAssetsUrl = document.getElementById('baseimg').src.replace(/man.png.*$/, '');

const $all27 = document.getElementsByClassName('all27')[0];
$all27.style.backgroundImage = `url("${baseAssetsUrl}bkg.jpg")`;

// Set the SVG area
const margin = { LEFT: 20, RIGHT: 40, TOP: 15, BOTTOM: 0 }
const width = 950 - margin.LEFT - margin.RIGHT
const height = 500 - margin.TOP - margin.BOTTOM
const chart_size = 130
const x_global = 150

// STREAMGRAPH
function create_graph(data, svg, name){

  const XY_axis = height/2-200

  svg.append("text")
  .attr("transform", `translate(${x_global-60}, ${XY_axis-10})`)
  .attr("font-size", "24px")
  .attr("text-anchor", "middle")
  .attr("font-weight", 600)
  .text(name)

  data.forEach(d => {
    d.Year = Number(d.Year)
  })

  // List of groups
  var keys = data.columns.slice(1)

  // Stacking
  var series = d3.stack()
    .keys(keys)
    .offset(d3.stackOffsetSilhouette)
    (data)

  // X Axis
  var x = d3.scaleLinear()
    .domain(d3.extent(data, d => d.Year))
    .range([0, width/1.5])

  var xAxis = g => g
    .attr("transform", `translate(${x_global}, ${XY_axis})`)
    .call(d3.axisBottom(x)
            .tickSize(height/1.2).ticks(2).tickFormat(d3.format("d")))
    .call(g => g.select(".domain").remove())
    .call(g => g.selectAll(".tick line")
                .attr("stroke", "#3A3A3A")
                .attr('stroke-dasharray', '5 5'))
    .call(g => g.selectAll(".tick")
                .select('text')
                .attr('fill', "#3A3A3A")
                .style("font-size", 15))

  // Y Axis
  var y = d3.scaleLinear()
  .domain([-chart_size, chart_size])
  .range([height/1.2, 0])
  svg.append("g")
  .attr("class", "axisHidden")
  .attr("transform", `translate(${x_global}, ${XY_axis})`)
    .call(d3.axisLeft(y).ticks(0))

  // Area Chart
  var area = d3.area()
  .x(d => x(d.data.Year))
  .y0(d => y(d[0]))
  .y1(d => y(d[1]))

  // Color
  const color = d3.scaleOrdinal()
    .domain(keys)
     .range(["#C77C90", "#BD5A6E", "#C66E60",
     "#CB905F", "#799E7F", "#249B93",
     "#609CB7", "#357DA7", "#B14E9D",
     "#803871"])

  // Show
  const path = svg.append("g")
    .selectAll("path")
    .data(series)
    .join("path")
    .attr("transform", `translate(${x_global}, ${XY_axis})`)
      .attr("data-label", d => d.key)
      .attr("fill", ({key}) => color(key))
      .attr("stroke", "black")
      .attr("stroke-width", 0.2)
      .attr("d", area)
      .attr("opacity", 0.8)

  svg.append("g")
      .call(xAxis)

  // ==== Interactive ====
  const hover = (svg, path) => {
  
    const line = svg.append("g")
         .attr("display", "none");
  
    line.append("g")
        .selectAll("line")
        .data(series)
        .join("line")
        .attr("class", "cursor-line")
        .attr("fill", "#fff")
        .attr("stroke-width", 0.8)
        .attr("x1", 10)
        .attr("y1", height)
        .attr("x2", 10)
        .attr("y2", 10);

    line.append("text")
      .attr("class", "text-year")
      .attr("font-size", 14)
      .attr("x", 0)
      .attr("y", 26)
      .attr("transform", "rotate(-90 20 20) translate(0, -20)");
    
    line.append("g")
        .selectAll("text")
        .data(series)
        .join("text")
        .attr("class", "text-label")
        .attr("font-weight", 900)
        .attr("font-size", 17)
        .attr("y", height-10)
        .attr("x", -8)
        .attr("transform", 
        "rotate(-90 10 460)");

    line.append("g")
        .selectAll("text")
        .data(data)
        .join("text")
        .attr("class", "text-value")
        .attr("font-weight", 700)
        .attr("font-size", 14)
        .attr("y", height-(height*0.94))
        .attr("x", 14);

    const mousemove = (event) => {
  
      event?.preventDefault();
      const pointer = d3.pointer(event);

      const xm = x.invert(pointer[0]);
      const label = d3.select(event.target).attr("data-label");
  
      line.attr("transform", `translate(${pointer[0]}, 0)`)
          .style("visibility", "inherit");
    
          line.select(".text-year")
          .text(parseInt(xm));
      
      line.selectAll(".text-value")
        .text(d => d.Year === parseInt(xm) ? d[label]+"%" : "");

      line.selectAll(".text-label")
        .style("visibility", "hidden")
        .filter(d => d.key === label)
        .style("visibility", "inherit")
        .attr("fill", d => d.key === label ? color(d.key) : "#000")
        .text(label);
      
      line.selectAll(".cursor-line")
        .attr("stroke", "fff0")
        .filter(d => d.key === label)
        .attr("stroke", d => d.key === label ? color(d.key) : "#fff0")
      
      path.attr("opacity", d => d.key === label ? 0.8 : 0.3);
      
      if (label === null) {
        path.attr("opacity", 0.8);
        line.style("visibility", "hidden");
      }
    }

    const mouseenter = () => {
      line.attr("display", null);
    }
  
    const mouseleave = () => {
      line.attr("display", "none");
    }

    svg
        .on("mousemove", mousemove)
        .on("mouseenter", mouseenter)
        .on("mouseleave", mouseleave)

  }

  svg.call(hover, path);
}

// ===== MEN =====
const svg1 = d3.select("#map27-gents").append("svg")
// .attr("style", "outline: thin solid red;")
  .attr("preserveAspectRatio", "xMinYMin meet")
  .attr("viewBox", "0 0 950 500")
  .classed("svg-content", true)
  .attr("transform",
   `translate(${margin.LEFT}, ${margin.TOP})`)

// Gentleman
svg1.append("image")
.attr("xlink:href", baseAssetsUrl+"man27.png")
    .attr("x", "-12%")
    .attr("y", "9%")
    .attr("width", 400)
    .attr("height", 400)
    .style("opacity", 1)

// Comment
const annot1 = [
  {
  note: { 
    label: "There is an overall decrease within the media sources - due to a decrease in the percetange of people that answered this question.",
    title: "Overall decrease",
    wrap: 100, 
    padding: 5, 
  },
  className: "color",
  // connector: { end: "arrow" },
  color: ["#ffffff"],
  x: 780,
  y: 380,
  dy: -1,
  dx: +1
}]

svg1.append("g")
  .style('font-size', 17)
  .call(d3.annotation()
          .annotations(annot1))

d3.csv(baseAssetsUrl + "media_man.csv").then(function(data){

  const name = "Gentlemen"
  create_graph(data, svg1, name)

});

// ===== WOMEN =====
const svg2 = d3.select("#map27-ladies").append("svg")
// .attr("style", "outline: thin solid red;")
.attr("preserveAspectRatio", "xMinYMin meet")
.attr("viewBox", "0 0 950 500")
.classed("svg-content", true)
.attr("transform",
  `translate(${margin.LEFT}, ${margin.TOP})`)

// Lady
svg2.append("image")
.attr("xlink:href", baseAssetsUrl+"lady27.png")
.attr("x", "-18%")
.attr("y", "5%")
.attr("width", 520)
.attr("height", 520)
.style("opacity", 1)

// Comment
const annot2 = [
  {
  note: { 
    label: "Youtube & Newsletters are the only sources that increased in popularity in 2021 vs 2019.",
    title: "Youtube",
    wrap: 120, 
    padding: 5, 
  },
  className: "color",
  connector: { end: "arrow" },
  color: ["#ffffff"],
  x: 755,
  y: 230,
  dy: +35,
  dx: +30
}]

svg2.append("g")
  .style('font-size', 17)
  .call(d3.annotation()
          .annotations(annot2))

d3.csv(baseAssetsUrl + "media_woman.csv").then(function(data){

  const name = "Ladies"
  create_graph(data, svg2, name)

});

});
'''


h = display(HTML(htmlt))
j = py_display.Javascript(js_t)
py_display.display_javascript(j)

<div class="alert simple-alert" style="font-family: times-new-roman">
  <p style="font-family: times-new-roman"><center>👀 Kaggle has the general lead in all 3 social categories - learning, sharing DS work and media source - and it increased in all categories in 2021 vs the last years. GitHub and Colab are the other two very popular sharing platforms, while as for media sources we can affirm that Youtube and Personal Blogs remain in lead.</center></p>
</div>

# 6. Ending the Party in Style

<center><img src="https://i.imgur.com/seDTn13.gif" width=800></center>

<p style="font-family: times-new-roman">A dataset is very volatile. In my time as a Data Analyst, and then Data Scientist, there is one thing I can say I have discovered and I live by: looking at a dataset is not a straightforward recipe. A dataset is like a living, breathing organism.</p>

<p style="font-family: times-new-roman">Sometimes it confirms some conclusions that you maybe might have already known, or let's say had a hunch, an intuition on them. Sometimes it does the exact opposite, giving the reader some insights they would have never expected. Sometimes it even shocks you. And other times it doesn't really say anything. Nothing new, nothing you already knew, nothing at all. But in these "nothing" insights there are still some conclusions to be taken, even actions to be made.</p>

<p style="font-family: times-new-roman">The following are some of my own, personal insights and findings on this beautiful dataset. However, I would encourage you to read and draw you own conclusions, even challenge mine. A healthy debate brings us more forward than always agreeing with one another - right? :)</p>

<div class="row">
  <div class="col-sm-4">
    <center><img src="https://i.imgur.com/CPggnbs.png" width=100></center>
  </div>
    
  <div class="col-sm-8">
      <p style="font-family: times-new-roman"><b>💃 Ladies</b></p>
    <p style="font-family: times-new-roman">As we have seen so far throughout the analysis, there are no major differences between ladies and gents in terms of preferences towards tools. I will be honest, I have expected to see many more dissimilarities than I am seeing now, however ladies and gents do seem to have in common much more than one would anticipate.</p>
    <p style="font-family: times-new-roman">The percentage of women that responded to the survey has maintained through time at around 20%. Most ladies are from USA, India and more recently from Nigeria, Egypt and Indonesia. They work as Data Scientists, Data Analysts or Business Analysts in companies oriented towards Tech and Medical industries, or they can be found studying/teaching within Academia. </p>
    <p style="font-family: times-new-roman">Regarding the tools and skills, ladies are very fond of Python and are starting to rapidly lose interest in R. They do use less special hardware and have fewer years of experience in coding due to the nature of their role/job, which doesn't involve mandatory coding practice.</p>
  </div>
</div>

<p style="font-family: times-new-roman"></p>

<div class="row">
  <div class="col-sm-4">
    <center><img src="https://i.imgur.com/3oPhb3k.png" width=100></center>
  </div>
    
  <div class="col-sm-8">
      <p style="font-family: times-new-roman"><b>🎩 Gentlemen</b></p>
    <p style="font-family: times-new-roman">Gentlemen are very similar to ladies in terms of preferences and used tools. However, they do have a few areas where they stand out, having different aspects than the ladies. </p>
    <p style="font-family: times-new-roman">Around 80% of men are responding to the survey each year, with this number maintaining throughout all 5 analyzed years. Usually, they have rezidency within India, USA, Japan, with a big surge in people from Nigeria and Pakistan. Most of the gents are Data Scientists or Software Engineers working within the Tech and Finance industries, or they are teachers/students within Academia. </p>
    <p style="font-family: times-new-roman">Python is and remains king in the programming languages arena, and they do seem to be liking to work more with specialized hardware, such as GPU and TPU. Gents do tend to have a bit more seniority within the machine learning and coding arena, as their role within companies requires more necessary knowledge on this part.</p>
  </div>
</div>

<p style="font-family: times-new-roman"></p>

<div class="row">
  <div class="col-sm-4">
    <center><img src="https://i.imgur.com/djTPD9J.png" width=300></center>
  </div>
    
  <div class="col-sm-8">
      <p style="font-family: times-new-roman"><b>🕒 Time</b></p>
    <p style="font-family: times-new-roman">Many of the shifts observed within the evolutions were affected by the increase in youth (people aged 18 to 24 years old). Hence, the decrease we have observed in Doctorate/ Masters respondents, in overall coding experience and in income were naturally influenced by this aspect.
</p>
    <p style="font-family: times-new-roman">For programming languages we see more activity within C/C++, while R is losing popularity each year. Visual Studio is also enjoying an increase in usage, vs Jupyter who had a dip in 2021. TPU is growing more and more popular each year, and it seems that it does so to the detriment of GPU, which is slowly decreasing. Another algorithm that had a steep increase is the Transformer (e.g.: BERT).</p>
    <p style="font-family: times-new-roman">Regarding the companies and their structure, the respondents seem to be split quite in the middle: around half saying they work in small companies and teams, while the other stated that they work in larger companies and teams (with no apparent change through time). However, I was surprised that the interest and investment in ML don't tend to increase throughout the years and it doesn't correlate with the company size - meaning that larger companies don't seem to necessarily invest more in ML.</p>
  </div>
</div>


# 7. Giving Thanks

* <p style="font-family: times-new-roman">Font Generator for ParkLane (Great Gatsby Font). (n.d.). Font Generator. Retrieved November 19, 2021, from https://www.font-generator.com/fonts/ParkLane/?size=58&color=000000&bg=none</p>
* <p style="font-family: times-new-roman">Sources for graph images. (n.d.). Pinterest Board. Retrieved November 19, 2021, from https://ro.pinterest.com/andrada_teodora/1920/</p>
* <p style="font-family: times-new-roman">Templates for the D3 Graphs. (n.d.). The D3.Js Graph Gallery. Retrieved November 22, 2021, from https://www.d3-graph-gallery.com/index.html</p>
* <p style="font-family: times-new-roman"><b>Special loving thanks:</b> to my amazing partner, who has been extremely supportive and eager to help in this analysis. This notebook would have never looked like it does now without him, and for his patience and kindness, I am forever grateful.</p>


## My Specs

> <p style="font-family: times-new-roman">My <a href="https://wandb.ai/andrada/kaggle-survey-2021?workspace=user-andrada">W&B Dashboard</a> is growing:</p>
<center><img src="https://i.imgur.com/EcQQ3IN.gif" width=800></center>

<center><img src="https://i.imgur.com/cUQXtS7.png"></center>

* <p style="font-family: times-new-roman">🖥 Z8 G4: Workstation</p>
* <p style="font-family: times-new-roman">💾 2 CPUs & 96GB Memory</p>
* <p style="font-family: times-new-roman">🎮 2x NVIDIA A6000</p>
* <p style="font-family: times-new-roman">💻 Zbook Studio G7 on the go</p>